In [3]:
"""
Enhanced Robust GP Optimizer with Warm-Start & 3-Way Comparison

NEW FEATURES:
1. Warm-start from reference parameters (baseline-aware)
2. Multi-fidelity optimization (30d → 180d with separate tracking)
3. Thompson sampling exploration
4. 3-way comparison: high-res vs low-res default vs low-res optimized (contourf plots)
5. Comprehensive visualization with improvement metrics
6. Seed-robust initialization using multiple complementary random sequences
7. Separate best-loss tracking per fidelity level (no confusing jumps!)
"""

import numpy as np
import pickle
import os
import warnings
from scipy.stats import qmc, norm
from scipy.optimize import minimize
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, RBF, ConstantKernel, WhiteKernel
from sklearn.linear_model import LinearRegression
from qg_model import QGTwoLayerModel
from scipy.ndimage import uniform_filter
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.patches import Rectangle
import seaborn as sns

warnings.filterwarnings('ignore', category=UserWarning, module='sklearn.gaussian_process')
warnings.filterwarnings('ignore', message='The optimal value found for dimension')

class Colors:
    """Compact color printing"""
    @staticmethod
    def green(t): return f"\033[92m{t}\033[0m"
    @staticmethod
    def cyan(t): return f"\033[96m{t}\033[0m"
    @staticmethod
    def yellow(t): return f"\033[93m{t}\033[0m"
    @staticmethod
    def red(t): return f"\033[91m{t}\033[0m"
    @staticmethod
    def bold(t): return f"\033[1m{t}\033[0m"
    @staticmethod
    def star(t): return f"\033[93m\033[1m★ {t}\033[0m"

# Parameter configuration with REFERENCE BASELINE
PARAM_BOUNDS = {
    'viscosity_scale': {'bounds': (0.5, 5.0), 'type': 'linear'},
    'drag_scale': {'bounds': (0.5, 3.0), 'type': 'linear'},
    'eddy_diffusivity': {'bounds': (1e3, 1e5), 'type': 'log'},
    'smagorinsky_coeff': {'bounds': (0.0, 0.3), 'type': 'linear'},
    'energy_correction': {'bounds': (-0.01, 0.01), 'type': 'linear'},
    'enstrophy_correction': {'bounds': (0.0, 1e-6), 'type': 'log'},
}

# REFERENCE/DEFAULT PARAMETERS (known baseline)
# Note: If defaults are outside bounds, they will be clipped automatically
DEFAULT_PARAMS = {
    'viscosity_scale': 0.5,
    'drag_scale': 0.5,
    'eddy_diffusivity': 0.005,  # User-provided default (will be clipped to bounds if needed)
    'smagorinsky_coeff': 0.015,
    'energy_correction': -0.002,
    'enstrophy_correction': 3e-9,
}

PARAM_NAMES = list(PARAM_BOUNDS.keys())
N_PARAMS = len(PARAM_NAMES)

# Input warping for log-scale parameters
def warp_parameters(params_array):
    """Transform to warped space: log params → log space, linear → [0,1]"""
    warped = np.zeros(N_PARAMS)
    for i, name in enumerate(PARAM_NAMES):
        val, info = params_array[i], PARAM_BOUNDS[name]
        lower, upper = info['bounds']
        if info['type'] == 'log':
            log_lower, log_upper = np.log10(lower) if lower > 0 else -10, np.log10(upper)
            warped[i] = (np.log10(val + 1e-20) - log_lower) / (log_upper - log_lower)
        else:
            warped[i] = (val - lower) / (upper - lower)
    return warped

def unwarp_parameters(warped_array):
    """Transform from warped space back to original space"""
    params = np.zeros(N_PARAMS)
    for i, name in enumerate(PARAM_NAMES):
        info = PARAM_BOUNDS[name]
        lower, upper = info['bounds']
        if info['type'] == 'log':
            log_lower, log_upper = np.log10(lower) if lower > 0 else -10, np.log10(upper)
            params[i] = 10 ** (warped_array[i] * (log_upper - log_lower) + log_lower)
        else:
            params[i] = warped_array[i] * (upper - lower) + lower
        params[i] = np.clip(params[i], lower, upper)
    return params

def params_dict_to_array(params_dict):
    """Convert parameter dictionary to array, clipping to bounds"""
    params = []
    for name in PARAM_NAMES:
        val = params_dict[name]
        lower, upper = PARAM_BOUNDS[name]['bounds']
        clipped_val = np.clip(val, lower, upper)
        if clipped_val != val:
            print(Colors.yellow(f"  ⚠ Clipped {name}: {val:.6e} → {clipped_val:.6e} (bounds: [{lower:.6e}, {upper:.6e}])"))
        params.append(clipped_val)
    return np.array(params)

def params_array_to_dict(params_array):
    """Convert parameter array to dictionary"""
    return {PARAM_NAMES[i]: float(params_array[i]) for i in range(N_PARAMS)}

# ============================================================================
# SMART INITIALIZATION WITH WARM-START
# ============================================================================

def generate_smart_initial_samples(n_samples, include_default=True, base_seed=42):
    """
    Combine default params + Latin Hypercube + Sobol for warm-start
    Uses multiple complementary seeds for better diversity and robustness
    
    Args:
        n_samples: Total number of samples
        include_default: If True, first sample is DEFAULT_PARAMS
        base_seed: Base random seed (will generate complementary seeds from this)
    """
    samples = []
    
    # WARM-START: Include default parameters as first sample
    if include_default:
        default_array = params_dict_to_array(DEFAULT_PARAMS)
        samples.append(default_array)
        n_samples -= 1
        print(Colors.cyan("  ✓ Including reference parameters as warm-start"))
    
    # Generate space-filling samples for remaining
    # Use MULTIPLE seeds for robustness - reduces sensitivity to single seed choice
    n_lhs = n_samples // 2
    n_sobol = n_samples - n_lhs
    
    # LHS with primary seed
    lhs_sampler = qmc.LatinHypercube(d=N_PARAMS, seed=base_seed)
    lhs_samples = lhs_sampler.random(n=n_lhs)
    
    # Sobol with complementary seed (offset by 1000)
    # This ensures different quasi-random sequences
    sobol_sampler = qmc.Sobol(d=N_PARAMS, seed=base_seed + 1000, scramble=True)
    sobol_samples = sobol_sampler.random(n=n_sobol)
    
    # Combine samples
    unit_samples = np.vstack([lhs_samples, sobol_samples])
    
    # Add small random perturbations to avoid exact grid points
    # This helps exploration and reduces sensitivity to specific seed values
    np.random.seed(base_seed + 2000)
    perturbations = np.random.normal(0, 0.02, size=unit_samples.shape)
    unit_samples = np.clip(unit_samples + perturbations, 0, 1)
    
    for unit_sample in unit_samples:
        samples.append(unwarp_parameters(unit_sample))
    
    print(Colors.cyan(f"  ✓ Generated {len(samples)} diverse initial samples (base_seed={base_seed})"))
    
    return np.array(samples)

# ============================================================================
# ENSEMBLE GP
# ============================================================================

class EnsembleGP:
    """Ensemble of Gaussian Processes with different kernels"""
    
    def __init__(self, n_models=8):
        self.models = []
        self.model_weights = []
        
        kernels = [
            ConstantKernel(1.0, (1e-3, 1e3)) * 
            Matern(length_scale=[1.0]*N_PARAMS, length_scale_bounds=(1e-3, 1e3), nu=1.5) +
            WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-10, 1e-1)),
            
            ConstantKernel(1.0, (1e-3, 1e3)) * 
            Matern(length_scale=[1.0]*N_PARAMS, length_scale_bounds=(1e-3, 1e3), nu=2.5) +
            WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-10, 1e-1)),
            
            ConstantKernel(1.0, (1e-3, 1e3)) * 
            RBF(length_scale=[1.0]*N_PARAMS, length_scale_bounds=(1e-3, 1e3)) +
            WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-10, 1e-1)),
            
            ConstantKernel(1.0, (1e-3, 1e3)) * 
            Matern(length_scale=[0.5]*N_PARAMS, length_scale_bounds=(1e-3, 1e2), nu=2.5) +
            WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-10, 1e-1)),
            
            ConstantKernel(1.0, (1e-3, 1e3)) * 
            RBF(length_scale=[2.0]*N_PARAMS, length_scale_bounds=(1e-2, 1e3)) +
            WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-10, 1e-1)),
            
            ConstantKernel(1.0, (1e-3, 1e3)) * 
            Matern(length_scale=[0.3]*N_PARAMS, length_scale_bounds=(1e-3, 1e2), nu=1.5) +
            WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-10, 1e-1)),
            
            ConstantKernel(1.0, (1e-3, 1e3)) * 
            RBF(length_scale=[0.7]*N_PARAMS, length_scale_bounds=(1e-3, 1e3)) +
            WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-10, 1e-1)),
            
            ConstantKernel(1.0, (1e-3, 1e3)) * 
            Matern(length_scale=[1.5]*N_PARAMS, length_scale_bounds=(1e-3, 1e3), nu=2.5) +
            WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-10, 1e-1)),
        ]
        
        for kernel in kernels[:n_models]:
            self.models.append(GaussianProcessRegressor(
                kernel=kernel,
                alpha=1e-6,
                normalize_y=True,
                n_restarts_optimizer=15,
                random_state=None
            ))
    
    def fit(self, X, y):
        """Fit all models and compute weights based on marginal likelihood"""
        self.model_weights = []
        for i, model in enumerate(self.models):
            try:
                model.fit(X, y)
                # Weight by log marginal likelihood
                log_ml = model.log_marginal_likelihood()
                self.model_weights.append(np.exp(log_ml))
            except Exception as e:
                print(f"  Warning: Model {i} fitting failed: {e}")
                self.model_weights.append(0.0)
        
        # Normalize weights
        total_weight = sum(self.model_weights)
        if total_weight > 0:
            self.model_weights = [w / total_weight for w in self.model_weights]
        else:
            self.model_weights = [1.0 / len(self.models)] * len(self.models)
    
    def predict(self, X, return_std=True):
        """Weighted ensemble prediction"""
        X = np.atleast_2d(X)
        
        if return_std:
            predictions = []
            uncertainties = []
            weights = []
            
            for i, model in enumerate(self.models):
                if self.model_weights[i] > 0:
                    try:
                        mu, sigma = model.predict(X, return_std=True)
                        predictions.append(mu)
                        uncertainties.append(sigma)
                        weights.append(self.model_weights[i])
                    except:
                        continue
            
            if len(predictions) == 0:
                return np.zeros(len(X)), np.ones(len(X))
            
            # Weighted mean and maximum uncertainty
            weights = np.array(weights)
            mean = np.average(predictions, axis=0, weights=weights)
            std = np.max(uncertainties, axis=0)
            
            return mean, std
        else:
            predictions = []
            weights = []
            for i, model in enumerate(self.models):
                if self.model_weights[i] > 0:
                    try:
                        predictions.append(model.predict(X))
                        weights.append(self.model_weights[i])
                    except:
                        continue
            
            return np.average(predictions, axis=0, weights=weights) if predictions else np.zeros(len(X))
    
    def get_parameter_importance(self):
        """Extract parameter importance from length scales"""
        importance_scores = []
        
        for i, model in enumerate(self.models):
            if self.model_weights[i] > 0:
                try:
                    kernel = model.kernel_
                    # Try to extract length scales from different kernel structures
                    length_scales = None
                    
                    # For composite kernels (ConstantKernel * Matern/RBF + WhiteKernel)
                    if hasattr(kernel, 'k1') and hasattr(kernel.k1, 'k2'):
                        length_scales = kernel.k1.k2.length_scale
                    elif hasattr(kernel, 'k2'):
                        length_scales = kernel.k2.length_scale
                    elif hasattr(kernel, 'length_scale'):
                        length_scales = kernel.length_scale
                    
                    if length_scales is not None and hasattr(length_scales, '__len__'):
                        # Inverse of length scale = importance (smaller length scale = more sensitive)
                        # Normalize by median to get relative importance
                        ls_array = np.array(length_scales)
                        importance = 1.0 / (ls_array + 1e-10)
                        # Normalize so median = 1.0
                        importance = importance / (np.median(importance) + 1e-10)
                        importance_scores.append(importance)
                except Exception as e:
                    continue
        
        if importance_scores and len(importance_scores) > 0:
            # Weighted average importance
            weights = [w for w in self.model_weights if w > 0][:len(importance_scores)]
            weighted_importance = np.average(importance_scores, axis=0, weights=weights)
            return weighted_importance
        else:
            # Fallback: return ones (equal importance)
            return np.ones(N_PARAMS)

# Trust region with reset capability
class TrustRegion:
    def __init__(self):
        self.trust_radius, self.best_center = 0.5, None
        self.success_count, self.fail_count = 0, 0
        self.min_radius, self.max_radius = 0.05, 1.0
        self.radius_history = []
    
    def get_trust_region_bounds(self):
        if self.best_center is None:
            return [(0, 1)] * N_PARAMS
        bounds = []
        for i in range(N_PARAMS):
            center, half_width = self.best_center[i], self.trust_radius / 2
            bounds.append((max(0.0, center - half_width), min(1.0, center + half_width)))
        return bounds
    
    def update(self, new_best_found, new_center=None):
        if new_best_found:
            self.success_count += 1
            self.fail_count = 0
            if new_center is not None:
                self.best_center = new_center
            if self.success_count >= 3:
                self.trust_radius = min(self.max_radius, self.trust_radius * 1.5)
                self.success_count = 0
                print(f"  → Trust region expanded to {self.trust_radius:.2f}")
        else:
            self.fail_count += 1
            self.success_count = 0
            if self.fail_count >= 3:
                self.trust_radius = max(self.min_radius, self.trust_radius * 0.5)
                self.fail_count = 0
                print(f"  → Trust region shrunk to {self.trust_radius:.2f}")
        
        self.radius_history.append(self.trust_radius)
    
    def reset_for_exploration(self):
        self.trust_radius = 0.8
        self.success_count, self.fail_count = 0, 0
        self.radius_history.append(self.trust_radius)
        print(f"  → Trust region RESET to {self.trust_radius:.2f}")

# NEW: Thompson Sampling for exploration
def thompson_sampling(gp, bounds, n_samples=1):
    """Sample from GP posterior for exploration"""
    samples = []
    for _ in range(n_samples):
        # Sample a function from GP posterior
        X_grid = np.random.uniform([b[0] for b in bounds], [b[1] for b in bounds], size=(500, N_PARAMS))
        mu, sigma = gp.predict(X_grid, return_std=True)
        
        # Sample from posterior at each point
        posterior_samples = np.random.normal(mu, sigma)
        
        # Find minimum of sampled function
        best_idx = np.argmin(posterior_samples)
        samples.append(X_grid[best_idx])
    
    return np.array(samples)

# Hybrid acquisition with LOCAL PENALIZATION
def hybrid_acquisition_with_penalization(X, gp, best_y, X_samples, xi=0.01, kappa=2.0, weight_ei=0.6, penalization_weight=0.3):
    """Hybrid EI+UCB with local penalization"""
    X = np.atleast_2d(X)
    mu, sigma = gp.predict(X, return_std=True)
    
    # Expected Improvement
    with np.errstate(divide='warn', invalid='warn'):
        imp = best_y - mu - xi
        Z = imp / (sigma + 1e-9)
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0
    
    # Upper Confidence Bound
    ucb = -(mu - kappa * sigma)
    
    # Normalize
    ei_norm = (ei - ei.min()) / (ei.max() - ei.min() + 1e-9)
    ucb_norm = (ucb - ucb.min()) / (ucb.max() - ucb.min() + 1e-9)
    
    # Base acquisition
    acq = weight_ei * ei_norm + (1 - weight_ei) * ucb_norm
    
    # Local penalization
    if len(X_samples) > 0 and penalization_weight > 0:
        X_samples_warped = np.array([warp_parameters(x) for x in X_samples])
        min_distances = np.min([np.linalg.norm(X - x_sample, axis=1) for x_sample in X_samples_warped], axis=0)
        penalty = np.exp(-10 * min_distances)
        acq = acq * (1 - penalization_weight * penalty)
    
    return acq

# Acquisition optimizer with multi-start
def optimize_acquisition_multistart(acquisition_fn, bounds, n_starts=20, n_random=500):
    """Multi-start optimization of acquisition function"""
    best_acq, best_x = -np.inf, None
    
    # Random sampling
    random_samples = np.random.uniform([b[0] for b in bounds], [b[1] for b in bounds], size=(n_random, N_PARAMS))
    acq_random = acquisition_fn(random_samples)
    best_random_idx = np.argmax(acq_random)
    if acq_random[best_random_idx] > best_acq:
        best_acq, best_x = acq_random[best_random_idx], random_samples[best_random_idx]
    
    # Gradient-based optimization
    for _ in range(n_starts):
        x0 = np.array([np.random.uniform(b[0], b[1]) for b in bounds])
        result = minimize(lambda x: -acquisition_fn(x.reshape(1, -1))[0], x0, method='L-BFGS-B', bounds=bounds)
        if result.success and -result.fun > best_acq:
            best_acq, best_x = -result.fun, result.x
    
    return best_x

# NEW: Simplified 2-level multi-fidelity (30 days → 180 days)
def get_adaptive_sim_days(iteration, base_days=180):
    """
    Two-level fidelity strategy:
    - Iterations 0-40: Fast 30-day runs (6x faster exploration)
    - Iterations 40+:  Full 180-day runs (final precision)
    
    Returns: (sim_days, description)
    """
    if iteration < 40:
        return 30, "FAST (30d)"  # Fast exploration
    else:
        return base_days, "FULL (180d)"  # Full precision

# Simulation runner
def run_lowres_with_params(params_array, config_base, highres_results, sim_days=180, iteration=0):
    from main_comparison import run_simulation
    
    # Adaptive fidelity
    adaptive_days, fidelity_desc = get_adaptive_sim_days(iteration, sim_days)
    
    config = config_base.copy()
    config['subgrid_params'] = {PARAM_NAMES[i]: float(params_array[i]) for i in range(N_PARAMS)}
    
    print(f"\n{'='*70}")
    print(f"Testing parameters - Fidelity: {Colors.cyan(fidelity_desc)}")
    print(f"{'='*70}")
    for param_name, val in config['subgrid_params'].items():
        print(f"  {param_name}: {val:.6e}")
    
    try:
        results = run_simulation(config, sim_days=adaptive_days, save_interval_hours=12)
        # Adaptive loss computation will automatically use appropriate time window
        loss, detailed = compute_loss(results, highres_results, return_fields=True, adaptive_window=True)
        if not np.isfinite(loss):
            print(Colors.yellow(f"  ⚠ Loss not finite: {loss}"))
            return np.nan, None, None
        print(f"  Loss: {Colors.green(f'{loss:.6f}')}")
        return loss, results, detailed
    except Exception as e:
        print(Colors.yellow(f"  ⚠ Simulation failed: {e}"))
        return np.nan, None, None

# Loss computation with adaptive time window
def compute_loss(lowres_results, highres_results, n_days_avg=30, return_fields=False, adaptive_window=True):
    """
    Compute loss with adaptive time window based on simulation length
    
    Args:
        adaptive_window: If True, adjust time window based on lowres simulation length
                        - For 30-day runs: use entire simulation (days 0-30)
                        - For 180-day runs: use last 30 days (days 150-180, equilibrated)
    """
    nx_hr, ny_hr = highres_results['config']['nx'], highres_results['config']['ny']
    nx_lr, ny_lr = lowres_results['config']['nx'], lowres_results['config']['ny']
    coarsen_factor_x, coarsen_factor_y = nx_hr // nx_lr, ny_hr // ny_lr
    
    times_hr, times_lr = highres_results['times'], lowres_results['times']
    
    # Adaptive time window based on simulation length
    if adaptive_window:
        lr_duration = times_lr[-1] - times_lr[0]
        
        if lr_duration <= 40:  # 30-day runs
            # Use entire simulation
            time_start, time_end = times_lr[0], times_lr[-1]
            print(f"  → Using entire simulation (days 0-{time_end - time_start:.0f}) for loss")
        else:  # Full 180-day runs
            # Use last 30 days for equilibrated state
            time_start, time_end = times_lr[-1] - n_days_avg, times_lr[-1]
            print(f"  → Using last {n_days_avg} days for loss (equilibrated state)")
        
        # Get matching time window from high-res
        if lr_duration <= 40:
            # For short runs, match the same absolute time window
            indices_hr = np.where((times_hr >= time_start) & (times_hr <= time_end))[0]
        else:
            # For full runs, use last 30 days of high-res too
            indices_hr = np.where(times_hr >= times_hr[-1] - n_days_avg)[0]
        
        indices_lr = np.where((times_lr >= time_start) & (times_lr <= time_end))[0]
    else:
        # Original behavior: use last n_days_avg
        indices_hr = np.where(times_hr >= times_hr[-1] - n_days_avg)[0]
        indices_lr = np.where(times_lr >= times_lr[-1] - n_days_avg)[0]
    
    q1_hr_avg = np.mean([highres_results['q1_history'][i] for i in indices_hr], axis=0)
    q2_hr_avg = np.mean([highres_results['q2_history'][i] for i in indices_hr], axis=0)
    q1_lr_avg = np.mean([lowres_results['q1_history'][i] for i in indices_lr], axis=0)
    q2_lr_avg = np.mean([lowres_results['q2_history'][i] for i in indices_lr], axis=0)
    
    model_hr, model_lr = highres_results['model'], lowres_results['model']
    psi1_hr_avg, psi2_hr_avg = model_hr.q_to_psi(q1_hr_avg, q2_hr_avg)
    psi1_lr_avg, psi2_lr_avg = model_lr.q_to_psi(q1_lr_avg, q2_lr_avg)
    
    H1, H2, H_total = model_hr.H1, model_hr.H2, model_hr.H1 + model_hr.H2
    q_bt_hr = (H1 * q1_hr_avg + H2 * q2_hr_avg) / H_total
    psi_bt_hr = (H1 * psi1_hr_avg + H2 * psi2_hr_avg) / H_total
    q_bt_lr = (H1 * q1_lr_avg + H2 * q2_lr_avg) / H_total
    psi_bt_lr = (H1 * psi1_lr_avg + H2 * psi2_lr_avg) / H_total
    
    def coarsen(field, fx, fy):
        return uniform_filter(field, size=(fy, fx), mode='wrap')[::fy, ::fx]
    
    q_bt_hr_coarse = coarsen(q_bt_hr, coarsen_factor_x, coarsen_factor_y)
    psi_bt_hr_coarse = coarsen(psi_bt_hr, coarsen_factor_x, coarsen_factor_y)
    
    nrmse = lambda pred, target: np.sqrt(np.mean((pred - target)**2)) / (np.std(target) + 1e-20)
    loss_q_bt, loss_psi_bt = nrmse(q_bt_lr, q_bt_hr_coarse), nrmse(psi_bt_lr, psi_bt_hr_coarse)
    weight_pv, weight_psi = 0.6, 0.4
    total_loss = weight_pv * loss_q_bt + weight_psi * loss_psi_bt
    
    if return_fields:
        return total_loss, {'q_bt_hr_coarse': q_bt_hr_coarse, 'psi_bt_hr_coarse': psi_bt_hr_coarse,
                           'q_bt_lr': q_bt_lr, 'psi_bt_lr': psi_bt_lr, 'loss_q_bt': loss_q_bt,
                           'loss_psi_bt': loss_psi_bt, 'total_loss': total_loss}
    return total_loss

# ============================================================================
# VISUALIZATION SUITE
# ============================================================================

class OptimizationVisualizer:
    """Comprehensive visualization of optimization progress"""
    
    def __init__(self, optimizer):
        self.optimizer = optimizer
        sns.set_style("whitegrid")
        plt.rcParams['figure.dpi'] = 100
        plt.rcParams['savefig.dpi'] = 300
    
    def plot_comprehensive_analysis(self, save_path='optimization_analysis.png'):
        """Create comprehensive multi-panel analysis"""
        fig = plt.figure(figsize=(20, 12))
        gs = gridspec.GridSpec(3, 3, figure=fig, hspace=0.3, wspace=0.3)
        
        # 1. Loss evolution
        ax1 = fig.add_subplot(gs[0, :2])
        self._plot_loss_evolution(ax1)
        
        # 2. Parameter evolution
        ax2 = fig.add_subplot(gs[1, :2])
        self._plot_parameter_evolution(ax2)
        
        # 3. Parameter importance
        ax3 = fig.add_subplot(gs[2, :2])
        self._plot_parameter_importance(ax3)
        
        # 4. Trust region evolution
        ax4 = fig.add_subplot(gs[0, 2])
        self._plot_trust_region(ax4)
        
        # 5. Convergence diagnostics
        ax5 = fig.add_subplot(gs[1, 2])
        self._plot_convergence_diagnostics(ax5)
        
        # 6. Best parameters bar chart
        ax6 = fig.add_subplot(gs[2, 2])
        self._plot_best_parameters(ax6)
        
        plt.suptitle('Bayesian Optimization - Comprehensive Analysis', 
                     fontsize=16, fontweight='bold', y=0.995)
        
        plt.savefig(save_path, bbox_inches='tight', dpi=300)
        print(f"\n✓ Saved comprehensive analysis: {save_path}")
        plt.close()
    
    def _plot_loss_evolution(self, ax):
        """Plot loss vs iterations with best loss tracking and fidelity phases"""
        y_samples = np.array(self.optimizer.y_samples)
        valid_mask = np.isfinite(y_samples)
        
        iterations = np.arange(len(y_samples))
        
        # Plot all losses
        ax.scatter(iterations[valid_mask], y_samples[valid_mask], 
                  alpha=0.6, s=50, c='steelblue', label='Valid evaluations', zorder=3)
        ax.scatter(iterations[~valid_mask], np.ones(np.sum(~valid_mask)) * np.nanmax(y_samples) * 1.1, 
                  alpha=0.4, s=30, c='red', marker='x', label='Failed evaluations', zorder=2)
        
        # Plot best loss trajectory - SEPARATE FOR EACH FIDELITY
        best_trajectory_30d = []
        best_trajectory_180d = []
        current_best_30d = np.inf
        current_best_180d = np.inf
        
        for i, loss in enumerate(y_samples):
            sim_days, _ = get_adaptive_sim_days(i)
            
            if sim_days <= 40:  # 30-day fidelity
                if np.isfinite(loss) and loss < current_best_30d:
                    current_best_30d = loss
                best_trajectory_30d.append(current_best_30d if current_best_30d != np.inf else np.nan)
                best_trajectory_180d.append(np.nan)
            else:  # 180-day fidelity
                if np.isfinite(loss) and loss < current_best_180d:
                    current_best_180d = loss
                best_trajectory_30d.append(np.nan)
                best_trajectory_180d.append(current_best_180d if current_best_180d != np.inf else np.nan)
        
        # Plot 30-day best loss
        valid_30d = [(i, best_trajectory_30d[i]) for i in range(len(best_trajectory_30d)) if np.isfinite(best_trajectory_30d[i])]
        if valid_30d:
            indices_30d, values_30d = zip(*valid_30d)
            ax.plot(indices_30d, values_30d, 'limegreen', linewidth=2.5, 
                   label='Best loss (30d)', zorder=4, marker='o', markersize=5)
        
        # Plot 180-day best loss
        valid_180d = [(i, best_trajectory_180d[i]) for i in range(len(best_trajectory_180d)) if np.isfinite(best_trajectory_180d[i])]
        if valid_180d:
            indices_180d, values_180d = zip(*valid_180d)
            ax.plot(indices_180d, values_180d, 'darkgreen', linewidth=3, 
                   label='Best loss (180d)', zorder=4, marker='*', markersize=8)
        
        # Highlight different fidelity phases with colored backgrounds
        n_initial = self.optimizer.n_initial_samples
        ax.axvspan(0, n_initial-1, alpha=0.1, color='orange', label='Initial sampling')
        
        # 2-level multi-fidelity phases
        fidelity_transition = 40
        
        if len(iterations) > n_initial:
            # 30-day runs (fast exploration)
            ax.axvspan(n_initial, min(fidelity_transition, len(iterations)-1), 
                      alpha=0.10, color='lightblue', label='30d runs (fast)')
        
        if len(iterations) > fidelity_transition:
            # 180-day runs (full precision)
            ax.axvspan(fidelity_transition, len(iterations)-1, 
                      alpha=0.10, color='lightcoral', label='180d runs (full)')
        
        # Mark fidelity transition with vertical line
        if len(iterations) > fidelity_transition:
            ax.axvline(fidelity_transition, color='red', linestyle='--', linewidth=2, 
                      alpha=0.7, label='Fidelity jump')
        
        # Mark best iteration
        ax.scatter([self.optimizer.best_iteration], [self.optimizer.best_loss],
                  s=200, c='gold', marker='*', edgecolors='red', linewidth=2,
                  label=f'Best (iter {self.optimizer.best_iteration+1})', zorder=5)
        
        # If best was found in 30d phase, add annotation
        if self.optimizer.best_params_original_iteration is not None and self.optimizer.best_params_original_iteration < 40:
            # Check if we have 180d baseline (meaning re-evaluation happened)
            has_full_baseline = 'FULL (180d)' in self.optimizer.baseline_loss_by_fidelity
            if has_full_baseline:
                annotation_text = f'Found at iter {self.optimizer.best_params_original_iteration+1}\n(30d phase)\nRe-eval at 180d'
            else:
                annotation_text = f'Found at iter {self.optimizer.best_params_original_iteration+1}\n(30d phase)'
            
            ax.annotate(annotation_text, 
                       xy=(self.optimizer.best_params_original_iteration, self.optimizer.best_loss),
                       xytext=(self.optimizer.best_params_original_iteration + 10, self.optimizer.best_loss * 1.2),
                       arrowprops=dict(arrowstyle='->', color='red', lw=1.5),
                       fontsize=7, color='red', fontweight='bold',
                       bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7))
        
        ax.set_xlabel('Iteration', fontsize=12, fontweight='bold')
        ax.set_ylabel('Loss', fontsize=12, fontweight='bold')
        ax.set_title('Loss Evolution (2-Level Multi-Fidelity: 30d → 180d)\nSeparate tracking per fidelity', 
                    fontsize=12, fontweight='bold')
        ax.legend(loc='best', fontsize=7, ncol=2)
        ax.grid(True, alpha=0.3)
        
        # Add improvement info with all baselines
        if self.optimizer.baseline_loss_by_fidelity:
            info_lines = []
            
            # Final improvement (use FULL baseline if available)
            final_baseline = self.optimizer.baseline_loss_by_fidelity.get('FULL (180d)')
            if final_baseline:
                improvement = (final_baseline - self.optimizer.best_loss) / final_baseline * 100
                info_lines.append(f'Final improvement: {improvement:+.1f}%')
            
            # Show all baselines
            info_lines.append('Baselines:')
            for fid, base in sorted(self.optimizer.baseline_loss_by_fidelity.items()):
                info_lines.append(f'  {fid}: {base:.4f}')
            
            info_text = '\n'.join(info_lines)
            ax.text(0.02, 0.98, info_text, 
                   transform=ax.transAxes, fontsize=8, verticalalignment='top',
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.7))
    
    def _plot_parameter_evolution(self, ax):
        """Plot how parameters evolved over iterations"""
        X_samples = np.array(self.optimizer.X_samples)
        n_iters = len(X_samples)
        
        # Normalize parameters to [0, 1] for visualization
        X_normalized = np.array([warp_parameters(x) for x in X_samples])
        
        for i, param_name in enumerate(PARAM_NAMES):
            ax.plot(range(n_iters), X_normalized[:, i], 
                   marker='o', markersize=4, alpha=0.7, linewidth=1.5, 
                   label=param_name)
        
        # Highlight initial samples phase
        n_initial = self.optimizer.n_initial_samples
        ax.axvspan(0, n_initial-1, alpha=0.1, color='orange')
        
        # Mark best iteration
        ax.axvline(self.optimizer.best_iteration, color='red', linestyle='--', 
                  linewidth=2, alpha=0.7, label='Best found')
        
        ax.set_xlabel('Iteration', fontsize=12, fontweight='bold')
        ax.set_ylabel('Normalized Parameter Value', fontsize=12, fontweight='bold')
        ax.set_title('Parameter Evolution', fontsize=13, fontweight='bold')
        ax.legend(loc='best', fontsize=8, ncol=2)
        ax.grid(True, alpha=0.3)
        ax.set_ylim(-0.05, 1.05)
    
    def _plot_parameter_importance(self, ax):
        """Plot parameter sensitivity over time"""
        if not hasattr(self.optimizer, 'importance_history') or not self.optimizer.importance_history:
            ax.text(0.5, 0.5, 'Parameter importance\nnot tracked\n(Need more iterations)', 
                   ha='center', va='center', transform=ax.transAxes, fontsize=12)
            ax.set_title('Parameter Importance Over Time', fontsize=13, fontweight='bold')
            return
        
        importance_array = np.array(self.optimizer.importance_history)
        
        # Check if all values are constant (indicating a problem)
        if importance_array.shape[0] < 2 or np.allclose(importance_array[0], importance_array[-1]):
            # Fall back to correlation-based importance from current samples
            ax.text(0.5, 0.7, 'GP-based importance unavailable', 
                   ha='center', va='center', transform=ax.transAxes, fontsize=11, style='italic')
            ax.text(0.5, 0.5, 'Using correlation-based\nimportance instead', 
                   ha='center', va='center', transform=ax.transAxes, fontsize=10)
            ax.text(0.5, 0.3, '(See parameter_sensitivity.png\nfor detailed analysis)', 
                   ha='center', va='center', transform=ax.transAxes, fontsize=9, style='italic')
            ax.set_title('Parameter Importance Over Time', fontsize=13, fontweight='bold')
            return
        
        # Plot importance evolution
        iterations = np.arange(len(importance_array)) + self.optimizer.n_initial_samples
        
        for i, param_name in enumerate(PARAM_NAMES):
            ax.plot(iterations, importance_array[:, i], 
                   marker='o', markersize=3, alpha=0.7, linewidth=1.5,
                   label=param_name)
        
        ax.set_xlabel('Iteration', fontsize=12, fontweight='bold')
        ax.set_ylabel('Importance Score (1/length_scale)', fontsize=12, fontweight='bold')
        ax.set_title('Parameter Importance Over Time\n(Higher = More Sensitive)', fontsize=13, fontweight='bold')
        ax.legend(loc='best', fontsize=8, ncol=2)
        ax.grid(True, alpha=0.3)
        
        # Add interpretation note
        ax.text(0.98, 0.02, 'Note: Based on GP length scales\nSmaller length scale → Higher importance', 
               transform=ax.transAxes, fontsize=7, ha='right', va='bottom',
               bbox=dict(boxstyle='round,pad=0.3', facecolor='wheat', alpha=0.5))
    
    def _plot_trust_region(self, ax):
        """Plot trust region radius evolution"""
        if not self.optimizer.trust_region.radius_history:
            ax.text(0.5, 0.5, 'Trust region\nhistory empty', 
                   ha='center', va='center', transform=ax.transAxes, fontsize=10)
            ax.set_title('Trust Region Evolution', fontsize=11, fontweight='bold')
            return
        
        radius_history = self.optimizer.trust_region.radius_history
        ax.plot(radius_history, marker='o', markersize=4, linewidth=2, color='purple')
        ax.axhline(self.optimizer.trust_region.min_radius, color='red', 
                  linestyle='--', alpha=0.5, label='Min radius')
        ax.axhline(self.optimizer.trust_region.max_radius, color='green', 
                  linestyle='--', alpha=0.5, label='Max radius')
        
        ax.set_xlabel('Update Step', fontsize=10, fontweight='bold')
        ax.set_ylabel('Trust Radius', fontsize=10, fontweight='bold')
        ax.set_title('Trust Region Evolution', fontsize=11, fontweight='bold')
        ax.legend(fontsize=8)
        ax.grid(True, alpha=0.3)
    
    def _plot_convergence_diagnostics(self, ax):
        """Plot convergence metrics"""
        y_samples = np.array(self.optimizer.y_samples)
        valid_mask = np.isfinite(y_samples)
        
        # Moving average of improvement
        window = 5
        improvements = []
        for i in range(window, len(y_samples)):
            if valid_mask[i]:
                recent_best = np.nanmin(y_samples[max(0, i-window):i])
                current = y_samples[i]
                improvements.append(max(0, recent_best - current))
            else:
                improvements.append(0)
        
        iterations = np.arange(window, len(y_samples))
        ax.bar(iterations, improvements, alpha=0.6, color='teal')
        ax.set_xlabel('Iteration', fontsize=10, fontweight='bold')
        ax.set_ylabel('Recent Improvement', fontsize=10, fontweight='bold')
        ax.set_title('Convergence Diagnostics', fontsize=11, fontweight='bold')
        ax.grid(True, alpha=0.3, axis='y')
    
    def _plot_best_parameters(self, ax):
        """Bar chart of best parameters"""
        if self.optimizer.best_params is None:
            ax.text(0.5, 0.5, 'No best\nparameters yet', 
                   ha='center', va='center', transform=ax.transAxes, fontsize=10)
            ax.set_title('Best Parameters', fontsize=11, fontweight='bold')
            return
        
        # Normalize to [0, 1]
        best_normalized = warp_parameters(self.optimizer.best_params)
        
        colors = plt.cm.viridis(best_normalized)
        bars = ax.barh(PARAM_NAMES, best_normalized, color=colors, alpha=0.7)
        
        ax.set_xlabel('Normalized Value', fontsize=10, fontweight='bold')
        ax.set_title('Best Parameters (Normalized)', fontsize=11, fontweight='bold')
        ax.set_xlim(0, 1)
        ax.grid(True, alpha=0.3, axis='x')
        
        # Add actual values as text
        for i, (bar, name) in enumerate(zip(bars, PARAM_NAMES)):
            actual_val = self.optimizer.best_params[i]
            ax.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height()/2, 
                   f'{actual_val:.2e}', va='center', fontsize=7)
    
    def plot_parameter_sensitivity_heatmap(self, save_path='parameter_sensitivity.png'):
        """Create comprehensive parameter sensitivity analysis"""
        X_samples = np.array(self.optimizer.X_samples)
        y_samples = np.array(self.optimizer.y_samples)
        valid_mask = np.isfinite(y_samples)
        
        if np.sum(valid_mask) < 5:
            print("Not enough valid samples for sensitivity analysis")
            return
        
        X_valid = X_samples[valid_mask]
        y_valid = y_samples[valid_mask]
        
        # Normalize parameters
        X_normalized = np.array([warp_parameters(x) for x in X_valid])
        
        # Create comprehensive figure
        fig = plt.figure(figsize=(20, 12))
        gs = gridspec.GridSpec(3, 3, figure=fig, hspace=0.35, wspace=0.35)
        
        # ========== Panel 1: Parameter-Loss Correlations ==========
        ax1 = fig.add_subplot(gs[0, :2])
        correlations = []
        for i in range(N_PARAMS):
            corr = np.corrcoef(X_normalized[:, i], y_valid)[0, 1]
            correlations.append(corr)
        
        colors = ['crimson' if c > 0 else 'forestgreen' for c in correlations]
        bars = ax1.barh(PARAM_NAMES, correlations, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
        ax1.axvline(0, color='black', linewidth=2)
        ax1.set_xlabel('Correlation with Loss', fontsize=13, fontweight='bold')
        ax1.set_title('Parameter Sensitivity: Correlation with Loss\n' + 
                     'RED = Increasing parameter WORSENS performance | GREEN = Increasing parameter IMPROVES performance',
                     fontsize=12, fontweight='bold')
        ax1.grid(True, alpha=0.3, axis='x')
        
        for bar, corr in zip(bars, correlations):
            width = bar.get_width()
            label = f'{corr:+.3f}'
            ax1.text(width + (0.02 if width > 0 else -0.02), 
                    bar.get_y() + bar.get_height()/2, label,
                    va='center', ha='left' if width > 0 else 'right',
                    fontsize=10, fontweight='bold')
        
        # ========== Panel 2: Variance Explained ==========
        ax2 = fig.add_subplot(gs[0, 2])
        
        # Simple variance explained: R² from linear fit
        from sklearn.linear_model import LinearRegression
        var_explained = []
        for i in range(N_PARAMS):
            X_param = X_normalized[:, i].reshape(-1, 1)
            model = LinearRegression()
            model.fit(X_param, y_valid)
            r2 = model.score(X_param, y_valid)
            var_explained.append(max(0, r2))  # Clip negative R²
        
        colors_var = plt.cm.RdYlGn_r(np.array(var_explained) / max(var_explained))
        ax2.barh(PARAM_NAMES, var_explained, color=colors_var, alpha=0.8, edgecolor='black')
        ax2.set_xlabel('Variance Explained (R²)', fontsize=11, fontweight='bold')
        ax2.set_title('Parameter Importance\n(Higher = More Influential)', fontsize=11, fontweight='bold')
        ax2.grid(True, alpha=0.3, axis='x')
        
        for i, (val, name) in enumerate(zip(var_explained, PARAM_NAMES)):
            ax2.text(val + 0.01, i, f'{val:.3f}', va='center', fontsize=9, fontweight='bold')
        
        # ========== Panel 3: Parameter Ranges Explored ==========
        ax3 = fig.add_subplot(gs[1, :2])
        
        # Box plots showing explored ranges
        positions = np.arange(N_PARAMS)
        bp = ax3.boxplot([X_normalized[:, i] for i in range(N_PARAMS)],
                         positions=positions, vert=False, patch_artist=True,
                         widths=0.6, showfliers=True)
        
        for patch, corr in zip(bp['boxes'], correlations):
            color = 'lightcoral' if corr > 0 else 'lightgreen'
            patch.set_facecolor(color)
            patch.set_alpha(0.6)
        
        ax3.set_yticks(positions)
        ax3.set_yticklabels(PARAM_NAMES)
        ax3.set_xlabel('Normalized Parameter Value [0=min, 1=max]', fontsize=12, fontweight='bold')
        ax3.set_title('Parameter Space Exploration\n(Box = 25th-75th percentile, Whiskers = min-max, Dots = outliers)',
                     fontsize=11, fontweight='bold')
        ax3.grid(True, alpha=0.3, axis='x')
        ax3.set_xlim(-0.05, 1.05)
        
        # Mark best parameters
        if self.optimizer.best_params is not None:
            best_normalized = warp_parameters(self.optimizer.best_params)
            ax3.scatter(best_normalized, positions, s=200, c='gold', marker='*', 
                       edgecolors='red', linewidth=2, zorder=10, label='Best Found')
            ax3.legend(fontsize=10, loc='upper right')
        
        # ========== Panel 4: Loss vs Top 2 Parameters (Scatter) ==========
        abs_corr = np.abs(correlations)
        top_2_indices = np.argsort(abs_corr)[-2:]
        
        ax4 = fig.add_subplot(gs[1, 2])
        param_idx_1, param_idx_2 = top_2_indices[1], top_2_indices[0]
        
        scatter = ax4.scatter(X_normalized[:, param_idx_1], X_normalized[:, param_idx_2],
                            c=y_valid, cmap='viridis_r', s=80, alpha=0.6,
                            edgecolors='black', linewidth=0.5)
        
        # Mark best point
        if self.optimizer.best_params is not None:
            best_norm = warp_parameters(self.optimizer.best_params)
            ax4.scatter(best_norm[param_idx_1], best_norm[param_idx_2],
                       s=300, c='gold', marker='*', edgecolors='red', linewidth=2.5,
                       zorder=10, label='Best')
        
        ax4.set_xlabel(f'{PARAM_NAMES[param_idx_1]}', fontsize=11, fontweight='bold')
        ax4.set_ylabel(f'{PARAM_NAMES[param_idx_2]}', fontsize=11, fontweight='bold')
        ax4.set_title(f'Top 2 Most Influential Parameters\n(Lower loss = Better)', 
                     fontsize=11, fontweight='bold')
        ax4.grid(True, alpha=0.3)
        ax4.legend(fontsize=9)
        
        cbar = plt.colorbar(scatter, ax=ax4)
        cbar.set_label('Loss', fontsize=10, fontweight='bold')
        
        # ========== Panel 5: Parameter Value Distributions ==========
        ax5 = fig.add_subplot(gs[2, :2])
        
        # Show distribution of sampled values for top 3 parameters
        top_3_indices = np.argsort(abs_corr)[-3:]
        colors_dist = ['red', 'orange', 'green']
        
        for idx_rank, param_idx in enumerate(top_3_indices[::-1]):
            values = X_normalized[:, param_idx]
            ax5.hist(values, bins=15, alpha=0.5, color=colors_dist[idx_rank], 
                    label=PARAM_NAMES[param_idx], edgecolor='black', linewidth=1)
        
        ax5.set_xlabel('Normalized Parameter Value', fontsize=12, fontweight='bold')
        ax5.set_ylabel('Frequency', fontsize=12, fontweight='bold')
        ax5.set_title('Sampling Distribution of Top 3 Most Influential Parameters',
                     fontsize=11, fontweight='bold')
        ax5.legend(fontsize=10)
        ax5.grid(True, alpha=0.3, axis='y')
        
        # ========== Panel 6: Parameter Importance Summary ==========
        ax6 = fig.add_subplot(gs[2, 2])
        ax6.axis('off')
        
        # Create summary text
        summary_lines = [
            "INTERPRETATION GUIDE:",
            "",
            "Correlation:",
            "  • Positive = increasing parameter worsens loss",
            "  • Negative = increasing parameter improves loss",
            "  • Magnitude = strength of relationship",
            "",
            "Variance Explained (R²):",
            "  • How much loss variation this parameter explains",
            "  • Higher = more important to tune carefully",
            "",
            "Top 3 Most Important Parameters:",
        ]
        
        top_3_with_corr = [(PARAM_NAMES[i], correlations[i], var_explained[i]) 
                           for i in np.argsort(abs_corr)[-3:][::-1]]
        
        for rank, (name, corr, var_exp) in enumerate(top_3_with_corr, 1):
            direction = "↑ worsens" if corr > 0 else "↓ improves"
            summary_lines.append(f"  {rank}. {name}")
            summary_lines.append(f"     Corr: {corr:+.3f} ({direction})")
            summary_lines.append(f"     R²: {var_exp:.3f}")
        
        summary_text = '\n'.join(summary_lines)
        ax6.text(0.05, 0.95, summary_text, transform=ax6.transAxes,
                fontsize=10, verticalalignment='top', family='monospace',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
        
        plt.suptitle('Comprehensive Parameter Sensitivity Analysis', 
                    fontsize=16, fontweight='bold', y=0.995)
        
        plt.savefig(save_path, bbox_inches='tight', dpi=300)
        print(f"✓ Saved sensitivity analysis: {save_path}")
        plt.close()
    
    def plot_computational_efficiency(self, save_path='computational_efficiency.png'):
        """Analyze computational cost vs improvement"""
        y_samples = np.array(self.optimizer.y_samples)
        valid_mask = np.isfinite(y_samples)
        
        if np.sum(valid_mask) < 5:
            print("Not enough valid samples for efficiency analysis")
            return
        
        # Estimate computational cost (30-day = 1 unit, 180-day = 6 units)
        cumulative_cost = []
        cost_per_iteration = []
        total_cost = 0
        
        for i in range(len(y_samples)):
            sim_days, _ = get_adaptive_sim_days(i)
            cost = sim_days / 30.0  # Normalize to 30-day cost
            cost_per_iteration.append(cost)
            total_cost += cost
            cumulative_cost.append(total_cost)
        
        # Create figure
        fig = plt.figure(figsize=(18, 10))
        gs = gridspec.GridSpec(2, 3, figure=fig, hspace=0.3, wspace=0.3)
        
        # ========== Panel 1: Cumulative Cost vs Improvement ==========
        ax1 = fig.add_subplot(gs[0, :2])
        
        # Best loss trajectory - split by fidelity
        best_trajectory_30d = []
        best_trajectory_180d = []
        current_best_30d = np.inf
        current_best_180d = np.inf
        
        for i, loss in enumerate(y_samples):
            sim_days, _ = get_adaptive_sim_days(i)
            
            if sim_days <= 40:  # 30-day fidelity
                if np.isfinite(loss) and loss < current_best_30d:
                    current_best_30d = loss
                best_trajectory_30d.append(current_best_30d if current_best_30d != np.inf else np.nan)
                best_trajectory_180d.append(np.nan)  # Not applicable yet
            else:  # 180-day fidelity
                if np.isfinite(loss) and loss < current_best_180d:
                    current_best_180d = loss
                best_trajectory_30d.append(np.nan)  # 30-day phase is over
                best_trajectory_180d.append(current_best_180d if current_best_180d != np.inf else np.nan)
        
        ax1_twin = ax1.twinx()
        
        # Plot cumulative cost
        color_cost = 'steelblue'
        line1 = ax1.plot(range(len(cumulative_cost)), cumulative_cost, 
                color=color_cost, linewidth=2.5, label='Cumulative Cost', marker='o', markersize=4)
        ax1.set_xlabel('Iteration', fontsize=12, fontweight='bold')
        ax1.set_ylabel('Cumulative Computational Cost (30-day equiv.)', 
                      fontsize=11, fontweight='bold', color=color_cost)
        ax1.tick_params(axis='y', labelcolor=color_cost)
        ax1.grid(True, alpha=0.3)
        
        # Plot best loss for 30-day fidelity
        color_loss_30d = 'limegreen'
        valid_indices_30d = [i for i, loss in enumerate(best_trajectory_30d) if np.isfinite(loss)]
        valid_trajectory_30d = [best_trajectory_30d[i] for i in valid_indices_30d]
        line2 = ax1_twin.plot(valid_indices_30d, valid_trajectory_30d, 
                     color=color_loss_30d, linewidth=2.5, label='Best Loss (30d)', 
                     marker='o', markersize=6, linestyle='-', alpha=0.8)
        
        # Plot best loss for 180-day fidelity
        color_loss_180d = 'darkgreen'
        valid_indices_180d = [i for i, loss in enumerate(best_trajectory_180d) if np.isfinite(loss)]
        valid_trajectory_180d = [best_trajectory_180d[i] for i in valid_indices_180d]
        line3 = ax1_twin.plot(valid_indices_180d, valid_trajectory_180d, 
                     color=color_loss_180d, linewidth=3, label='Best Loss (180d)', 
                     marker='*', markersize=8, linestyle='-')
        
        ax1_twin.set_ylabel('Best Loss', fontsize=11, fontweight='bold', color='darkgreen')
        ax1_twin.tick_params(axis='y', labelcolor='darkgreen')
        
        # Mark fidelity transition
        line4 = ax1.axvline(40, color='red', linestyle='--', linewidth=2, alpha=0.7, label='Fidelity Jump')
        
        # Add annotation explaining the jump
        if len(valid_indices_30d) > 0 and len(valid_indices_180d) > 0:
            last_30d_loss = valid_trajectory_30d[-1]
            first_180d_loss = valid_trajectory_180d[0]
            if np.isfinite(last_30d_loss) and np.isfinite(first_180d_loss):
                jump_pct = (first_180d_loss - last_30d_loss) / last_30d_loss * 100
                ax1_twin.annotate(f'Re-eval: {jump_pct:+.1f}%',
                                 xy=(40, first_180d_loss), xytext=(45, first_180d_loss * 1.1),
                                 arrowprops=dict(arrowstyle='->', color='red', lw=1.5),
                                 fontsize=8, color='red', fontweight='bold',
                                 bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7))
        
        ax1.set_title('Computational Efficiency: Cost vs Improvement Over Time\n(Separate best loss tracking for each fidelity)', 
                     fontsize=12, fontweight='bold')
        
        # Combine legends from both axes
        lines = line1 + line2 + line3 + [line4]
        labels = [l.get_label() for l in lines]
        ax1.legend(lines, labels, loc='upper left', fontsize=9)
        
        # ========== Panel 2: Improvement per Cost Unit ==========
        ax2 = fig.add_subplot(gs[0, 2])
        
        # Calculate improvement per cost for each iteration
        if self.optimizer.baseline_loss:
            improvements = []
            for i, loss in enumerate(y_samples):
                if np.isfinite(loss):
                    improvement = max(0, self.optimizer.baseline_loss - loss)
                    improvements.append(improvement / cost_per_iteration[i])
                else:
                    improvements.append(0)
            
            # Moving average
            window = 5
            smooth_improvements = []
            for i in range(len(improvements)):
                start = max(0, i - window + 1)
                smooth_improvements.append(np.mean(improvements[start:i+1]))
            
            ax2.plot(range(len(smooth_improvements)), smooth_improvements, 
                    color='purple', linewidth=2.5, label='Smoothed')
            ax2.scatter(range(len(improvements)), improvements, 
                       alpha=0.4, s=30, c='gray', label='Raw')
            
            ax2.set_xlabel('Iteration', fontsize=11, fontweight='bold')
            ax2.set_ylabel('Improvement per Cost Unit', fontsize=10, fontweight='bold')
            ax2.set_title('Sample Efficiency\n(Higher = Better)', fontsize=11, fontweight='bold')
            ax2.legend(fontsize=9)
            ax2.grid(True, alpha=0.3)
        
        # ========== Panel 3: Cost Breakdown by Phase ==========
        ax3 = fig.add_subplot(gs[1, 0])
        
        # Calculate costs by phase
        phase_names = ['Initial\nSampling', 'Fast\nExploration\n(30d)', 'Full\nPrecision\n(180d)']
        phase_costs = [0, 0, 0]
        phase_iters = [0, 0, 0]
        
        n_init = self.optimizer.n_initial_samples
        for i, cost in enumerate(cost_per_iteration):
            if i < n_init:
                phase_costs[0] += cost
                phase_iters[0] += 1
            elif i < 40:
                phase_costs[1] += cost
                phase_iters[1] += 1
            else:
                phase_costs[2] += cost
                phase_iters[2] += 1
        
        colors_phase = ['orange', 'lightblue', 'lightcoral']
        bars = ax3.bar(phase_names, phase_costs, color=colors_phase, alpha=0.7, edgecolor='black', linewidth=2)
        ax3.set_ylabel('Total Computational Cost', fontsize=11, fontweight='bold')
        ax3.set_title('Cost Breakdown by Phase', fontsize=11, fontweight='bold')
        ax3.grid(True, alpha=0.3, axis='y')
        
        # Add iteration counts and percentages
        for bar, cost, n_iter in zip(bars, phase_costs, phase_iters):
            height = bar.get_height()
            pct = cost / sum(phase_costs) * 100
            ax3.text(bar.get_x() + bar.get_width()/2., height,
                    f'{cost:.1f}\n({n_iter} iters)\n{pct:.1f}%',
                    ha='center', va='bottom', fontsize=9, fontweight='bold')
        
        # ========== Panel 4: Improvements Found by Phase ==========
        ax4 = fig.add_subplot(gs[1, 1])
        
        # Count new bests found in each phase
        best_found_phase = [0, 0, 0]
        current_best = np.inf
        
        for i, loss in enumerate(y_samples):
            if np.isfinite(loss) and loss < current_best:
                current_best = loss
                if i < n_init:
                    best_found_phase[0] += 1
                elif i < 40:
                    best_found_phase[1] += 1
                else:
                    best_found_phase[2] += 1
        
        bars2 = ax4.bar(phase_names, best_found_phase, color=colors_phase, alpha=0.7, 
                       edgecolor='black', linewidth=2)
        ax4.set_ylabel('Number of Improvements Found', fontsize=11, fontweight='bold')
        ax4.set_title('Improvements Discovered by Phase', fontsize=11, fontweight='bold')
        ax4.grid(True, alpha=0.3, axis='y')
        
        for bar, count in zip(bars2, best_found_phase):
            height = bar.get_height()
            if height > 0:
                ax4.text(bar.get_x() + bar.get_width()/2., height,
                        f'{int(count)}', ha='center', va='bottom', 
                        fontsize=12, fontweight='bold')
        
        # ========== Panel 5: Efficiency Summary ==========
        ax5 = fig.add_subplot(gs[1, 2])
        ax5.axis('off')
        
        # Calculate summary statistics
        total_simulations = len(y_samples)
        total_cost_units = cumulative_cost[-1] if cumulative_cost else 0
        avg_cost_per_iter = total_cost_units / total_simulations if total_simulations > 0 else 0
        
        if self.optimizer.baseline_loss:
            total_improvement = self.optimizer.baseline_loss - self.optimizer.best_loss
            improvement_pct = total_improvement / self.optimizer.baseline_loss * 100
            cost_per_pct_improvement = total_cost_units / improvement_pct if improvement_pct > 0 else np.inf
        else:
            total_improvement = 0
            improvement_pct = 0
            cost_per_pct_improvement = np.inf
        
        # Phase efficiency
        phase_efficiency = []
        for i in range(3):
            if phase_costs[i] > 0 and best_found_phase[i] > 0:
                eff = best_found_phase[i] / phase_costs[i]
                phase_efficiency.append(eff)
            else:
                phase_efficiency.append(0)
        
        summary_lines = [
            "COMPUTATIONAL EFFICIENCY SUMMARY",
            "=" * 35,
            "",
            f"Total Iterations: {total_simulations}",
            f"Total Cost: {total_cost_units:.1f} units",
            f"  (1 unit = one 30-day simulation)",
            f"Avg Cost/Iter: {avg_cost_per_iter:.2f} units",
            "",
            f"Total Improvement: {improvement_pct:.1f}%",
            f"Cost per 1% Improvement: {cost_per_pct_improvement:.2f} units",
            "",
            "Phase Efficiency (improvements/cost):",
            f"  Initial: {phase_efficiency[0]:.3f}",
            f"  Fast (30d): {phase_efficiency[1]:.3f}",
            f"  Full (180d): {phase_efficiency[2]:.3f}",
            "",
            "INTERPRETATION:",
            "• Higher efficiency = more improvements",
            "  per computational cost",
            "• Fast phase should have high efficiency",
            "• Full phase validates with precision",
        ]
        
        summary_text = '\n'.join(summary_lines)
        ax5.text(0.05, 0.95, summary_text, transform=ax5.transAxes,
                fontsize=9, verticalalignment='top', family='monospace',
                bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))
        
        plt.suptitle('Computational Efficiency Analysis', 
                    fontsize=16, fontweight='bold', y=0.995)
        
        plt.savefig(save_path, bbox_inches='tight', dpi=300)
        print(f"✓ Saved efficiency analysis: {save_path}")
        plt.close()
    
    def create_all_plots(self):
        """Generate all visualization plots"""
        print("\n" + "="*70)
        print("GENERATING VISUALIZATION SUITE")
        print("="*70)
        
        self.plot_comprehensive_analysis()
        self.plot_parameter_sensitivity_heatmap()
        self.plot_computational_efficiency()
        
        print("="*70)
        print("✓ All visualizations complete!")
        print("  - optimization_analysis.png: Loss curves, parameters, trust region")
        print("  - parameter_sensitivity.png: Which parameters matter most")
        print("  - computational_efficiency.png: Cost vs improvement analysis")
        print("="*70)

# ============================================================================
# 3-WAY COMPARISON VISUALIZATION
# ============================================================================

def create_three_way_comparison(highres_results, lowres_default_results, lowres_optimized_results, 
                                 save_path='three_way_comparison.png'):
    """
    Compare high-res vs low-res default vs low-res optimized
    Shows spatial fields and quantitative metrics using contourf
    NOTE: Uses last 30 days for all (assumes all are 180-day runs)
    """
    print("\n" + "="*70)
    print("GENERATING 3-WAY COMPARISON")
    print("="*70)
    
    # Compute losses with FIXED window (last 30 days) for fair comparison
    loss_default, fields_default = compute_loss(lowres_default_results, highres_results, 
                                                n_days_avg=30, return_fields=True, adaptive_window=False)
    loss_optimized, fields_optimized = compute_loss(lowres_optimized_results, highres_results, 
                                                    n_days_avg=30, return_fields=True, adaptive_window=False)
    
    # Calculate improvement
    improvement_pct = (loss_default - loss_optimized) / loss_default * 100
    
    # Helper function for consistent contourf plotting
    def add_contourf(ax, data, levels, cmap, title=None):
        cf = ax.contourf(data, levels=levels, cmap=cmap, extend='both', origin='lower')
        ax.set_title(title or "", fontsize=11, fontweight='bold')
        ax.axis('off')
        return cf
    
    # Create figure/grid
    fig = plt.figure(figsize=(20, 14))
    gs = gridspec.GridSpec(4, 3, figure=fig, hspace=0.35, wspace=0.25)
    
    # ========== Row 1: Potential Vorticity Fields (contourf) ==========
    q_ref = fields_default['q_bt_hr_coarse']
    qmin, qmax = float(np.nanmin(q_ref)), float(np.nanmax(q_ref))
    q_levels = np.linspace(qmin, qmax, 31)  # shared discrete levels -> consistent color meaning
    
    ax1 = fig.add_subplot(gs[0, 0])
    cf1 = add_contourf(ax1, q_ref, q_levels, 'RdBu_r',
                       'High-Res (Ground Truth)\nPotential Vorticity')
    plt.colorbar(cf1, ax=ax1, fraction=0.046, pad=0.04)
    
    ax2 = fig.add_subplot(gs[0, 1])
    cf2 = add_contourf(ax2, fields_default['q_bt_lr'], q_levels, 'RdBu_r',
                       f'Low-Res DEFAULT\nLoss: {loss_default:.4f}')
    ax2.title.set_color('red')
    plt.colorbar(cf2, ax=ax2, fraction=0.046, pad=0.04)
    
    ax3 = fig.add_subplot(gs[0, 2])
    cf3 = add_contourf(ax3, fields_optimized['q_bt_lr'], q_levels, 'RdBu_r',
                       f'Low-Res OPTIMIZED\nLoss: {loss_optimized:.4f}')
    ax3.title.set_color('green')
    plt.colorbar(cf3, ax=ax3, fraction=0.046, pad=0.04)
    
    # ========== Row 2: PV Error Maps (contourf) ==========
    error_default = np.abs(fields_default['q_bt_lr'] - fields_default['q_bt_hr_coarse'])
    error_optimized = np.abs(fields_optimized['q_bt_lr'] - fields_optimized['q_bt_hr_coarse'])
    errmax = float(max(np.nanmax(error_default), np.nanmax(error_optimized)))
    err_levels = np.linspace(0.0, errmax, 31)
    
    ax4 = fig.add_subplot(gs[1, 0])
    ax4.text(0.5, 0.5, 'Reference\n(zero error)', ha='center', va='center',
             transform=ax4.transAxes, fontsize=14, fontweight='bold', color='green')
    ax4.axis('off')
    
    ax5 = fig.add_subplot(gs[1, 1])
    cf5 = add_contourf(ax5, error_default, err_levels, 'magma',
                       f'DEFAULT Error (PV)\nNRMSE: {fields_default["loss_q_bt"]:.4f}')
    plt.colorbar(cf5, ax=ax5, fraction=0.046, pad=0.04)
    
    ax6 = fig.add_subplot(gs[1, 2])
    cf6 = add_contourf(ax6, error_optimized, err_levels, 'magma',
                       f'OPTIMIZED Error (PV)\nNRMSE: {fields_optimized["loss_q_bt"]:.4f}')
    plt.colorbar(cf6, ax=ax6, fraction=0.046, pad=0.04)
    
    # ========== Row 3: Streamfunction Fields (contourf, fixed cmap + range) ==========
    psi_ref = fields_default['psi_bt_hr_coarse']
    # symmetric about zero for a proper diverging map
    psi_absmax = float(np.nanmax(np.abs(psi_ref)))
    psi_levels = np.linspace(-psi_absmax, psi_absmax, 41)  # same levels across panels
    
    ax7 = fig.add_subplot(gs[2, 0])
    cf7 = add_contourf(ax7, psi_ref, psi_levels, 'RdBu_r', 'High-Res\nStreamfunction')
    plt.colorbar(cf7, ax=ax7, fraction=0.046, pad=0.04)
    
    ax8 = fig.add_subplot(gs[2, 1])
    cf8 = add_contourf(ax8, fields_default['psi_bt_lr'], psi_levels, 'RdBu_r', 'Low-Res DEFAULT')
    ax8.title.set_color('red')
    plt.colorbar(cf8, ax=ax8, fraction=0.046, pad=0.04)
    
    ax9 = fig.add_subplot(gs[2, 2])
    cf9 = add_contourf(ax9, fields_optimized['psi_bt_lr'], psi_levels, 'RdBu_r', 'Low-Res OPTIMIZED')
    ax9.title.set_color('green')
    plt.colorbar(cf9, ax=ax9, fraction=0.046, pad=0.04)
    
    # ========== Row 4: Metrics Comparison ==========
    ax10 = fig.add_subplot(gs[3, :])
    
    metrics = {
        'Configuration': ['High-Res (Reference)', 'Low-Res DEFAULT', 'Low-Res OPTIMIZED'],
        'PV Loss': [0.0, fields_default['loss_q_bt'], fields_optimized['loss_q_bt']],
        'Streamfn Loss': [0.0, fields_default['loss_psi_bt'], fields_optimized['loss_psi_bt']],
        'Total Loss': [0.0, loss_default, loss_optimized],
    }
    
    x = np.arange(3)
    width = 0.25
    bars1 = ax10.bar(x - width, metrics['PV Loss'], width, label='PV Loss', color='steelblue', alpha=0.8)
    bars2 = ax10.bar(x, metrics['Streamfn Loss'], width, label='Streamfn Loss', color='orange', alpha=0.8)
    bars3 = ax10.bar(x + width, metrics['Total Loss'], width, label='Total Loss', color='green', alpha=0.8)
    
    ax10.set_ylabel('Loss (NRMSE)', fontsize=12, fontweight='bold')
    ax10.set_xticks(x)
    ax10.set_xticklabels(metrics['Configuration'], fontsize=11)
    ax10.legend(fontsize=10, loc='upper left')
    ax10.grid(True, alpha=0.3, axis='y')
    
    ax10.text(0.98, 0.98, f'IMPROVEMENT: {improvement_pct:.1f}%\n({loss_default:.4f} → {loss_optimized:.4f})',
              transform=ax10.transAxes, fontsize=12, fontweight='bold',
              va='top', ha='right', bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8))
    
    for bars in [bars1, bars2, bars3]:
        for bar in bars:
            h = bar.get_height()
            if h > 0:
                ax10.text(bar.get_x() + bar.get_width()/2., h, f'{h:.4f}',
                          ha='center', va='bottom', fontsize=8)
    
    plt.suptitle('3-Way Comparison: High-Res vs Low-Res Default vs Low-Res Optimized',
                 fontsize=16, fontweight='bold', y=0.995)
    
    plt.savefig(save_path, bbox_inches='tight', dpi=300)
    print(f"✓ Saved 3-way comparison: {save_path}")
    plt.close()
    
    # Print summary
    print("\n" + "="*70)
    print("COMPARISON SUMMARY (All using last 30 days)")
    print("="*70)
    print(f"High-Res (Reference):")
    print(f"  Resolution: {highres_results['config']['nx']}x{highres_results['config']['ny']}")
    print(f"  Time window: last 30 days (equilibrated state)")
    print(f"\nLow-Res DEFAULT:")
    print(f"  Resolution: {lowres_default_results['config']['nx']}x{lowres_default_results['config']['ny']}")
    print(f"  PV Loss: {fields_default['loss_q_bt']:.6f}")
    print(f"  Streamfn Loss: {fields_default['loss_psi_bt']:.6f}")
    print(f"  Total Loss: {loss_default:.6f}")
    print(f"\nLow-Res OPTIMIZED:")
    print(f"  Resolution: {lowres_optimized_results['config']['nx']}x{lowres_optimized_results['config']['ny']}")
    print(f"  PV Loss: {fields_optimized['loss_q_bt']:.6f}")
    print(f"  Streamfn Loss: {fields_optimized['loss_psi_bt']:.6f}")
    print(f"  Total Loss: {loss_optimized:.6f}")
    print(f"\n{Colors.star(f'IMPROVEMENT: {improvement_pct:.1f}%')}")
    print("="*70)

# Enhanced GP Optimizer with visualization and warm-start
class EnhancedGPOptimizer:
    """Enhanced GP with visualization, warm-start, and 3-way comparison"""
    
    def __init__(self, n_initial_samples=15, random_seed=42):
        self.n_initial_samples = n_initial_samples
        self.random_seed = random_seed  # Store for reproducibility
        self.X_samples, self.y_samples, self.detailed_outputs = [], [], []
        self.best_loss, self.best_params, self.best_iteration = np.inf, None, -1
        self.iteration, self.iterations_without_improvement = 0, 0
        self.stagnation_threshold = 15
        self.gp = EnsembleGP(n_models=8)
        self.trust_region = TrustRegion()
        self.importance_history = []
        self.use_thompson_sampling_prob = 0.1
        self.baseline_loss = None  # Track default params loss at FINAL fidelity
        self.baseline_loss_by_fidelity = {}  # Track baseline for each fidelity level
        self.default_results = None  # Store default results for comparison
        self.current_fidelity = None  # Track current fidelity level
        self.best_params_original_iteration = None  # Track when best params were first discovered
        
        # Set numpy random seed for reproducibility
        np.random.seed(random_seed)
    
    def optimize(self, config_base, highres_results, max_iterations=100):
        print("\n" + "="*70)
        print("ENHANCED GP: WARM-START + 2-LEVEL MULTI-FIDELITY + VISUALIZATION")
        print("="*70)
        print("Features:")
        print("  ✓ Warm-start from reference parameters")
        print("  ✓ 8-model weighted ensemble")
        print("  ✓ Local penalization (space coverage)")
        print("  ✓ 2-level multi-fidelity strategy:")
        print("    • Iterations 0-40:  30-day runs (fast exploration, ~6x speedup)")
        print("    • Iterations 40+:   180-day runs (full precision)")
        print("  ✓ Adaptive time windows for fair comparison:")
        print("    • 30-day runs: compare entire simulation (days 0-30)")
        print("    • 180-day runs: compare last 30 days (equilibrated)")
        print("  ✓ Fidelity-aware baseline tracking")
        print("  ✓ Thompson sampling (10% exploration)")
        print("  ✓ Anti-stagnation (auto-restart)")
        print("  ✓ 3-way comparison visualization")
        print(f"  Max iterations: {max_iterations}")
        print("="*70)
        
        # Phase 1: Initial sampling with WARM-START
        n_existing = len(self.X_samples)
        if n_existing < self.n_initial_samples:
            print(f"\n{'='*70}\nPHASE 1: WARM-START INITIALIZATION (seed={self.random_seed})\n{'='*70}")
            initial_samples = generate_smart_initial_samples(self.n_initial_samples, 
                                                            include_default=True,
                                                            base_seed=self.random_seed)
            
            for i, params in enumerate(initial_samples[n_existing:]):
                iter_num = i + n_existing
                is_default = (iter_num == 0 and n_existing == 0)  # First sample is default
                
                print(f"\n[Initial {iter_num+1}/{self.n_initial_samples}]" + 
                      (Colors.star(" DEFAULT PARAMETERS") if is_default else ""))
                
                loss, results, detailed = run_lowres_with_params(
                    params, config_base, highres_results, iteration=iter_num
                )
                self.X_samples.append(params)
                self.y_samples.append(loss)
                self.detailed_outputs.append(detailed)
                
                # Track baseline from default params
                if is_default and np.isfinite(loss):
                    _, fidelity_desc = get_adaptive_sim_days(iter_num)
                    self.current_fidelity = fidelity_desc
                    self.baseline_loss = loss
                    self.baseline_loss_by_fidelity[fidelity_desc] = loss
                    self.default_results = results
                    print(Colors.cyan(f"  → Baseline loss at {fidelity_desc}: {loss:.6f}"))
                
                if np.isfinite(loss) and loss < self.best_loss:
                    self.best_loss, self.best_params = loss, params.copy()
                    self.best_iteration, self.iterations_without_improvement = len(self.X_samples) - 1, 0
                    self.best_params_original_iteration = iter_num  # Track discovery iteration
                    if is_default:
                        print(Colors.star(f"BASELINE SET: {Colors.green(f'{loss:.6f}')}"))
                    else:
                        # Compare to baseline at SAME fidelity
                        _, fidelity_desc = get_adaptive_sim_days(iter_num)
                        fidelity_baseline = self.baseline_loss_by_fidelity.get(fidelity_desc, self.baseline_loss)
                        if fidelity_baseline:
                            improvement = (fidelity_baseline - loss) / fidelity_baseline * 100
                            print(Colors.star(f"NEW BEST: {Colors.green(f'{loss:.6f}')} ({improvement:+.1f}% vs baseline @ {fidelity_desc})"))
                        else:
                            print(Colors.star(f"NEW BEST: {Colors.green(f'{loss:.6f}')}"))
                self.save_progress()
        
        # Phase 2: Bayesian optimization
        print(f"\n{'='*70}\nPHASE 2: BAYESIAN OPTIMIZATION\n{'='*70}")
        
        for iteration in range(len(self.X_samples), max_iterations):
            self.iteration, self.iterations_without_improvement = iteration, self.iterations_without_improvement + 1
            
            print(f"\n{'='*70}\n{Colors.cyan(f'ITERATION {iteration + 1}/{max_iterations}')}\n{'='*70}")
            
            # Check if fidelity level changed - if so, re-evaluate baseline AND best params
            _, fidelity_desc = get_adaptive_sim_days(iteration)
            if fidelity_desc != self.current_fidelity:
                old_fidelity = self.current_fidelity
                self.current_fidelity = fidelity_desc
                print(Colors.red(f"\n{'='*70}"))
                print(Colors.red(f"⚠ FIDELITY TRANSITION: {old_fidelity} → {fidelity_desc}"))
                print(Colors.red(f"{'='*70}"))
                
                # Re-evaluate baseline at new fidelity if not already done
                if fidelity_desc not in self.baseline_loss_by_fidelity:
                    print(Colors.cyan(f"→ Re-evaluating BASELINE at {fidelity_desc} fidelity..."))
                    default_array = params_dict_to_array(DEFAULT_PARAMS)
                    baseline_loss, baseline_results, _ = run_lowres_with_params(
                        default_array, config_base, highres_results, iteration=iteration
                    )
                    if np.isfinite(baseline_loss):
                        self.baseline_loss_by_fidelity[fidelity_desc] = baseline_loss
                        self.baseline_loss = baseline_loss
                        # Store the results if this is the final fidelity
                        if fidelity_desc == 'FULL (180d)':
                            self.default_results = baseline_results
                        print(Colors.cyan(f"→ Baseline at {fidelity_desc}: {baseline_loss:.6f}"))
                    else:
                        print(Colors.red(f"→ Baseline evaluation failed at {fidelity_desc}"))
                
                # CRITICAL: Re-evaluate current best parameters at new fidelity!
                if self.best_params is not None:
                    print(Colors.yellow(f"\n→ Re-evaluating BEST PARAMETERS at {fidelity_desc} fidelity..."))
                    print(Colors.yellow(f"   Old best loss ({old_fidelity}): {self.best_loss:.6f}"))
                    
                    best_loss_new_fidelity, _, _ = run_lowres_with_params(
                        self.best_params, config_base, highres_results, iteration=iteration
                    )
                    
                    if np.isfinite(best_loss_new_fidelity):
                        old_best = self.best_loss
                        self.best_loss = best_loss_new_fidelity
                        print(Colors.yellow(f"   New best loss ({fidelity_desc}): {best_loss_new_fidelity:.6f}"))
                        
                        # Calculate change
                        change_pct = (best_loss_new_fidelity - old_best) / old_best * 100
                        if change_pct > 0:
                            print(Colors.red(f"   ⚠ Loss INCREASED by {change_pct:.1f}% at higher fidelity"))
                        else:
                            print(Colors.green(f"   ✓ Loss decreased by {-change_pct:.1f}% at higher fidelity"))
                        
                        # Compare to new baseline
                        if fidelity_desc in self.baseline_loss_by_fidelity:
                            improvement = (self.baseline_loss_by_fidelity[fidelity_desc] - best_loss_new_fidelity) / \
                                        self.baseline_loss_by_fidelity[fidelity_desc] * 100
                            print(Colors.cyan(f"   → Improvement vs {fidelity_desc} baseline: {improvement:+.1f}%"))
                    else:
                        print(Colors.red(f"   ✗ Re-evaluation failed, keeping old best loss"))
                
                print(Colors.red(f"{'='*70}\n"))
            
            # Stagnation check
            if self.iterations_without_improvement >= self.stagnation_threshold:
                print(Colors.red(f"\n⚠ STAGNATION: {self.iterations_without_improvement} iterations w/o improvement"))
                print(Colors.yellow("→ Triggering exploration restart"))
                self.trigger_exploration_restart()
            
            # Fit GP
            X_warped = np.array([warp_parameters(x) for x in self.X_samples])
            y_array = np.array(self.y_samples)
            valid_mask = np.isfinite(y_array)
            n_valid = np.sum(valid_mask)
            
            print(f"Valid samples: {Colors.cyan(str(n_valid))}/{len(y_array)}")
            
            kappa = self.get_adaptive_kappa()
            if kappa > 2.0:
                print(Colors.yellow(f"  ℹ Increased exploration: kappa = {kappa:.1f}"))
            
            # Thompson sampling with some probability
            use_thompson = np.random.rand() < self.use_thompson_sampling_prob
            
            if n_valid < 5:
                print(Colors.yellow("  ⚠ Too few valid samples, random exploration"))
                next_params = unwarp_parameters(np.random.uniform(0, 1, N_PARAMS))
            elif use_thompson:
                print(Colors.cyan("  → Using Thompson sampling for exploration"))
                X_valid, y_valid = X_warped[valid_mask], y_array[valid_mask]
                self.gp.fit(X_valid, y_valid)
                
                # Track parameter importance even with Thompson sampling
                importance = self.gp.get_parameter_importance()
                self.importance_history.append(importance)
                
                # Print top 3 most important parameters
                sorted_indices = np.argsort(importance)[::-1][:3]
                print("  Top 3 important parameters:")
                for rank, idx in enumerate(sorted_indices, 1):
                    print(f"    {rank}. {PARAM_NAMES[idx]}: {importance[idx]:.3f}")
                
                tr_bounds = self.trust_region.get_trust_region_bounds()
                thompson_sample = thompson_sampling(self.gp, tr_bounds, n_samples=1)[0]
                next_params = unwarp_parameters(thompson_sample)
            else:
                X_valid, y_valid = X_warped[valid_mask], y_array[valid_mask]
                print("  Fitting 8-model ensemble GP...")
                self.gp.fit(X_valid, y_valid)
                
                # Track parameter importance
                importance = self.gp.get_parameter_importance()
                self.importance_history.append(importance)
                
                # Print parameter importance (show relative values)
                print("  Parameter importance (relative):")
                sorted_indices = np.argsort(importance)[::-1]  # Sort descending
                for rank, idx in enumerate(sorted_indices, 1):
                    name = PARAM_NAMES[idx]
                    imp_val = importance[idx]
                    if rank <= 3:
                        imp_str = f"{Colors.green('HIGH')}"
                    elif rank <= 5:
                        imp_str = f"{Colors.cyan('med')}"
                    else:
                        imp_str = "low"
                    print(f"    {rank}. {name}: {imp_val:.3f} ({imp_str})")
                
                # Update trust region
                if self.best_params is not None:
                    self.trust_region.best_center = warp_parameters(self.best_params)
                
                tr_bounds = self.trust_region.get_trust_region_bounds()
                print(f"  Trust region: {Colors.cyan(f'{self.trust_region.trust_radius:.2f}')}")
                
                # Optimize acquisition
                print(f"  Optimizing acquisition (kappa={kappa:.1f})...")
                best_y = np.min(y_valid)
                acq_fn = lambda X: hybrid_acquisition_with_penalization(
                    X, self.gp, best_y, self.X_samples, xi=0.01, kappa=kappa, penalization_weight=0.3
                )
                
                next_params_warped = optimize_acquisition_multistart(acq_fn, tr_bounds, n_starts=20, n_random=1000)
                acq_val = acq_fn(next_params_warped.reshape(1, -1))[0]
                print(f"  Selected point (acq={Colors.cyan(f'{acq_val:.4f}')})")
                next_params = unwarp_parameters(next_params_warped)
            
            # Evaluate with adaptive fidelity
            loss, results, detailed = run_lowres_with_params(
                next_params, config_base, highres_results, iteration=iteration
            )
            self.X_samples.append(next_params)
            self.y_samples.append(loss)
            self.detailed_outputs.append(detailed)
            
            # Update best
            new_best = False
            if np.isfinite(loss) and loss < self.best_loss:
                self.best_loss, self.best_params = loss, next_params.copy()
                self.best_iteration, self.iterations_without_improvement = iteration, 0
                self.best_params_original_iteration = iteration  # Track discovery iteration
                new_best = True
                # Compare to baseline at CURRENT fidelity
                fidelity_baseline = self.baseline_loss_by_fidelity.get(self.current_fidelity, self.baseline_loss)
                if fidelity_baseline:
                    improvement = (fidelity_baseline - loss) / fidelity_baseline * 100
                    print(Colors.star(f"NEW BEST: {Colors.green(f'{loss:.6f}')} ({improvement:+.1f}% vs baseline @ {self.current_fidelity})"))
                else:
                    print(Colors.star(f"NEW BEST: {Colors.green(f'{loss:.6f}')}"))
            
            self.trust_region.update(new_best, warp_parameters(self.best_params) if new_best else None)
            self.print_status()
            self.save_progress()
            
            # Generate plots every 10 iterations
            if (iteration + 1) % 10 == 0:
                print("\n  Generating visualization...")
                visualizer = OptimizationVisualizer(self)
                visualizer.create_all_plots()
        
        # Final visualization
        print("\n" + "="*70)
        print("GENERATING FINAL VISUALIZATIONS")
        print("="*70)
        visualizer = OptimizationVisualizer(self)
        visualizer.create_all_plots()
        
        return self.get_best_params()
    
    def get_adaptive_kappa(self):
        """Adaptive kappa: higher when stuck"""
        if self.iterations_without_improvement < 6:
            return 2.0
        elif self.iterations_without_improvement < 10:
            return 3.0
        return 4.0
    
    def trigger_exploration_restart(self):
        """Reset trust region and add random sample"""
        self.trust_region.reset_for_exploration()
        print(Colors.yellow("  → Random sample will be added next"))
        self.iterations_without_improvement = 0
        print(Colors.green("  ✓ Restart complete"))
    
    def print_status(self):
        """Print status with fidelity-aware baseline comparison"""
        n_valid = np.sum(np.isfinite(self.y_samples))
        n_failed = len(self.y_samples) - n_valid
        print(f"\n{Colors.bold('Status:')}")
        print(f"  Valid: {Colors.cyan(str(n_valid))}/{len(self.y_samples)}")
        print(f"  Failed: {Colors.yellow(str(n_failed))}")
        
        # Show current fidelity
        if self.current_fidelity:
            print(f"  Current fidelity: {Colors.cyan(self.current_fidelity)}")
        
        # Show baselines for each fidelity
        if self.baseline_loss_by_fidelity:
            print(f"  Baselines by fidelity:")
            for fidelity, baseline in sorted(self.baseline_loss_by_fidelity.items()):
                print(f"    {fidelity}: {Colors.cyan(f'{baseline:.6f}')}")
        
        # Compare best to baseline at current fidelity
        if self.best_params_original_iteration is not None:
            print(f"  {Colors.bold('Best loss:')} {Colors.green(f'{self.best_loss:.6f}')} " +
                  Colors.cyan(f'(discovered at iteration {self.best_params_original_iteration + 1})'))
        else:
            print(f"  {Colors.bold('Best loss:')} {Colors.green(f'{self.best_loss:.6f}')} " +
                  Colors.cyan(f'(iteration {self.best_iteration + 1})'))
        
        if self.current_fidelity and self.current_fidelity in self.baseline_loss_by_fidelity:
            fidelity_baseline = self.baseline_loss_by_fidelity[self.current_fidelity]
            improvement = (fidelity_baseline - self.best_loss) / fidelity_baseline * 100
            print(f"    → vs {self.current_fidelity} baseline: {Colors.green(f'{improvement:+.1f}%')}")
        
        stag_str = f"{self.iterations_without_improvement}/{self.stagnation_threshold}"
        stag_str = Colors.yellow(stag_str) if self.iterations_without_improvement >= 10 else Colors.cyan(stag_str)
        print(f"  Iterations w/o improvement: {stag_str}")
    
    def get_best_params(self):
        if self.best_params is None:
            raise ValueError("No valid parameters found!")
        return {PARAM_NAMES[i]: float(self.best_params[i]) for i in range(N_PARAMS)}
    
    def save_progress(self, filename='enhanced_gp_progress.pkl'):
        data = {
            'X_samples': self.X_samples, 'y_samples': self.y_samples, 'detailed_outputs': self.detailed_outputs,
            'best_loss': self.best_loss, 'best_params': self.best_params, 'best_iteration': self.best_iteration,
            'best_params_original_iteration': self.best_params_original_iteration,
            'iteration': self.iteration, 'iterations_without_improvement': self.iterations_without_improvement,
            'trust_region_state': {'radius': self.trust_region.trust_radius, 'center': self.trust_region.best_center,
                                  'success_count': self.trust_region.success_count, 'fail_count': self.trust_region.fail_count},
            'importance_history': self.importance_history,
            'baseline_loss': self.baseline_loss,
            'baseline_loss_by_fidelity': self.baseline_loss_by_fidelity,
            'current_fidelity': self.current_fidelity,
            'default_results': self.default_results,
            'random_seed': self.random_seed
        }
        with open(filename, 'wb') as f:
            pickle.dump(data, f)
        print(f"  ✓ Progress saved")
    
    @classmethod
    def load_progress(cls, filename='enhanced_gp_progress.pkl'):
        with open(filename, 'rb') as f:
            data = pickle.load(f)
        
        optimizer = cls(random_seed=data.get('random_seed', 42))
        optimizer.X_samples, optimizer.y_samples = data['X_samples'], data['y_samples']
        optimizer.detailed_outputs = data['detailed_outputs']
        optimizer.best_loss, optimizer.best_params = data['best_loss'], data['best_params']
        optimizer.best_iteration, optimizer.iteration = data['best_iteration'], data['iteration']
        optimizer.best_params_original_iteration = data.get('best_params_original_iteration', optimizer.best_iteration)
        optimizer.iterations_without_improvement = data.get('iterations_without_improvement', 0)
        optimizer.importance_history = data.get('importance_history', [])
        optimizer.baseline_loss = data.get('baseline_loss', None)
        optimizer.baseline_loss_by_fidelity = data.get('baseline_loss_by_fidelity', {})
        optimizer.current_fidelity = data.get('current_fidelity', None)
        optimizer.default_results = data.get('default_results', None)
        
        if 'trust_region_state' in data:
            tr = data['trust_region_state']
            optimizer.trust_region.trust_radius, optimizer.trust_region.best_center = tr['radius'], tr['center']
            optimizer.trust_region.success_count, optimizer.trust_region.fail_count = tr['success_count'], tr['fail_count']
        
        print(f"✓ Loaded checkpoint (seed={optimizer.random_seed}):")
        print(f"  Iterations: {len(optimizer.X_samples)}")
        n_valid = np.sum(np.isfinite(optimizer.y_samples))
        print(f"  Valid: {Colors.cyan(str(n_valid))}/{len(optimizer.y_samples)}")
        
        if optimizer.baseline_loss_by_fidelity:
            print(f"  Baselines by fidelity:")
            for fidelity, baseline in sorted(optimizer.baseline_loss_by_fidelity.items()):
                print(f"    {fidelity}: {Colors.cyan(f'{baseline:.6f}')}")
        
        print(f"  {Colors.bold('Best loss:')} {Colors.green(f'{optimizer.best_loss:.6f}')} " +
              Colors.cyan(f'(discovered at iteration {optimizer.best_params_original_iteration + 1})'))
        
        if optimizer.current_fidelity and optimizer.current_fidelity in optimizer.baseline_loss_by_fidelity:
            fidelity_baseline = optimizer.baseline_loss_by_fidelity[optimizer.current_fidelity]
            improvement = (fidelity_baseline - optimizer.best_loss) / fidelity_baseline * 100
            print(f"    → vs {optimizer.current_fidelity}: {Colors.green(f'{improvement:+.1f}%')}")
        
        return optimizer

# Main function with 3-way comparison
def main(checkpoint_file='enhanced_gp_progress.pkl', max_iterations=100, random_seed=42):
    """
    Main optimization routine with 3-way comparison
    
    Args:
        checkpoint_file: Path to checkpoint file for resuming
        max_iterations: Maximum number of optimization iterations
        random_seed: Random seed for reproducibility (affects initial sampling and exploration)
    """
    if not os.path.exists('highres_results.pkl'):
        print("\n✗ Error: highres_results.pkl not found!")
        return
    
    with open('highres_results.pkl', 'rb') as f:
        highres_results = pickle.load(f)
    print(f"\n✓ Loaded high-res: {highres_results['config']['nx']}x{highres_results['config']['ny']}")
    
    from main_comparison import config_lowres
    config_base = config_lowres.copy()
    
    if os.path.exists(checkpoint_file):
        print(f"\n✓ Checkpoint found")
        optimizer = EnhancedGPOptimizer.load_progress(checkpoint_file)
    else:
        print(f"\n✓ Starting new optimization (seed={random_seed})")
        optimizer = EnhancedGPOptimizer(n_initial_samples=18, random_seed=random_seed)
    
    best_params = optimizer.optimize(config_base, highres_results, max_iterations)
    
    print("\n" + "="*70)
    print("OPTIMIZATION COMPLETE")
    print("="*70)
    
    n_valid = np.sum(np.isfinite(optimizer.y_samples))
    n_failed = len(optimizer.y_samples) - n_valid
    print(f"\nTotal iterations: {len(optimizer.y_samples)}")
    print(f"  Valid: {Colors.cyan(str(n_valid))}")
    print(f"  Failed: {Colors.yellow(str(n_failed))}")
    
    # Show all baselines
    if optimizer.baseline_loss_by_fidelity:
        print(f"\nBaselines by fidelity:")
        for fidelity, baseline in sorted(optimizer.baseline_loss_by_fidelity.items()):
            print(f"  {fidelity}: {Colors.cyan(f'{baseline:.6f}')}")
    
    # Final comparison at FULL fidelity
    final_baseline = optimizer.baseline_loss_by_fidelity.get('FULL (180d)', optimizer.baseline_loss)
    if final_baseline:
        improvement = (final_baseline - optimizer.best_loss) / final_baseline * 100
        print(f"\n{Colors.bold('Final Comparison at FULL (180d) Fidelity:')}")
        print(f"  Baseline (default): {Colors.cyan(f'{final_baseline:.6f}')}")
        print(f"  Best loss: {Colors.green(f'{optimizer.best_loss:.6f}')} " +
              Colors.green(f'[{improvement:+.1f}% improvement]'))
        
        # Show discovery info
        if optimizer.best_params_original_iteration is not None:
            print(f"  Best parameters discovered at: iteration {optimizer.best_params_original_iteration + 1}")
            if optimizer.best_params_original_iteration < 40:
                print(Colors.yellow(f"    (during 30-day fast exploration phase)"))
                print(Colors.cyan(f"    Loss was re-evaluated at full 180-day fidelity"))
            else:
                print(Colors.cyan(f"    (during 180-day full precision phase)"))
    else:
        print(f"\n{Colors.bold('Best loss:')} {Colors.green(f'{optimizer.best_loss:.6f}')}")
        print(Colors.yellow("  Note: No full-fidelity baseline available"))
    
    print(f"\n{Colors.bold('Best parameters:')}")
    for name, val in best_params.items():
        default_val = DEFAULT_PARAMS[name]
        change = (val - default_val) / default_val * 100 if default_val != 0 else 0
        print(f"  {name}: {Colors.cyan(f'{val:.6e}')} (default: {default_val:.6e}, {change:+.1f}%)")
    
    # Save results
    with open('enhanced_gp_optimal_params.pkl', 'wb') as f:
        pickle.dump(best_params, f)
    with open('enhanced_gp_optimal_config.txt', 'w') as f:
        f.write("'subgrid_params': {\n")
        for name, val in best_params.items():
            f.write(f"    '{name}': {val:.6e},\n")
        f.write("}\n")
    
    print("\n✓ Saved: enhanced_gp_optimal_params.pkl")
    print("✓ Saved: enhanced_gp_optimal_config.txt")
    
    print("\n" + "="*70)
    print("NOTE: 2-LEVEL MULTI-FIDELITY WITH ADAPTIVE BASELINES")
    print("="*70)
    print("The optimizer uses a simple 2-level fidelity strategy:")
    print("  Phase 1 (iterations 0-40):  30-day runs (~6x faster)")
    print("    - Compares entire simulation (days 0-30)")
    print("    - Baseline tracked at 30-day fidelity")
    print("  Phase 2 (iterations 40+):   180-day runs (full precision)")
    print("    - Compares last 30 days (equilibrated state)")
    print("    - Baseline tracked at 180-day fidelity")
    print("\nThis ensures:")
    print("  ✓ Fast exploration in early iterations")
    print("  ✓ Fair apples-to-apples comparisons at each fidelity")
    print("  ✓ Final results use full 180-day simulations")
    print("\nSeed robustness:")
    print(f"  ✓ Random seed used: {optimizer.random_seed}")
    print("  ✓ Multiple complementary seeds used internally")
    print("  ✓ Small perturbations added to reduce grid artifacts")
    print("  ℹ Different seeds may find best at different iterations")
    print("    but final performance should be similar (~5-10% variation)")
    print("="*70)
    
    # Run final simulation with optimized parameters for 3-way comparison
    print("\n" + "="*70)
    print("RUNNING FINAL COMPARISON SIMULATIONS")
    print("="*70)
    
    # Ensure we have full-fidelity baseline (180 days)
    if 'FULL (180d)' not in optimizer.baseline_loss_by_fidelity or optimizer.default_results is None:
        print("\nRunning default parameters at FULL fidelity (180 days)...")
        config_default = config_base.copy()
        config_default['subgrid_params'] = DEFAULT_PARAMS
        from main_comparison import run_simulation
        optimizer.default_results = run_simulation(config_default, sim_days=180, save_interval_hours=12)
        
        # Compute baseline loss at full fidelity
        baseline_loss_full, _ = compute_loss(optimizer.default_results, highres_results, 
                                             n_days_avg=30, return_fields=False, adaptive_window=False)
        optimizer.baseline_loss_by_fidelity['FULL (180d)'] = baseline_loss_full
        optimizer.baseline_loss = baseline_loss_full
        print(f"  ✓ Default simulation complete - Loss: {baseline_loss_full:.6f}")
    else:
        print("\n✓ Using cached default results at FULL fidelity")
    
    # Run optimized parameters simulation (full 180 days for final comparison)
    print("\nRunning optimized parameters simulation (full 180 days)...")
    config_optimized = config_base.copy()
    config_optimized['subgrid_params'] = best_params
    from main_comparison import run_simulation
    optimized_results = run_simulation(config_optimized, sim_days=180, save_interval_hours=12)
    print(f"  ✓ Optimized simulation complete")
    
    # Create 3-way comparison
    create_three_way_comparison(highres_results, optimizer.default_results, optimized_results)
    
    print("\n✓ Saved: optimization_analysis.png")
    print("✓ Saved: parameter_sensitivity.png")
    print("✓ Saved: computational_efficiency.png")
    print("✓ Saved: three_way_comparison.png")
    
    print("\n" + "="*70)
    print("VISUALIZATION GUIDE")
    print("="*70)
    print("1. optimization_analysis.png")
    print("   → Loss evolution, parameter trajectories, trust region")
    print("   → Shows HOW the optimization progressed")
    print("")
    print("2. parameter_sensitivity.png")
    print("   → Correlation analysis, variance explained, ranges explored")
    print("   → Shows WHICH parameters matter most")
    print("   → Red = increasing parameter worsens loss")
    print("   → Green = increasing parameter improves loss")
    print("")
    print("3. computational_efficiency.png")
    print("   → Cost vs improvement, phase breakdown, sample efficiency")
    print("   → Shows HOW EFFICIENTLY we found improvements")
    print("")
    print("4. three_way_comparison.png")
    print("   → Spatial fields: high-res vs default vs optimized")
    print("   → Shows FINAL RESULTS quality")
    print("="*70)
    
    return optimizer, best_params

if __name__ == "__main__":
    # You can change the random_seed parameter to test different initializations
    # The optimizer uses multiple complementary seeds internally for robustness
    optimizer, best_params = main(max_iterations=80, random_seed=42)

/panfs/ccds02/nobackup/people/afahad/pkgs/qg/lib/python3.10/site-packages/scipy/stats/_qmc.py:993: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  sample = self._random(n, workers=workers)



✓ Loaded high-res: 512x256

✓ Starting new optimization (seed=42)

ENHANCED GP: WARM-START + 2-LEVEL MULTI-FIDELITY + VISUALIZATION
Features:
  ✓ Warm-start from reference parameters
  ✓ 8-model weighted ensemble
  ✓ Local penalization (space coverage)
  ✓ 2-level multi-fidelity strategy:
    • Iterations 0-40:  30-day runs (fast exploration, ~6x speedup)
    • Iterations 40+:   180-day runs (full precision)
  ✓ Adaptive time windows for fair comparison:
    • 30-day runs: compare entire simulation (days 0-30)
    • 180-day runs: compare last 30 days (equilibrated)
  ✓ Fidelity-aware baseline tracking
  ✓ Thompson sampling (10% exploration)
  ✓ Anti-stagnation (auto-restart)
  ✓ 3-way comparison visualization
  Max iterations: 80

PHASE 1: WARM-START INITIALIZATION (seed=42)
  ⚠ Clipped eddy_diffusivity: 5.000000e-03 → 1.000000e+03 (bounds: [1.000000e+03, 1.000000e+05])
  ✓ Including reference parameters as warm-start
  ✓ Generated 18 diverse initial samples (base_seed=42)

[Initial 1

100%|██████████| 1440/1440 [00:04<00:00, 324.87it/s]



LowRes_64x32 Simulation Complete!
  → Using entire simulation (days 0-30) for loss
  Loss: 0.206844
  → Baseline loss at FAST (30d): 0.206844
★ BASELINE SET: 0.206844
  ✓ Progress saved

[Initial 2/18]

Testing parameters - Fidelity: FAST (30d)
  viscosity_scale: 6.774869e-01
  drag_scale: 1.989922e+00
  eddy_diffusivity: 1.337498e+03
  smagorinsky_coeff: 1.544568e-02
  energy_correction: 7.164037e-03
  enstrophy_correction: 3.650586e-07

Running LowRes_64x32 Simulation
Grid: 64 x 32
Resolution: 31.2 km per grid point

Subgrid Parameters:
  viscosity_scale: 0.6774869474036614
  drag_scale: 1.9899222664208966
  eddy_diffusivity: 1337.4975968397164
  smagorinsky_coeff: 0.015445681389968675
  energy_correction: 0.007164036771964243
  enstrophy_correction: 3.650586104933196e-07

Initial Energy: 5.940e+02
Initial Enstrophy: 8.404e-12

Integrating...


100%|██████████| 1440/1440 [00:04<00:00, 326.63it/s]


...


LowRes_64x32 Simulation Complete!
  → Using last 30 days for loss (equilibrated state)
  Loss: 0.435952
  → Trust region shrunk to 0.40

Status:
  Valid: 79/79
  Failed: 0
  Current fidelity: FULL (180d)
  Baselines by fidelity:
    FAST (30d): 0.206844
    FULL (180d): 0.628715
  Best loss: 0.129293 (discovered at iteration 47)
    → vs FULL (180d) baseline: +79.4%
  Iterations w/o improvement: 2/15
  ✓ Progress saved

ITERATION 80/80
Valid samples: 79/79
  → Using Thompson sampling for exploration
  Top 3 important parameters:
    1. energy_correction: 6.120
    2. smagorinsky_coeff: 2.341
    3. viscosity_scale: 1.277

Testing parameters - Fidelity: FULL (180d)
  viscosity_scale: 1.649916e+00
  drag_scale: 1.490896e+00
  eddy_diffusivity: 7.947503e+04
  smagorinsky_coeff: 2.539015e-01
  energy_correction: -3.275937e-03
  enstrophy_correction: 8.322113e-10

Running LowRes_64x32 Simulation
Grid: 64 x 32
Resolution: 31.2 km per grid point

Subgrid Parameters:
  viscosity_scale: 1.6499

100%|██████████| 8640/8640 [00:25<00:00, 335.49it/s]



LowRes_64x32 Simulation Complete!
  → Using last 30 days for loss (equilibrated state)
  Loss: 1.269184

Status:
  Valid: 80/80
  Failed: 0
  Current fidelity: FULL (180d)
  Baselines by fidelity:
    FAST (30d): 0.206844
    FULL (180d): 0.628715
  Best loss: 0.129293 (discovered at iteration 47)
    → vs FULL (180d) baseline: +79.4%
  Iterations w/o improvement: 3/15
  ✓ Progress saved

  Generating visualization...

GENERATING VISUALIZATION SUITE

✓ Saved comprehensive analysis: optimization_analysis.png
✓ Saved sensitivity analysis: parameter_sensitivity.png
✓ Saved efficiency analysis: computational_efficiency.png
✓ All visualizations complete!
  - optimization_analysis.png: Loss curves, parameters, trust region
  - parameter_sensitivity.png: Which parameters matter most
  - computational_efficiency.png: Cost vs improvement analysis

GENERATING FINAL VISUALIZATIONS

GENERATING VISUALIZATION SUITE

✓ Saved comprehensive analysis: optimization_analysis.png
✓ Saved sensitivity analy

100%|██████████| 8640/8640 [00:25<00:00, 336.08it/s]



LowRes_64x32 Simulation Complete!
  ✓ Optimized simulation complete

GENERATING 3-WAY COMPARISON
✓ Saved 3-way comparison: three_way_comparison.png

COMPARISON SUMMARY (All using last 30 days)
High-Res (Reference):
  Resolution: 512x256
  Time window: last 30 days (equilibrated state)

Low-Res DEFAULT:
  Resolution: 64x32
  PV Loss: 0.726491
  Streamfn Loss: 0.482050
  Total Loss: 0.628715

Low-Res OPTIMIZED:
  Resolution: 64x32
  PV Loss: 0.176751
  Streamfn Loss: 0.058104
  Total Loss: 0.129293

★ IMPROVEMENT: 79.4%

✓ Saved: optimization_analysis.png
✓ Saved: parameter_sensitivity.png
✓ Saved: computational_efficiency.png
✓ Saved: three_way_comparison.png

VISUALIZATION GUIDE
1. optimization_analysis.png
   → Loss evolution, parameter trajectories, trust region
   → Shows HOW the optimization progressed

2. parameter_sensitivity.png
   → Correlation analysis, variance explained, ranges explored
   → Shows WHICH parameters matter most
   → Red = increasing parameter worsens loss
   

In [2]:
"""
Enhanced Robust GP Optimizer with Warm-Start & 3-Way Comparison

NEW FEATURES:
1. Warm-start from reference parameters (baseline-aware)
2. Dynamic multi-fidelity optimization (30d → 180d with parameter-based thresholds)
3. Thompson sampling exploration
4. 3-way comparison: high-res vs low-res default vs low-res optimized (contourf plots)
5. Comprehensive visualization with improvement metrics
6. Seed-robust initialization using multiple complementary random sequences
7. Separate best-loss tracking per fidelity level (no confusing jumps!)
8. Dynamic sampling: 4*N_PARAMS initial, 2*N_PARAMS fast, 2*N_PARAMS full
"""

import numpy as np
import pickle
import os
import warnings
from scipy.stats import qmc, norm
from scipy.optimize import minimize
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, RBF, ConstantKernel, WhiteKernel
from sklearn.linear_model import LinearRegression
from qg_model import QGTwoLayerModel
from scipy.ndimage import uniform_filter
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.patches import Rectangle
import seaborn as sns

warnings.filterwarnings('ignore', category=UserWarning, module='sklearn.gaussian_process')
warnings.filterwarnings('ignore', message='The optimal value found for dimension')

class Colors:
    """Compact color printing"""
    @staticmethod
    def green(t): return f"\033[92m{t}\033[0m"
    @staticmethod
    def cyan(t): return f"\033[96m{t}\033[0m"
    @staticmethod
    def yellow(t): return f"\033[93m{t}\033[0m"
    @staticmethod
    def red(t): return f"\033[91m{t}\033[0m"
    @staticmethod
    def bold(t): return f"\033[1m{t}\033[0m"
    @staticmethod
    def star(t): return f"\033[93m\033[1m★ {t}\033[0m"

# Parameter configuration with REFERENCE BASELINE
PARAM_BOUNDS = {
    'viscosity_scale': {'bounds': (0.5, 5.0), 'type': 'linear'},
    'drag_scale': {'bounds': (0.5, 3.0), 'type': 'linear'},
    'eddy_diffusivity': {'bounds': (1e3, 1e5), 'type': 'log'},
    'smagorinsky_coeff': {'bounds': (0.0, 0.3), 'type': 'linear'},
    'energy_correction': {'bounds': (-0.01, 0.01), 'type': 'linear'},
    'enstrophy_correction': {'bounds': (0.0, 1e-6), 'type': 'log'},
}

# REFERENCE/DEFAULT PARAMETERS (known baseline)
# Note: If defaults are outside bounds, they will be clipped automatically
DEFAULT_PARAMS = {
    'viscosity_scale': 0.5,
    'drag_scale': 0.5,
    'eddy_diffusivity': 0.005,  # User-provided default (will be clipped to bounds if needed)
    'smagorinsky_coeff': 0.015,
    'energy_correction': -0.002,
    'enstrophy_correction': 3e-9,
}

PARAM_NAMES = list(PARAM_BOUNDS.keys())
N_PARAMS = len(PARAM_NAMES)

# Input warping for log-scale parameters
def warp_parameters(params_array):
    """Transform to warped space: log params → log space, linear → [0,1]"""
    warped = np.zeros(N_PARAMS)
    for i, name in enumerate(PARAM_NAMES):
        val, info = params_array[i], PARAM_BOUNDS[name]
        lower, upper = info['bounds']
        if info['type'] == 'log':
            log_lower, log_upper = np.log10(lower) if lower > 0 else -10, np.log10(upper)
            warped[i] = (np.log10(val + 1e-20) - log_lower) / (log_upper - log_lower)
        else:
            warped[i] = (val - lower) / (upper - lower)
    return warped

def unwarp_parameters(warped_array):
    """Transform from warped space back to original space"""
    params = np.zeros(N_PARAMS)
    for i, name in enumerate(PARAM_NAMES):
        info = PARAM_BOUNDS[name]
        lower, upper = info['bounds']
        if info['type'] == 'log':
            log_lower, log_upper = np.log10(lower) if lower > 0 else -10, np.log10(upper)
            params[i] = 10 ** (warped_array[i] * (log_upper - log_lower) + log_lower)
        else:
            params[i] = warped_array[i] * (upper - lower) + lower
        params[i] = np.clip(params[i], lower, upper)
    return params

def params_dict_to_array(params_dict):
    """Convert parameter dictionary to array, clipping to bounds"""
    params = []
    for name in PARAM_NAMES:
        val = params_dict[name]
        lower, upper = PARAM_BOUNDS[name]['bounds']
        clipped_val = np.clip(val, lower, upper)
        if clipped_val != val:
            print(Colors.yellow(f"  ⚠ Clipped {name}: {val:.6e} → {clipped_val:.6e} (bounds: [{lower:.6e}, {upper:.6e}])"))
        params.append(clipped_val)
    return np.array(params)

def params_array_to_dict(params_array):
    """Convert parameter array to dictionary"""
    return {PARAM_NAMES[i]: float(params_array[i]) for i in range(N_PARAMS)}

# ============================================================================
# SMART INITIALIZATION WITH WARM-START
# ============================================================================

def generate_smart_initial_samples(n_samples, include_default=True, base_seed=42):
    """
    Combine default params + Latin Hypercube + Sobol for warm-start
    Uses multiple complementary seeds for better diversity and robustness
    
    Args:
        n_samples: Total number of samples
        include_default: If True, first sample is DEFAULT_PARAMS
        base_seed: Base random seed (will generate complementary seeds from this)
    """
    samples = []
    
    # WARM-START: Include default parameters as first sample
    if include_default:
        default_array = params_dict_to_array(DEFAULT_PARAMS)
        samples.append(default_array)
        n_samples -= 1
        print(Colors.cyan("  ✓ Including reference parameters as warm-start"))
    
    # Generate space-filling samples for remaining
    # Use MULTIPLE seeds for robustness - reduces sensitivity to single seed choice
    n_lhs = n_samples // 2
    n_sobol = n_samples - n_lhs
    
    # LHS with primary seed
    lhs_sampler = qmc.LatinHypercube(d=N_PARAMS, seed=base_seed)
    lhs_samples = lhs_sampler.random(n=n_lhs)
    
    # Sobol with complementary seed (offset by 1000)
    # This ensures different quasi-random sequences
    sobol_sampler = qmc.Sobol(d=N_PARAMS, seed=base_seed + 1000, scramble=True)
    sobol_samples = sobol_sampler.random(n=n_sobol)
    
    # Combine samples
    unit_samples = np.vstack([lhs_samples, sobol_samples])
    
    # Add small random perturbations to avoid exact grid points
    # This helps exploration and reduces sensitivity to specific seed values
    np.random.seed(base_seed + 2000)
    perturbations = np.random.normal(0, 0.02, size=unit_samples.shape)
    unit_samples = np.clip(unit_samples + perturbations, 0, 1)
    
    for unit_sample in unit_samples:
        samples.append(unwarp_parameters(unit_sample))
    
    print(Colors.cyan(f"  ✓ Generated {len(samples)} diverse initial samples (base_seed={base_seed})"))
    
    return np.array(samples)

# ============================================================================
# ENSEMBLE GP
# ============================================================================

class EnsembleGP:
    """Ensemble of Gaussian Processes with different kernels"""
    
    def __init__(self, n_models=8):
        self.models = []
        self.model_weights = []
        
        kernels = [
            ConstantKernel(1.0, (1e-3, 1e3)) * 
            Matern(length_scale=[1.0]*N_PARAMS, length_scale_bounds=(1e-3, 1e3), nu=1.5) +
            WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-10, 1e-1)),
            
            ConstantKernel(1.0, (1e-3, 1e3)) * 
            Matern(length_scale=[1.0]*N_PARAMS, length_scale_bounds=(1e-3, 1e3), nu=2.5) +
            WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-10, 1e-1)),
            
            ConstantKernel(1.0, (1e-3, 1e3)) * 
            RBF(length_scale=[1.0]*N_PARAMS, length_scale_bounds=(1e-3, 1e3)) +
            WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-10, 1e-1)),
            
            ConstantKernel(1.0, (1e-3, 1e3)) * 
            Matern(length_scale=[0.5]*N_PARAMS, length_scale_bounds=(1e-3, 1e2), nu=2.5) +
            WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-10, 1e-1)),
            
            ConstantKernel(1.0, (1e-3, 1e3)) * 
            RBF(length_scale=[2.0]*N_PARAMS, length_scale_bounds=(1e-2, 1e3)) +
            WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-10, 1e-1)),
            
            ConstantKernel(1.0, (1e-3, 1e3)) * 
            Matern(length_scale=[0.3]*N_PARAMS, length_scale_bounds=(1e-3, 1e2), nu=1.5) +
            WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-10, 1e-1)),
            
            ConstantKernel(1.0, (1e-3, 1e3)) * 
            RBF(length_scale=[0.7]*N_PARAMS, length_scale_bounds=(1e-3, 1e3)) +
            WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-10, 1e-1)),
            
            ConstantKernel(1.0, (1e-3, 1e3)) * 
            Matern(length_scale=[1.5]*N_PARAMS, length_scale_bounds=(1e-3, 1e3), nu=2.5) +
            WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-10, 1e-1)),
        ]
        
        for kernel in kernels[:n_models]:
            self.models.append(GaussianProcessRegressor(
                kernel=kernel,
                alpha=1e-6,
                normalize_y=True,
                n_restarts_optimizer=15,
                random_state=None
            ))
    
    def fit(self, X, y):
        """Fit all models and compute weights based on marginal likelihood"""
        self.model_weights = []
        for i, model in enumerate(self.models):
            try:
                model.fit(X, y)
                # Weight by log marginal likelihood
                log_ml = model.log_marginal_likelihood()
                self.model_weights.append(np.exp(log_ml))
            except Exception as e:
                print(f"  Warning: Model {i} fitting failed: {e}")
                self.model_weights.append(0.0)
        
        # Normalize weights
        total_weight = sum(self.model_weights)
        if total_weight > 0:
            self.model_weights = [w / total_weight for w in self.model_weights]
        else:
            self.model_weights = [1.0 / len(self.models)] * len(self.models)
    
    def predict(self, X, return_std=True):
        """Weighted ensemble prediction"""
        X = np.atleast_2d(X)
        
        if return_std:
            predictions = []
            uncertainties = []
            weights = []
            
            for i, model in enumerate(self.models):
                if self.model_weights[i] > 0:
                    try:
                        mu, sigma = model.predict(X, return_std=True)
                        predictions.append(mu)
                        uncertainties.append(sigma)
                        weights.append(self.model_weights[i])
                    except:
                        continue
            
            if len(predictions) == 0:
                return np.zeros(len(X)), np.ones(len(X))
            
            # Weighted mean and maximum uncertainty
            weights = np.array(weights)
            mean = np.average(predictions, axis=0, weights=weights)
            std = np.max(uncertainties, axis=0)
            
            return mean, std
        else:
            predictions = []
            weights = []
            for i, model in enumerate(self.models):
                if self.model_weights[i] > 0:
                    try:
                        predictions.append(model.predict(X))
                        weights.append(self.model_weights[i])
                    except:
                        continue
            
            return np.average(predictions, axis=0, weights=weights) if predictions else np.zeros(len(X))
    
    def get_parameter_importance(self):
        """Extract parameter importance from length scales"""
        importance_scores = []
        
        for i, model in enumerate(self.models):
            if self.model_weights[i] > 0:
                try:
                    kernel = model.kernel_
                    # Try to extract length scales from different kernel structures
                    length_scales = None
                    
                    # For composite kernels (ConstantKernel * Matern/RBF + WhiteKernel)
                    if hasattr(kernel, 'k1') and hasattr(kernel.k1, 'k2'):
                        length_scales = kernel.k1.k2.length_scale
                    elif hasattr(kernel, 'k2'):
                        length_scales = kernel.k2.length_scale
                    elif hasattr(kernel, 'length_scale'):
                        length_scales = kernel.length_scale
                    
                    if length_scales is not None and hasattr(length_scales, '__len__'):
                        # Inverse of length scale = importance (smaller length scale = more sensitive)
                        # Normalize by median to get relative importance
                        ls_array = np.array(length_scales)
                        importance = 1.0 / (ls_array + 1e-10)
                        # Normalize so median = 1.0
                        importance = importance / (np.median(importance) + 1e-10)
                        importance_scores.append(importance)
                except Exception as e:
                    continue
        
        if importance_scores and len(importance_scores) > 0:
            # Weighted average importance
            weights = [w for w in self.model_weights if w > 0][:len(importance_scores)]
            weighted_importance = np.average(importance_scores, axis=0, weights=weights)
            return weighted_importance
        else:
            # Fallback: return ones (equal importance)
            return np.ones(N_PARAMS)

# Trust region with reset capability
class TrustRegion:
    def __init__(self):
        self.trust_radius, self.best_center = 0.5, None
        self.success_count, self.fail_count = 0, 0
        self.min_radius, self.max_radius = 0.05, 1.0
        self.radius_history = []
    
    def get_trust_region_bounds(self):
        if self.best_center is None:
            return [(0, 1)] * N_PARAMS
        bounds = []
        for i in range(N_PARAMS):
            center, half_width = self.best_center[i], self.trust_radius / 2
            bounds.append((max(0.0, center - half_width), min(1.0, center + half_width)))
        return bounds
    
    def update(self, new_best_found, new_center=None):
        if new_best_found:
            self.success_count += 1
            self.fail_count = 0
            if new_center is not None:
                self.best_center = new_center
            if self.success_count >= 3:
                self.trust_radius = min(self.max_radius, self.trust_radius * 1.5)
                self.success_count = 0
                print(f"  → Trust region expanded to {self.trust_radius:.2f}")
        else:
            self.fail_count += 1
            self.success_count = 0
            if self.fail_count >= 3:
                self.trust_radius = max(self.min_radius, self.trust_radius * 0.5)
                self.fail_count = 0
                print(f"  → Trust region shrunk to {self.trust_radius:.2f}")
        
        self.radius_history.append(self.trust_radius)
    
    def reset_for_exploration(self):
        self.trust_radius = 0.8
        self.success_count, self.fail_count = 0, 0
        self.radius_history.append(self.trust_radius)
        print(f"  → Trust region RESET to {self.trust_radius:.2f}")

# NEW: Thompson Sampling for exploration
def thompson_sampling(gp, bounds, n_samples=1):
    """Sample from GP posterior for exploration"""
    samples = []
    for _ in range(n_samples):
        # Sample a function from GP posterior
        X_grid = np.random.uniform([b[0] for b in bounds], [b[1] for b in bounds], size=(500, N_PARAMS))
        mu, sigma = gp.predict(X_grid, return_std=True)
        
        # Sample from posterior at each point
        posterior_samples = np.random.normal(mu, sigma)
        
        # Find minimum of sampled function
        best_idx = np.argmin(posterior_samples)
        samples.append(X_grid[best_idx])
    
    return np.array(samples)

# Hybrid acquisition with LOCAL PENALIZATION
def hybrid_acquisition_with_penalization(X, gp, best_y, X_samples, xi=0.01, kappa=2.0, weight_ei=0.6, penalization_weight=0.3):
    """Hybrid EI+UCB with local penalization"""
    X = np.atleast_2d(X)
    mu, sigma = gp.predict(X, return_std=True)
    
    # Expected Improvement
    with np.errstate(divide='warn', invalid='warn'):
        imp = best_y - mu - xi
        Z = imp / (sigma + 1e-9)
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0
    
    # Upper Confidence Bound
    ucb = -(mu - kappa * sigma)
    
    # Normalize
    ei_norm = (ei - ei.min()) / (ei.max() - ei.min() + 1e-9)
    ucb_norm = (ucb - ucb.min()) / (ucb.max() - ucb.min() + 1e-9)
    
    # Base acquisition
    acq = weight_ei * ei_norm + (1 - weight_ei) * ucb_norm
    
    # Local penalization
    if len(X_samples) > 0 and penalization_weight > 0:
        X_samples_warped = np.array([warp_parameters(x) for x in X_samples])
        min_distances = np.min([np.linalg.norm(X - x_sample, axis=1) for x_sample in X_samples_warped], axis=0)
        penalty = np.exp(-10 * min_distances)
        acq = acq * (1 - penalization_weight * penalty)
    
    return acq

# Acquisition optimizer with multi-start
def optimize_acquisition_multistart(acquisition_fn, bounds, n_starts=20, n_random=500):
    """Multi-start optimization of acquisition function"""
    best_acq, best_x = -np.inf, None
    
    # Random sampling
    random_samples = np.random.uniform([b[0] for b in bounds], [b[1] for b in bounds], size=(n_random, N_PARAMS))
    acq_random = acquisition_fn(random_samples)
    best_random_idx = np.argmax(acq_random)
    if acq_random[best_random_idx] > best_acq:
        best_acq, best_x = acq_random[best_random_idx], random_samples[best_random_idx]
    
    # Gradient-based optimization
    for _ in range(n_starts):
        x0 = np.array([np.random.uniform(b[0], b[1]) for b in bounds])
        result = minimize(lambda x: -acquisition_fn(x.reshape(1, -1))[0], x0, method='L-BFGS-B', bounds=bounds)
        if result.success and -result.fun > best_acq:
            best_acq, best_x = -result.fun, result.x
    
    return best_x

# NEW: Dynamic 2-level multi-fidelity with parameter-based thresholds
def get_adaptive_sim_days(iteration, base_days=180):
    """
    Dynamic 2-level fidelity strategy based on number of parameters:
    - Initial phase: 4 * N_PARAMS iterations at 30-day fidelity
    - Fast phase: 2 * N_PARAMS iterations at 30-day fidelity  
    - Full phase: 2 * N_PARAMS iterations at 180-day fidelity
    
    Returns: (sim_days, description, phase_name)
    """
    n_initial = 4 * N_PARAMS  # 24 for 6 parameters
    n_fast = 2 * N_PARAMS      # 12 for 6 parameters
    
    fast_phase_end = n_initial + n_fast
    
    if iteration < fast_phase_end:
        return 30, "FAST (30d)", "fast"  # Initial + Fast exploration
    else:
        return base_days, "FULL (180d)", "full"  # Full precision

# Simulation runner
def run_lowres_with_params(params_array, config_base, highres_results, sim_days=180, iteration=0):
    from main_comparison import run_simulation
    
    # Adaptive fidelity
    adaptive_days, fidelity_desc, phase_name = get_adaptive_sim_days(iteration, sim_days)
    
    config = config_base.copy()
    config['subgrid_params'] = {PARAM_NAMES[i]: float(params_array[i]) for i in range(N_PARAMS)}
    
    print(f"\n{'='*70}")
    print(f"Testing parameters - Fidelity: {Colors.cyan(fidelity_desc)}")
    print(f"{'='*70}")
    for param_name, val in config['subgrid_params'].items():
        print(f"  {param_name}: {val:.6e}")
    
    try:
        results = run_simulation(config, sim_days=adaptive_days, save_interval_hours=12)
        # Adaptive loss computation will automatically use appropriate time window
        loss, detailed = compute_loss(results, highres_results, return_fields=True, adaptive_window=True)
        if not np.isfinite(loss):
            print(Colors.yellow(f"  ⚠ Loss not finite: {loss}"))
            return np.nan, None, None
        print(f"  Loss: {Colors.green(f'{loss:.6f}')}")
        return loss, results, detailed
    except Exception as e:
        print(Colors.yellow(f"  ⚠ Simulation failed: {e}"))
        return np.nan, None, None

# Loss computation with adaptive time window
def compute_loss(lowres_results, highres_results, n_days_avg=30, return_fields=False, adaptive_window=True):
    """
    Compute loss with adaptive time window based on simulation length
    
    Args:
        adaptive_window: If True, adjust time window based on lowres simulation length
                        - For 30-day runs: use entire simulation (days 0-30)
                        - For 180-day runs: use last 30 days (days 150-180, equilibrated)
    """
    nx_hr, ny_hr = highres_results['config']['nx'], highres_results['config']['ny']
    nx_lr, ny_lr = lowres_results['config']['nx'], lowres_results['config']['ny']
    coarsen_factor_x, coarsen_factor_y = nx_hr // nx_lr, ny_hr // ny_lr
    
    times_hr, times_lr = highres_results['times'], lowres_results['times']
    
    # Adaptive time window based on simulation length
    if adaptive_window:
        lr_duration = times_lr[-1] - times_lr[0]
        
        if lr_duration <= 40:  # 30-day runs
            # Use entire simulation
            time_start, time_end = times_lr[0], times_lr[-1]
            print(f"  → Using entire simulation (days 0-{time_end - time_start:.0f}) for loss")
        else:  # Full 180-day runs
            # Use last 30 days for equilibrated state
            time_start, time_end = times_lr[-1] - n_days_avg, times_lr[-1]
            print(f"  → Using last {n_days_avg} days for loss (equilibrated state)")
        
        # Get matching time window from high-res
        if lr_duration <= 40:
            # For short runs, match the same absolute time window
            indices_hr = np.where((times_hr >= time_start) & (times_hr <= time_end))[0]
        else:
            # For full runs, use last 30 days of high-res too
            indices_hr = np.where(times_hr >= times_hr[-1] - n_days_avg)[0]
        
        indices_lr = np.where((times_lr >= time_start) & (times_lr <= time_end))[0]
    else:
        # Original behavior: use last n_days_avg
        indices_hr = np.where(times_hr >= times_hr[-1] - n_days_avg)[0]
        indices_lr = np.where(times_lr >= times_lr[-1] - n_days_avg)[0]
    
    q1_hr_avg = np.mean([highres_results['q1_history'][i] for i in indices_hr], axis=0)
    q2_hr_avg = np.mean([highres_results['q2_history'][i] for i in indices_hr], axis=0)
    q1_lr_avg = np.mean([lowres_results['q1_history'][i] for i in indices_lr], axis=0)
    q2_lr_avg = np.mean([lowres_results['q2_history'][i] for i in indices_lr], axis=0)
    
    model_hr, model_lr = highres_results['model'], lowres_results['model']
    psi1_hr_avg, psi2_hr_avg = model_hr.q_to_psi(q1_hr_avg, q2_hr_avg)
    psi1_lr_avg, psi2_lr_avg = model_lr.q_to_psi(q1_lr_avg, q2_lr_avg)
    
    H1, H2, H_total = model_hr.H1, model_hr.H2, model_hr.H1 + model_hr.H2
    q_bt_hr = (H1 * q1_hr_avg + H2 * q2_hr_avg) / H_total
    psi_bt_hr = (H1 * psi1_hr_avg + H2 * psi2_hr_avg) / H_total
    q_bt_lr = (H1 * q1_lr_avg + H2 * q2_lr_avg) / H_total
    psi_bt_lr = (H1 * psi1_lr_avg + H2 * psi2_lr_avg) / H_total
    
    def coarsen(field, fx, fy):
        return uniform_filter(field, size=(fy, fx), mode='wrap')[::fy, ::fx]
    
    q_bt_hr_coarse = coarsen(q_bt_hr, coarsen_factor_x, coarsen_factor_y)
    psi_bt_hr_coarse = coarsen(psi_bt_hr, coarsen_factor_x, coarsen_factor_y)
    
    nrmse = lambda pred, target: np.sqrt(np.mean((pred - target)**2)) / (np.std(target) + 1e-20)
    loss_q_bt, loss_psi_bt = nrmse(q_bt_lr, q_bt_hr_coarse), nrmse(psi_bt_lr, psi_bt_hr_coarse)
    weight_pv, weight_psi = 0.6, 0.4
    total_loss = weight_pv * loss_q_bt + weight_psi * loss_psi_bt
    
    if return_fields:
        return total_loss, {'q_bt_hr_coarse': q_bt_hr_coarse, 'psi_bt_hr_coarse': psi_bt_hr_coarse,
                           'q_bt_lr': q_bt_lr, 'psi_bt_lr': psi_bt_lr, 'loss_q_bt': loss_q_bt,
                           'loss_psi_bt': loss_psi_bt, 'total_loss': total_loss}
    return total_loss

# ============================================================================
# VISUALIZATION SUITE
# ============================================================================

class OptimizationVisualizer:
    """Comprehensive visualization of optimization progress"""
    
    def __init__(self, optimizer):
        self.optimizer = optimizer
        sns.set_style("whitegrid")
        plt.rcParams['figure.dpi'] = 100
        plt.rcParams['savefig.dpi'] = 300
    
    def plot_comprehensive_analysis(self, save_path='optimization_analysis.png'):
        """Create comprehensive multi-panel analysis"""
        fig = plt.figure(figsize=(20, 12))
        gs = gridspec.GridSpec(3, 3, figure=fig, hspace=0.3, wspace=0.3)
        
        # 1. Loss evolution
        ax1 = fig.add_subplot(gs[0, :2])
        self._plot_loss_evolution(ax1)
        
        # 2. Parameter evolution
        ax2 = fig.add_subplot(gs[1, :2])
        self._plot_parameter_evolution(ax2)
        
        # 3. Parameter importance
        ax3 = fig.add_subplot(gs[2, :2])
        self._plot_parameter_importance(ax3)
        
        # 4. Trust region evolution
        ax4 = fig.add_subplot(gs[0, 2])
        self._plot_trust_region(ax4)
        
        # 5. Convergence diagnostics
        ax5 = fig.add_subplot(gs[1, 2])
        self._plot_convergence_diagnostics(ax5)
        
        # 6. Best parameters bar chart
        ax6 = fig.add_subplot(gs[2, 2])
        self._plot_best_parameters(ax6)
        
        plt.suptitle('Bayesian Optimization - Comprehensive Analysis', 
                     fontsize=16, fontweight='bold', y=0.995)
        
        plt.savefig(save_path, bbox_inches='tight', dpi=300)
        print(f"\n✓ Saved comprehensive analysis: {save_path}")
        plt.close()
    
    def _plot_loss_evolution(self, ax):
        """Plot loss vs iterations with best loss tracking and fidelity phases"""
        y_samples = np.array(self.optimizer.y_samples)
        valid_mask = np.isfinite(y_samples)
        
        iterations = np.arange(len(y_samples))
        
        # Plot all losses
        ax.scatter(iterations[valid_mask], y_samples[valid_mask], 
                  alpha=0.6, s=50, c='steelblue', label='Valid evaluations', zorder=3)
        ax.scatter(iterations[~valid_mask], np.ones(np.sum(~valid_mask)) * np.nanmax(y_samples) * 1.1, 
                  alpha=0.4, s=30, c='red', marker='x', label='Failed evaluations', zorder=2)
        
        # Plot best loss trajectory - SEPARATE FOR EACH FIDELITY
        best_trajectory_30d = []
        best_trajectory_180d = []
        current_best_30d = np.inf
        current_best_180d = np.inf
        
        n_fast_phase_end = (4 * N_PARAMS) + (2 * N_PARAMS)
        
        for i, loss in enumerate(y_samples):
            _, fidelity_desc, _ = get_adaptive_sim_days(i)
            
            if i < n_fast_phase_end:  # 30-day fidelity
                if np.isfinite(loss) and loss < current_best_30d:
                    current_best_30d = loss
                best_trajectory_30d.append(current_best_30d if current_best_30d != np.inf else np.nan)
                best_trajectory_180d.append(np.nan)
            else:  # 180-day fidelity
                if np.isfinite(loss) and loss < current_best_180d:
                    current_best_180d = loss
                best_trajectory_30d.append(np.nan)
                best_trajectory_180d.append(current_best_180d if current_best_180d != np.inf else np.nan)
        
        # Plot 30-day best loss
        valid_30d = [(i, best_trajectory_30d[i]) for i in range(len(best_trajectory_30d)) if np.isfinite(best_trajectory_30d[i])]
        if valid_30d:
            indices_30d, values_30d = zip(*valid_30d)
            ax.plot(indices_30d, values_30d, 'limegreen', linewidth=2.5, 
                   label='Best loss (30d)', zorder=4, marker='o', markersize=5)
        
        # Plot 180-day best loss
        valid_180d = [(i, best_trajectory_180d[i]) for i in range(len(best_trajectory_180d)) if np.isfinite(best_trajectory_180d[i])]
        if valid_180d:
            indices_180d, values_180d = zip(*valid_180d)
            ax.plot(indices_180d, values_180d, 'darkgreen', linewidth=3, 
                   label='Best loss (180d)', zorder=4, marker='*', markersize=8)
        
        # Highlight different fidelity phases with colored backgrounds
        n_initial = self.optimizer.n_initial_samples
        n_fast_phase_end = (4 * N_PARAMS) + (2 * N_PARAMS)  # Initial + Fast phase
        
        ax.axvspan(0, n_initial-1, alpha=0.1, color='orange', label='Initial sampling')
        
        if len(iterations) > n_initial:
            # 30-day runs (fast exploration)
            ax.axvspan(n_initial, min(n_fast_phase_end, len(iterations)-1), 
                      alpha=0.10, color='lightblue', label='30d runs (fast)')
        
        if len(iterations) > n_fast_phase_end:
            # 180-day runs (full precision)
            ax.axvspan(n_fast_phase_end, len(iterations)-1, 
                      alpha=0.10, color='lightcoral', label='180d runs (full)')
        
        # Mark fidelity transition with vertical line
        if len(iterations) > n_fast_phase_end:
            ax.axvline(n_fast_phase_end, color='red', linestyle='--', linewidth=2, 
                      alpha=0.7, label='Fidelity jump')
        
        # Mark best iteration
        ax.scatter([self.optimizer.best_iteration], [self.optimizer.best_loss],
                  s=200, c='gold', marker='*', edgecolors='red', linewidth=2,
                  label=f'Best (iter {self.optimizer.best_iteration+1})', zorder=5)
        
        # If best was found in 30d phase, add annotation
        n_fast_phase_end = (4 * N_PARAMS) + (2 * N_PARAMS)
        if self.optimizer.best_params_original_iteration is not None and self.optimizer.best_params_original_iteration < n_fast_phase_end:
            # Check if we have 180d baseline (meaning re-evaluation happened)
            has_full_baseline = 'FULL (180d)' in self.optimizer.baseline_loss_by_fidelity
            if has_full_baseline:
                annotation_text = f'Found at iter {self.optimizer.best_params_original_iteration+1}\n(30d phase)\nRe-eval at 180d'
            else:
                annotation_text = f'Found at iter {self.optimizer.best_params_original_iteration+1}\n(30d phase)'
            
            ax.annotate(annotation_text, 
                       xy=(self.optimizer.best_params_original_iteration, self.optimizer.best_loss),
                       xytext=(self.optimizer.best_params_original_iteration + 10, self.optimizer.best_loss * 1.2),
                       arrowprops=dict(arrowstyle='->', color='red', lw=1.5),
                       fontsize=7, color='red', fontweight='bold',
                       bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7))
        
        ax.set_xlabel('Iteration', fontsize=12, fontweight='bold')
        ax.set_ylabel('Loss', fontsize=12, fontweight='bold')
        ax.set_title(f'Loss Evolution (Dynamic 2-Level Multi-Fidelity: 30d → 180d)\n' +
                    f'Initial: {4*N_PARAMS} samples, Fast: {2*N_PARAMS} iters, Full: {2*N_PARAMS}+ iters\n' +
                    'Separate tracking per fidelity', 
                    fontsize=11, fontweight='bold')
        ax.legend(loc='best', fontsize=7, ncol=2)
        ax.grid(True, alpha=0.3)
        
        # Add improvement info with all baselines
        if self.optimizer.baseline_loss_by_fidelity:
            info_lines = []
            
            # Final improvement (use FULL baseline if available)
            final_baseline = self.optimizer.baseline_loss_by_fidelity.get('FULL (180d)')
            if final_baseline:
                improvement = (final_baseline - self.optimizer.best_loss) / final_baseline * 100
                info_lines.append(f'Final improvement: {improvement:+.1f}%')
            
            # Show all baselines
            info_lines.append('Baselines:')
            for fid, base in sorted(self.optimizer.baseline_loss_by_fidelity.items()):
                info_lines.append(f'  {fid}: {base:.4f}')
            
            info_text = '\n'.join(info_lines)
            ax.text(0.02, 0.98, info_text, 
                   transform=ax.transAxes, fontsize=8, verticalalignment='top',
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.7))
    
    def _plot_parameter_evolution(self, ax):
        """Plot how parameters evolved over iterations"""
        X_samples = np.array(self.optimizer.X_samples)
        n_iters = len(X_samples)
        
        # Normalize parameters to [0, 1] for visualization
        X_normalized = np.array([warp_parameters(x) for x in X_samples])
        
        for i, param_name in enumerate(PARAM_NAMES):
            ax.plot(range(n_iters), X_normalized[:, i], 
                   marker='o', markersize=4, alpha=0.7, linewidth=1.5, 
                   label=param_name)
        
        # Highlight initial samples phase
        n_initial = self.optimizer.n_initial_samples
        ax.axvspan(0, n_initial-1, alpha=0.1, color='orange')
        
        # Mark best iteration
        ax.axvline(self.optimizer.best_iteration, color='red', linestyle='--', 
                  linewidth=2, alpha=0.7, label='Best found')
        
        ax.set_xlabel('Iteration', fontsize=12, fontweight='bold')
        ax.set_ylabel('Normalized Parameter Value', fontsize=12, fontweight='bold')
        ax.set_title('Parameter Evolution', fontsize=13, fontweight='bold')
        ax.legend(loc='best', fontsize=8, ncol=2)
        ax.grid(True, alpha=0.3)
        ax.set_ylim(-0.05, 1.05)
    
    def _plot_parameter_importance(self, ax):
        """Plot parameter sensitivity over time"""
        if not hasattr(self.optimizer, 'importance_history') or not self.optimizer.importance_history:
            ax.text(0.5, 0.5, 'Parameter importance\nnot tracked\n(Need more iterations)', 
                   ha='center', va='center', transform=ax.transAxes, fontsize=12)
            ax.set_title('Parameter Importance Over Time', fontsize=13, fontweight='bold')
            return
        
        importance_array = np.array(self.optimizer.importance_history)
        
        # Check if all values are constant (indicating a problem)
        if importance_array.shape[0] < 2 or np.allclose(importance_array[0], importance_array[-1]):
            # Fall back to correlation-based importance from current samples
            ax.text(0.5, 0.7, 'GP-based importance unavailable', 
                   ha='center', va='center', transform=ax.transAxes, fontsize=11, style='italic')
            ax.text(0.5, 0.5, 'Using correlation-based\nimportance instead', 
                   ha='center', va='center', transform=ax.transAxes, fontsize=10)
            ax.text(0.5, 0.3, '(See parameter_sensitivity.png\nfor detailed analysis)', 
                   ha='center', va='center', transform=ax.transAxes, fontsize=9, style='italic')
            ax.set_title('Parameter Importance Over Time', fontsize=13, fontweight='bold')
            return
        
        # Plot importance evolution
        iterations = np.arange(len(importance_array)) + self.optimizer.n_initial_samples
        
        for i, param_name in enumerate(PARAM_NAMES):
            ax.plot(iterations, importance_array[:, i], 
                   marker='o', markersize=3, alpha=0.7, linewidth=1.5,
                   label=param_name)
        
        ax.set_xlabel('Iteration', fontsize=12, fontweight='bold')
        ax.set_ylabel('Importance Score (1/length_scale)', fontsize=12, fontweight='bold')
        ax.set_title('Parameter Importance Over Time\n(Higher = More Sensitive)', fontsize=13, fontweight='bold')
        ax.legend(loc='best', fontsize=8, ncol=2)
        ax.grid(True, alpha=0.3)
        
        # Add interpretation note
        ax.text(0.98, 0.02, 'Note: Based on GP length scales\nSmaller length scale → Higher importance', 
               transform=ax.transAxes, fontsize=7, ha='right', va='bottom',
               bbox=dict(boxstyle='round,pad=0.3', facecolor='wheat', alpha=0.5))
    
    def _plot_trust_region(self, ax):
        """Plot trust region radius evolution"""
        if not self.optimizer.trust_region.radius_history:
            ax.text(0.5, 0.5, 'Trust region\nhistory empty', 
                   ha='center', va='center', transform=ax.transAxes, fontsize=10)
            ax.set_title('Trust Region Evolution', fontsize=11, fontweight='bold')
            return
        
        radius_history = self.optimizer.trust_region.radius_history
        ax.plot(radius_history, marker='o', markersize=4, linewidth=2, color='purple')
        ax.axhline(self.optimizer.trust_region.min_radius, color='red', 
                  linestyle='--', alpha=0.5, label='Min radius')
        ax.axhline(self.optimizer.trust_region.max_radius, color='green', 
                  linestyle='--', alpha=0.5, label='Max radius')
        
        ax.set_xlabel('Update Step', fontsize=10, fontweight='bold')
        ax.set_ylabel('Trust Radius', fontsize=10, fontweight='bold')
        ax.set_title('Trust Region Evolution', fontsize=11, fontweight='bold')
        ax.legend(fontsize=8)
        ax.grid(True, alpha=0.3)
    
    def _plot_convergence_diagnostics(self, ax):
        """Plot convergence metrics"""
        y_samples = np.array(self.optimizer.y_samples)
        valid_mask = np.isfinite(y_samples)
        
        # Moving average of improvement
        window = 5
        improvements = []
        for i in range(window, len(y_samples)):
            if valid_mask[i]:
                recent_best = np.nanmin(y_samples[max(0, i-window):i])
                current = y_samples[i]
                improvements.append(max(0, recent_best - current))
            else:
                improvements.append(0)
        
        iterations = np.arange(window, len(y_samples))
        ax.bar(iterations, improvements, alpha=0.6, color='teal')
        ax.set_xlabel('Iteration', fontsize=10, fontweight='bold')
        ax.set_ylabel('Recent Improvement', fontsize=10, fontweight='bold')
        ax.set_title('Convergence Diagnostics', fontsize=11, fontweight='bold')
        ax.grid(True, alpha=0.3, axis='y')
    
    def _plot_best_parameters(self, ax):
        """Bar chart of best parameters"""
        if self.optimizer.best_params is None:
            ax.text(0.5, 0.5, 'No best\nparameters yet', 
                   ha='center', va='center', transform=ax.transAxes, fontsize=10)
            ax.set_title('Best Parameters', fontsize=11, fontweight='bold')
            return
        
        # Normalize to [0, 1]
        best_normalized = warp_parameters(self.optimizer.best_params)
        
        colors = plt.cm.viridis(best_normalized)
        bars = ax.barh(PARAM_NAMES, best_normalized, color=colors, alpha=0.7)
        
        ax.set_xlabel('Normalized Value', fontsize=10, fontweight='bold')
        ax.set_title('Best Parameters (Normalized)', fontsize=11, fontweight='bold')
        ax.set_xlim(0, 1)
        ax.grid(True, alpha=0.3, axis='x')
        
        # Add actual values as text
        for i, (bar, name) in enumerate(zip(bars, PARAM_NAMES)):
            actual_val = self.optimizer.best_params[i]
            ax.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height()/2, 
                   f'{actual_val:.2e}', va='center', fontsize=7)
    
    def plot_parameter_sensitivity_heatmap(self, save_path='parameter_sensitivity.png'):
        """Create comprehensive parameter sensitivity analysis"""
        X_samples = np.array(self.optimizer.X_samples)
        y_samples = np.array(self.optimizer.y_samples)
        valid_mask = np.isfinite(y_samples)
        
        if np.sum(valid_mask) < 5:
            print("Not enough valid samples for sensitivity analysis")
            return
        
        X_valid = X_samples[valid_mask]
        y_valid = y_samples[valid_mask]
        
        # Normalize parameters
        X_normalized = np.array([warp_parameters(x) for x in X_valid])
        
        # Create comprehensive figure
        fig = plt.figure(figsize=(20, 12))
        gs = gridspec.GridSpec(3, 3, figure=fig, hspace=0.35, wspace=0.35)
        
        # ========== Panel 1: Parameter-Loss Correlations ==========
        ax1 = fig.add_subplot(gs[0, :2])
        correlations = []
        for i in range(N_PARAMS):
            corr = np.corrcoef(X_normalized[:, i], y_valid)[0, 1]
            correlations.append(corr)
        
        colors = ['crimson' if c > 0 else 'forestgreen' for c in correlations]
        bars = ax1.barh(PARAM_NAMES, correlations, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
        ax1.axvline(0, color='black', linewidth=2)
        ax1.set_xlabel('Correlation with Loss', fontsize=13, fontweight='bold')
        ax1.set_title('Parameter Sensitivity: Correlation with Loss\n' + 
                     'RED = Increasing parameter WORSENS performance | GREEN = Increasing parameter IMPROVES performance',
                     fontsize=12, fontweight='bold')
        ax1.grid(True, alpha=0.3, axis='x')
        
        for bar, corr in zip(bars, correlations):
            width = bar.get_width()
            label = f'{corr:+.3f}'
            ax1.text(width + (0.02 if width > 0 else -0.02), 
                    bar.get_y() + bar.get_height()/2, label,
                    va='center', ha='left' if width > 0 else 'right',
                    fontsize=10, fontweight='bold')
        
        # ========== Panel 2: Variance Explained ==========
        ax2 = fig.add_subplot(gs[0, 2])
        
        # Simple variance explained: R² from linear fit
        from sklearn.linear_model import LinearRegression
        var_explained = []
        for i in range(N_PARAMS):
            X_param = X_normalized[:, i].reshape(-1, 1)
            model = LinearRegression()
            model.fit(X_param, y_valid)
            r2 = model.score(X_param, y_valid)
            var_explained.append(max(0, r2))  # Clip negative R²
        
        colors_var = plt.cm.RdYlGn_r(np.array(var_explained) / max(var_explained))
        ax2.barh(PARAM_NAMES, var_explained, color=colors_var, alpha=0.8, edgecolor='black')
        ax2.set_xlabel('Variance Explained (R²)', fontsize=11, fontweight='bold')
        ax2.set_title('Parameter Importance\n(Higher = More Influential)', fontsize=11, fontweight='bold')
        ax2.grid(True, alpha=0.3, axis='x')
        
        for i, (val, name) in enumerate(zip(var_explained, PARAM_NAMES)):
            ax2.text(val + 0.01, i, f'{val:.3f}', va='center', fontsize=9, fontweight='bold')
        
        # ========== Panel 3: Parameter Ranges Explored ==========
        ax3 = fig.add_subplot(gs[1, :2])
        
        # Box plots showing explored ranges
        positions = np.arange(N_PARAMS)
        bp = ax3.boxplot([X_normalized[:, i] for i in range(N_PARAMS)],
                         positions=positions, vert=False, patch_artist=True,
                         widths=0.6, showfliers=True)
        
        for patch, corr in zip(bp['boxes'], correlations):
            color = 'lightcoral' if corr > 0 else 'lightgreen'
            patch.set_facecolor(color)
            patch.set_alpha(0.6)
        
        ax3.set_yticks(positions)
        ax3.set_yticklabels(PARAM_NAMES)
        ax3.set_xlabel('Normalized Parameter Value [0=min, 1=max]', fontsize=12, fontweight='bold')
        ax3.set_title('Parameter Space Exploration\n(Box = 25th-75th percentile, Whiskers = min-max, Dots = outliers)',
                     fontsize=11, fontweight='bold')
        ax3.grid(True, alpha=0.3, axis='x')
        ax3.set_xlim(-0.05, 1.05)
        
        # Mark best parameters
        if self.optimizer.best_params is not None:
            best_normalized = warp_parameters(self.optimizer.best_params)
            ax3.scatter(best_normalized, positions, s=200, c='gold', marker='*', 
                       edgecolors='red', linewidth=2, zorder=10, label='Best Found')
            ax3.legend(fontsize=10, loc='upper right')
        
        # ========== Panel 4: Loss vs Top 2 Parameters (Scatter) ==========
        abs_corr = np.abs(correlations)
        top_2_indices = np.argsort(abs_corr)[-2:]
        
        ax4 = fig.add_subplot(gs[1, 2])
        param_idx_1, param_idx_2 = top_2_indices[1], top_2_indices[0]
        
        scatter = ax4.scatter(X_normalized[:, param_idx_1], X_normalized[:, param_idx_2],
                            c=y_valid, cmap='viridis_r', s=80, alpha=0.6,
                            edgecolors='black', linewidth=0.5)
        
        # Mark best point
        if self.optimizer.best_params is not None:
            best_norm = warp_parameters(self.optimizer.best_params)
            ax4.scatter(best_norm[param_idx_1], best_norm[param_idx_2],
                       s=300, c='gold', marker='*', edgecolors='red', linewidth=2.5,
                       zorder=10, label='Best')
        
        ax4.set_xlabel(f'{PARAM_NAMES[param_idx_1]}', fontsize=11, fontweight='bold')
        ax4.set_ylabel(f'{PARAM_NAMES[param_idx_2]}', fontsize=11, fontweight='bold')
        ax4.set_title(f'Top 2 Most Influential Parameters\n(Lower loss = Better)', 
                     fontsize=11, fontweight='bold')
        ax4.grid(True, alpha=0.3)
        ax4.legend(fontsize=9)
        
        cbar = plt.colorbar(scatter, ax=ax4)
        cbar.set_label('Loss', fontsize=10, fontweight='bold')
        
        # ========== Panel 5: Parameter Value Distributions ==========
        ax5 = fig.add_subplot(gs[2, :2])
        
        # Show distribution of sampled values for top 3 parameters
        top_3_indices = np.argsort(abs_corr)[-3:]
        colors_dist = ['red', 'orange', 'green']
        
        for idx_rank, param_idx in enumerate(top_3_indices[::-1]):
            values = X_normalized[:, param_idx]
            ax5.hist(values, bins=15, alpha=0.5, color=colors_dist[idx_rank], 
                    label=PARAM_NAMES[param_idx], edgecolor='black', linewidth=1)
        
        ax5.set_xlabel('Normalized Parameter Value', fontsize=12, fontweight='bold')
        ax5.set_ylabel('Frequency', fontsize=12, fontweight='bold')
        ax5.set_title('Sampling Distribution of Top 3 Most Influential Parameters',
                     fontsize=11, fontweight='bold')
        ax5.legend(fontsize=10)
        ax5.grid(True, alpha=0.3, axis='y')
        
        # ========== Panel 6: Parameter Importance Summary ==========
        ax6 = fig.add_subplot(gs[2, 2])
        ax6.axis('off')
        
        # Create summary text
        summary_lines = [
            "INTERPRETATION GUIDE:",
            "",
            "Correlation:",
            "  • Positive = increasing parameter worsens loss",
            "  • Negative = increasing parameter improves loss",
            "  • Magnitude = strength of relationship",
            "",
            "Variance Explained (R²):",
            "  • How much loss variation this parameter explains",
            "  • Higher = more important to tune carefully",
            "",
            "Top 3 Most Important Parameters:",
        ]
        
        top_3_with_corr = [(PARAM_NAMES[i], correlations[i], var_explained[i]) 
                           for i in np.argsort(abs_corr)[-3:][::-1]]
        
        for rank, (name, corr, var_exp) in enumerate(top_3_with_corr, 1):
            direction = "↑ worsens" if corr > 0 else "↓ improves"
            summary_lines.append(f"  {rank}. {name}")
            summary_lines.append(f"     Corr: {corr:+.3f} ({direction})")
            summary_lines.append(f"     R²: {var_exp:.3f}")
        
        summary_text = '\n'.join(summary_lines)
        ax6.text(0.05, 0.95, summary_text, transform=ax6.transAxes,
                fontsize=10, verticalalignment='top', family='monospace',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
        
        plt.suptitle('Comprehensive Parameter Sensitivity Analysis', 
                    fontsize=16, fontweight='bold', y=0.995)
        
        plt.savefig(save_path, bbox_inches='tight', dpi=300)
        print(f"✓ Saved sensitivity analysis: {save_path}")
        plt.close()
    
    def plot_computational_efficiency(self, save_path='computational_efficiency.png'):
        """Analyze computational cost vs improvement"""
        y_samples = np.array(self.optimizer.y_samples)
        valid_mask = np.isfinite(y_samples)
        
        if np.sum(valid_mask) < 5:
            print("Not enough valid samples for efficiency analysis")
            return
        
        # Estimate computational cost (30-day = 1 unit, 180-day = 6 units)
        cumulative_cost = []
        cost_per_iteration = []
        total_cost = 0
        
        for i in range(len(y_samples)):
            sim_days, _, _ = get_adaptive_sim_days(i)
            cost = sim_days / 30.0  # Normalize to 30-day cost
            cost_per_iteration.append(cost)
            total_cost += cost
            cumulative_cost.append(total_cost)
        
        # Create figure
        fig = plt.figure(figsize=(18, 10))
        gs = gridspec.GridSpec(2, 3, figure=fig, hspace=0.3, wspace=0.3)
        
        # ========== Panel 1: Cumulative Cost vs Improvement ==========
        ax1 = fig.add_subplot(gs[0, :2])
        
        # Best loss trajectory - split by fidelity
        best_trajectory_30d = []
        best_trajectory_180d = []
        current_best_30d = np.inf
        current_best_180d = np.inf
        
        n_fast_phase_end = (4 * N_PARAMS) + (2 * N_PARAMS)
        
        for i, loss in enumerate(y_samples):
            _, fidelity_desc, _ = get_adaptive_sim_days(i)
            
            if i < n_fast_phase_end:  # 30-day fidelity
                if np.isfinite(loss) and loss < current_best_30d:
                    current_best_30d = loss
                best_trajectory_30d.append(current_best_30d if current_best_30d != np.inf else np.nan)
                best_trajectory_180d.append(np.nan)  # Not applicable yet
            else:  # 180-day fidelity
                if np.isfinite(loss) and loss < current_best_180d:
                    current_best_180d = loss
                best_trajectory_30d.append(np.nan)  # 30-day phase is over
                best_trajectory_180d.append(current_best_180d if current_best_180d != np.inf else np.nan)
        
        ax1_twin = ax1.twinx()
        
        # Plot cumulative cost
        color_cost = 'steelblue'
        line1 = ax1.plot(range(len(cumulative_cost)), cumulative_cost, 
                color=color_cost, linewidth=2.5, label='Cumulative Cost', marker='o', markersize=4)
        ax1.set_xlabel('Iteration', fontsize=12, fontweight='bold')
        ax1.set_ylabel('Cumulative Computational Cost (30-day equiv.)', 
                      fontsize=11, fontweight='bold', color=color_cost)
        ax1.tick_params(axis='y', labelcolor=color_cost)
        ax1.grid(True, alpha=0.3)
        
        # Plot best loss for 30-day fidelity
        color_loss_30d = 'limegreen'
        valid_indices_30d = [i for i, loss in enumerate(best_trajectory_30d) if np.isfinite(loss)]
        valid_trajectory_30d = [best_trajectory_30d[i] for i in valid_indices_30d]
        line2 = ax1_twin.plot(valid_indices_30d, valid_trajectory_30d, 
                     color=color_loss_30d, linewidth=2.5, label='Best Loss (30d)', 
                     marker='o', markersize=6, linestyle='-', alpha=0.8)
        
        # Plot best loss for 180-day fidelity
        color_loss_180d = 'darkgreen'
        valid_indices_180d = [i for i, loss in enumerate(best_trajectory_180d) if np.isfinite(loss)]
        valid_trajectory_180d = [best_trajectory_180d[i] for i in valid_indices_180d]
        line3 = ax1_twin.plot(valid_indices_180d, valid_trajectory_180d, 
                     color=color_loss_180d, linewidth=3, label='Best Loss (180d)', 
                     marker='*', markersize=8, linestyle='-')
        
        ax1_twin.set_ylabel('Best Loss', fontsize=11, fontweight='bold', color='darkgreen')
        ax1_twin.tick_params(axis='y', labelcolor='darkgreen')
        
        # Mark fidelity transition
        n_fast_phase_end = (4 * N_PARAMS) + (2 * N_PARAMS)
        line4 = ax1.axvline(n_fast_phase_end, color='red', linestyle='--', linewidth=2, alpha=0.7, label='Fidelity Jump')
        
        # Add annotation explaining the jump
        if len(valid_indices_30d) > 0 and len(valid_indices_180d) > 0:
            last_30d_loss = valid_trajectory_30d[-1]
            first_180d_loss = valid_trajectory_180d[0]
            if np.isfinite(last_30d_loss) and np.isfinite(first_180d_loss):
                jump_pct = (first_180d_loss - last_30d_loss) / last_30d_loss * 100
                ax1_twin.annotate(f'Re-eval: {jump_pct:+.1f}%',
                                 xy=(n_fast_phase_end, first_180d_loss), xytext=(n_fast_phase_end + 5, first_180d_loss * 1.1),
                                 arrowprops=dict(arrowstyle='->', color='red', lw=1.5),
                                 fontsize=8, color='red', fontweight='bold',
                                 bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7))
        
        ax1.set_title('Computational Efficiency: Cost vs Improvement Over Time\n(Separate best loss tracking for each fidelity)', 
                     fontsize=12, fontweight='bold')
        
        # Combine legends from both axes
        lines = line1 + line2 + line3 + [line4]
        labels = [l.get_label() for l in lines]
        ax1.legend(lines, labels, loc='upper left', fontsize=9)
        
        # ========== Panel 2: Improvement per Cost Unit ==========
        ax2 = fig.add_subplot(gs[0, 2])
        
        # Calculate improvement per cost for each iteration
        if self.optimizer.baseline_loss:
            improvements = []
            for i, loss in enumerate(y_samples):
                if np.isfinite(loss):
                    improvement = max(0, self.optimizer.baseline_loss - loss)
                    improvements.append(improvement / cost_per_iteration[i])
                else:
                    improvements.append(0)
            
            # Moving average
            window = 5
            smooth_improvements = []
            for i in range(len(improvements)):
                start = max(0, i - window + 1)
                smooth_improvements.append(np.mean(improvements[start:i+1]))
            
            ax2.plot(range(len(smooth_improvements)), smooth_improvements, 
                    color='purple', linewidth=2.5, label='Smoothed')
            ax2.scatter(range(len(improvements)), improvements, 
                       alpha=0.4, s=30, c='gray', label='Raw')
            
            ax2.set_xlabel('Iteration', fontsize=11, fontweight='bold')
            ax2.set_ylabel('Improvement per Cost Unit', fontsize=10, fontweight='bold')
            ax2.set_title('Sample Efficiency\n(Higher = Better)', fontsize=11, fontweight='bold')
            ax2.legend(fontsize=9)
            ax2.grid(True, alpha=0.3)
        
        # ========== Panel 3: Cost Breakdown by Phase ==========
        ax3 = fig.add_subplot(gs[1, 0])
        
        # Calculate costs by phase with dynamic thresholds
        n_init = self.optimizer.n_initial_samples
        n_fast_phase_end = (4 * N_PARAMS) + (2 * N_PARAMS)  # Initial + Fast phase
        
        phase_names = ['Initial\nSampling', 'Fast\nExploration\n(30d)', 'Full\nPrecision\n(180d)']
        phase_costs = [0, 0, 0]
        phase_iters = [0, 0, 0]
        
        for i, cost in enumerate(cost_per_iteration):
            if i < n_init:
                phase_costs[0] += cost
                phase_iters[0] += 1
            elif i < n_fast_phase_end:
                phase_costs[1] += cost
                phase_iters[1] += 1
            else:
                phase_costs[2] += cost
                phase_iters[2] += 1
        
        colors_phase = ['orange', 'lightblue', 'lightcoral']
        bars = ax3.bar(phase_names, phase_costs, color=colors_phase, alpha=0.7, edgecolor='black', linewidth=2)
        ax3.set_ylabel('Total Computational Cost', fontsize=11, fontweight='bold')
        ax3.set_title('Cost Breakdown by Phase', fontsize=11, fontweight='bold')
        ax3.grid(True, alpha=0.3, axis='y')
        
        # Add iteration counts and percentages
        for bar, cost, n_iter in zip(bars, phase_costs, phase_iters):
            height = bar.get_height()
            pct = cost / sum(phase_costs) * 100
            ax3.text(bar.get_x() + bar.get_width()/2., height,
                    f'{cost:.1f}\n({n_iter} iters)\n{pct:.1f}%',
                    ha='center', va='bottom', fontsize=9, fontweight='bold')
        
        # ========== Panel 4: Improvements Found by Phase ==========
        ax4 = fig.add_subplot(gs[1, 1])
        
        # Count new bests found in each phase - compare to baseline at SAME fidelity
        best_found_phase = [0, 0, 0]
        
        # Track best at each fidelity level separately
        best_30d = self.optimizer.baseline_loss_by_fidelity.get('FAST (30d)', np.inf)
        best_180d = self.optimizer.baseline_loss_by_fidelity.get('FULL (180d)', np.inf)
        
        n_fast_phase_end = (4 * N_PARAMS) + (2 * N_PARAMS)  # End of fast phase
        
        for i, loss in enumerate(y_samples):
            if not np.isfinite(loss):
                continue
            
            _, fidelity_desc, _ = get_adaptive_sim_days(i)
            
            # Compare against baseline at same fidelity
            if fidelity_desc == 'FAST (30d)':
                if loss < best_30d:
                    best_30d = loss
                    # Classify into phase
                    if i < n_init:
                        best_found_phase[0] += 1
                    else:
                        best_found_phase[1] += 1
            elif fidelity_desc == 'FULL (180d)':
                if loss < best_180d:
                    best_180d = loss
                    best_found_phase[2] += 1
        
        bars2 = ax4.bar(phase_names, best_found_phase, color=colors_phase, alpha=0.7, 
                       edgecolor='black', linewidth=2)
        ax4.set_ylabel('Number of Improvements Found', fontsize=11, fontweight='bold')
        ax4.set_title('Improvements Discovered by Phase\n(vs baseline at same fidelity)', fontsize=11, fontweight='bold')
        ax4.grid(True, alpha=0.3, axis='y')
        
        for bar, count in zip(bars2, best_found_phase):
            height = bar.get_height()
            if height > 0:
                ax4.text(bar.get_x() + bar.get_width()/2., height,
                        f'{int(count)}', ha='center', va='bottom', 
                        fontsize=12, fontweight='bold')
        
        # ========== Panel 5: Efficiency Summary ==========
        ax5 = fig.add_subplot(gs[1, 2])
        ax5.axis('off')
        
        # Calculate summary statistics
        total_simulations = len(y_samples)
        total_cost_units = cumulative_cost[-1] if cumulative_cost else 0
        avg_cost_per_iter = total_cost_units / total_simulations if total_simulations > 0 else 0
        
        if self.optimizer.baseline_loss:
            total_improvement = self.optimizer.baseline_loss - self.optimizer.best_loss
            improvement_pct = total_improvement / self.optimizer.baseline_loss * 100
            cost_per_pct_improvement = total_cost_units / improvement_pct if improvement_pct > 0 else np.inf
        else:
            total_improvement = 0
            improvement_pct = 0
            cost_per_pct_improvement = np.inf
        
        # Phase efficiency
        phase_efficiency = []
        for i in range(3):
            if phase_costs[i] > 0 and best_found_phase[i] > 0:
                eff = best_found_phase[i] / phase_costs[i]
                phase_efficiency.append(eff)
            else:
                phase_efficiency.append(0)
        
        summary_lines = [
            "COMPUTATIONAL EFFICIENCY SUMMARY",
            "=" * 35,
            "",
            f"Total Iterations: {total_simulations}",
            f"Total Cost: {total_cost_units:.1f} units",
            f"  (1 unit = one 30-day simulation)",
            f"Avg Cost/Iter: {avg_cost_per_iter:.2f} units",
            "",
            f"Total Improvement: {improvement_pct:.1f}%",
            f"Cost per 1% Improvement: {cost_per_pct_improvement:.2f} units",
            "",
            "Phase Efficiency (improvements/cost):",
            f"  Initial: {phase_efficiency[0]:.3f}",
            f"  Fast (30d): {phase_efficiency[1]:.3f}",
            f"  Full (180d): {phase_efficiency[2]:.3f}",
            "",
            "INTERPRETATION:",
            "• Higher efficiency = more improvements",
            "  per computational cost",
            "• Fast phase should have high efficiency",
            "• Full phase validates with precision",
            "• Improvements in Full phase are vs",
            "  the 180d baseline (not 30d best)",
        ]
        
        summary_text = '\n'.join(summary_lines)
        ax5.text(0.05, 0.95, summary_text, transform=ax5.transAxes,
                fontsize=9, verticalalignment='top', family='monospace',
                bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))
        
        plt.suptitle('Computational Efficiency Analysis', 
                    fontsize=16, fontweight='bold', y=0.995)
        
        plt.savefig(save_path, bbox_inches='tight', dpi=300)
        print(f"✓ Saved efficiency analysis: {save_path}")
        plt.close()
    
    def create_all_plots(self):
        """Generate all visualization plots"""
        print("\n" + "="*70)
        print("GENERATING VISUALIZATION SUITE")
        print("="*70)
        
        self.plot_comprehensive_analysis()
        self.plot_parameter_sensitivity_heatmap()
        self.plot_computational_efficiency()
        
        print("="*70)
        print("✓ All visualizations complete!")
        print("  - optimization_analysis.png: Loss curves, parameters, trust region")
        print("  - parameter_sensitivity.png: Which parameters matter most")
        print("  - computational_efficiency.png: Cost vs improvement analysis")
        print("="*70)

# ============================================================================
# 3-WAY COMPARISON VISUALIZATION
# ============================================================================

def create_three_way_comparison(highres_results, lowres_default_results, lowres_optimized_results, 
                                 save_path='three_way_comparison.png'):
    """
    Enhanced 3-way comparison with error differences and improvement maps
    Shows spatial fields, errors, and where optimization improved performance
    NOTE: Uses last 30 days for all (assumes all are 180-day runs)
    """
    print("\n" + "="*70)
    print("GENERATING ENHANCED 3-WAY COMPARISON")
    print("="*70)
    
    # Compute losses with FIXED window (last 30 days) for fair comparison
    loss_default, fields_default = compute_loss(lowres_default_results, highres_results, 
                                                n_days_avg=30, return_fields=True, adaptive_window=False)
    loss_optimized, fields_optimized = compute_loss(lowres_optimized_results, highres_results, 
                                                    n_days_avg=30, return_fields=True, adaptive_window=False)
    
    # Calculate improvement
    improvement_pct = (loss_default - loss_optimized) / loss_default * 100
    
    # Helper function for consistent contourf plotting
    def add_contourf(ax, data, levels, cmap, title=None, colorbar=True):
        cf = ax.contourf(data, levels=levels, cmap=cmap, extend='both', origin='lower')
        if title:
            ax.set_title(title, fontsize=10, fontweight='bold')
        ax.axis('off')
        if colorbar:
            plt.colorbar(cf, ax=ax, fraction=0.046, pad=0.04)
        return cf
    
    # Create figure/grid - 4 rows x 3 columns
    fig = plt.figure(figsize=(20, 16))
    gs = gridspec.GridSpec(4, 3, figure=fig, hspace=0.3, wspace=0.25)
    
    # ========== Row 1: Potential Vorticity Fields ==========
    q_ref = fields_default['q_bt_hr_coarse']
    qmin, qmax = float(np.nanmin(q_ref)), float(np.nanmax(q_ref))
    q_levels = np.linspace(qmin, qmax, 31)
    
    ax1 = fig.add_subplot(gs[0, 0])
    add_contourf(ax1, q_ref, q_levels, 'RdBu_r', 'High-Res Reference\nPotential Vorticity')
    
    ax2 = fig.add_subplot(gs[0, 1])
    cf2 = add_contourf(ax2, fields_default['q_bt_lr'], q_levels, 'RdBu_r',
                       f'DEFAULT (Loss: {loss_default:.4f})')
    ax2.title.set_color('darkred')
    
    ax3 = fig.add_subplot(gs[0, 2])
    cf3 = add_contourf(ax3, fields_optimized['q_bt_lr'], q_levels, 'RdBu_r',
                       f'OPTIMIZED (Loss: {loss_optimized:.4f})')
    ax3.title.set_color('darkgreen')
    
    # ========== Row 2: PV Errors + Improvement Map ==========
    error_default_pv = np.abs(fields_default['q_bt_lr'] - fields_default['q_bt_hr_coarse'])
    error_optimized_pv = np.abs(fields_optimized['q_bt_lr'] - fields_optimized['q_bt_hr_coarse'])
    errmax_pv = float(max(np.nanmax(error_default_pv), np.nanmax(error_optimized_pv)))
    err_levels_pv = np.linspace(0.0, errmax_pv, 31)
    
    ax4 = fig.add_subplot(gs[1, 0])
    add_contourf(ax4, error_default_pv, err_levels_pv, 'Reds',
                 f'DEFAULT Error\nNRMSE: {fields_default["loss_q_bt"]:.4f}')
    
    ax5 = fig.add_subplot(gs[1, 1])
    add_contourf(ax5, error_optimized_pv, err_levels_pv, 'Reds',
                 f'OPTIMIZED Error\nNRMSE: {fields_optimized["loss_q_bt"]:.4f}')
    
    # Improvement map: negative values = optimized is better (green), positive = worse (red)
    ax6 = fig.add_subplot(gs[1, 2])
    error_diff_pv = error_optimized_pv - error_default_pv
    diff_max = float(max(abs(np.nanmin(error_diff_pv)), abs(np.nanmax(error_diff_pv))))
    diff_levels = np.linspace(-diff_max, diff_max, 31)
    cf_diff = add_contourf(ax6, error_diff_pv, diff_levels, 'RdYlGn_r',
                          f'Error Difference (Opt - Def)\nGreen = Improvement')
    
    # ========== Row 3: Streamfunction Fields ==========
    psi_ref = fields_default['psi_bt_hr_coarse']
    psi_absmax = float(np.nanmax(np.abs(psi_ref)))
    psi_levels = np.linspace(-psi_absmax, psi_absmax, 41)
    
    ax7 = fig.add_subplot(gs[2, 0])
    add_contourf(ax7, psi_ref, psi_levels, 'RdBu_r', 'High-Res Reference\nStreamfunction')
    
    ax8 = fig.add_subplot(gs[2, 1])
    add_contourf(ax8, fields_default['psi_bt_lr'], psi_levels, 'RdBu_r', 'DEFAULT')
    
    ax9 = fig.add_subplot(gs[2, 2])
    add_contourf(ax9, fields_optimized['psi_bt_lr'], psi_levels, 'RdBu_r', 'OPTIMIZED')
    
    # ========== Row 4: Streamfunction Errors + Improvement Map ==========
    error_default_psi = np.abs(fields_default['psi_bt_lr'] - fields_default['psi_bt_hr_coarse'])
    error_optimized_psi = np.abs(fields_optimized['psi_bt_lr'] - fields_optimized['psi_bt_hr_coarse'])
    errmax_psi = float(max(np.nanmax(error_default_psi), np.nanmax(error_optimized_psi)))
    err_levels_psi = np.linspace(0.0, errmax_psi, 31)
    
    ax10 = fig.add_subplot(gs[3, 0])
    add_contourf(ax10, error_default_psi, err_levels_psi, 'Reds',
                 f'DEFAULT Error\nNRMSE: {fields_default["loss_psi_bt"]:.4f}')
    
    ax11 = fig.add_subplot(gs[3, 1])
    add_contourf(ax11, error_optimized_psi, err_levels_psi, 'Reds',
                 f'OPTIMIZED Error\nNRMSE: {fields_optimized["loss_psi_bt"]:.4f}')
    
    # Improvement map for streamfunction
    ax12 = fig.add_subplot(gs[3, 2])
    error_diff_psi = error_optimized_psi - error_default_psi
    diff_max_psi = float(max(abs(np.nanmin(error_diff_psi)), abs(np.nanmax(error_diff_psi))))
    diff_levels_psi = np.linspace(-diff_max_psi, diff_max_psi, 31)
    add_contourf(ax12, error_diff_psi, diff_levels_psi, 'RdYlGn_r',
                 f'Error Difference (Opt - Def)\nGreen = Improvement')
    
    # Add overall summary text box
    summary_text = (
        f"OVERALL IMPROVEMENT: {improvement_pct:+.1f}%\n"
        f"━━━━━━━━━━━━━━━━━━━━━━━━━━━\n"
        f"Total Loss:  {loss_default:.4f} → {loss_optimized:.4f}\n"
        f"PV Loss:     {fields_default['loss_q_bt']:.4f} → {fields_optimized['loss_q_bt']:.4f} ({(fields_default['loss_q_bt']-fields_optimized['loss_q_bt'])/fields_default['loss_q_bt']*100:+.1f}%)\n"
        f"Ψ Loss:      {fields_default['loss_psi_bt']:.4f} → {fields_optimized['loss_psi_bt']:.4f} ({(fields_default['loss_psi_bt']-fields_optimized['loss_psi_bt'])/fields_default['loss_psi_bt']*100:+.1f}%)\n"
        f"━━━━━━━━━━━━━━━━━━━━━━━━━━━\n"
        f"Green regions = Optimized better\n"
        f"Red regions = Optimized worse"
    )
    
    fig.text(0.5, 0.01, summary_text, ha='center', va='bottom', fontsize=10, 
             family='monospace', bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.9))
    
    plt.suptitle('Enhanced 3-Way Comparison: Spatial Fields & Error Analysis',
                 fontsize=16, fontweight='bold', y=0.995)
    
    plt.savefig(save_path, bbox_inches='tight', dpi=300)
    print(f"✓ Saved enhanced 3-way comparison: {save_path}")
    plt.close()
    
    # Print detailed summary
    print("\n" + "="*70)
    print("COMPARISON SUMMARY (All using last 30 days)")
    print("="*70)
    print(f"High-Res (Reference):")
    print(f"  Resolution: {highres_results['config']['nx']}x{highres_results['config']['ny']}")
    print(f"  Time window: last 30 days (equilibrated state)")
    print(f"\nLow-Res DEFAULT:")
    print(f"  Resolution: {lowres_default_results['config']['nx']}x{lowres_default_results['config']['ny']}")
    print(f"  PV Loss: {fields_default['loss_q_bt']:.6f}")
    print(f"  Streamfn Loss: {fields_default['loss_psi_bt']:.6f}")
    print(f"  Total Loss: {loss_default:.6f}")
    print(f"\nLow-Res OPTIMIZED:")
    print(f"  Resolution: {lowres_optimized_results['config']['nx']}x{lowres_optimized_results['config']['ny']}")
    print(f"  PV Loss: {fields_optimized['loss_q_bt']:.6f} ({(fields_default['loss_q_bt']-fields_optimized['loss_q_bt'])/fields_default['loss_q_bt']*100:+.1f}%)")
    print(f"  Streamfn Loss: {fields_optimized['loss_psi_bt']:.6f} ({(fields_default['loss_psi_bt']-fields_optimized['loss_psi_bt'])/fields_default['loss_psi_bt']*100:+.1f}%)")
    print(f"  Total Loss: {loss_optimized:.6f}")
    
    # Spatial improvement statistics
    pv_improvements = error_default_pv - error_optimized_pv
    psi_improvements = error_default_psi - error_optimized_psi
    
    print(f"\nSpatial Improvement Statistics:")
    print(f"  PV Error Reduction:")
    print(f"    Mean: {np.mean(pv_improvements):.6f}")
    print(f"    Median: {np.median(pv_improvements):.6f}")
    print(f"    % improved points: {100*np.sum(pv_improvements > 0)/pv_improvements.size:.1f}%")
    print(f"  Streamfunction Error Reduction:")
    print(f"    Mean: {np.mean(psi_improvements):.6f}")
    print(f"    Median: {np.median(psi_improvements):.6f}")
    print(f"    % improved points: {100*np.sum(psi_improvements > 0)/psi_improvements.size:.1f}%")
    
    print(f"\n{Colors.star(f'TOTAL IMPROVEMENT: {improvement_pct:.1f}%')}")
    print("="*70)

# Enhanced GP Optimizer with visualization and warm-start
class EnhancedGPOptimizer:
    """Enhanced GP with visualization, warm-start, and 3-way comparison"""
    
    def __init__(self, n_initial_samples=None, random_seed=42):
        # Dynamic initial samples based on parameters (4x for good coverage)
        self.n_initial_samples = n_initial_samples if n_initial_samples is not None else (4 * N_PARAMS)
        self.random_seed = random_seed  # Store for reproducibility
        self.X_samples, self.y_samples, self.detailed_outputs = [], [], []
        self.best_loss, self.best_params, self.best_iteration = np.inf, None, -1
        self.iteration, self.iterations_without_improvement = 0, 0
        self.stagnation_threshold = 15
        self.gp = EnsembleGP(n_models=8)
        self.trust_region = TrustRegion()
        self.importance_history = []
        self.use_thompson_sampling_prob = 0.1
        self.baseline_loss = None  # Track default params loss at FINAL fidelity
        self.baseline_loss_by_fidelity = {}  # Track baseline for each fidelity level
        self.default_results = None  # Store default results for comparison
        self.current_fidelity = None  # Track current fidelity level
        self.best_params_original_iteration = None  # Track when best params were first discovered
        
        # Set numpy random seed for reproducibility
        np.random.seed(random_seed)
    
    def optimize(self, config_base, highres_results, max_iterations=100):
        print("\n" + "="*70)
        print("ENHANCED GP: WARM-START + DYNAMIC 2-LEVEL MULTI-FIDELITY + VISUALIZATION")
        print("="*70)
        print("Features:")
        print("  ✓ Warm-start from reference parameters")
        print("  ✓ 8-model weighted ensemble")
        print("  ✓ Local penalization (space coverage)")
        print(f"  ✓ Dynamic 2-level multi-fidelity strategy (based on {N_PARAMS} params):")
        print(f"    • Initial phase: {4*N_PARAMS} iterations at 30-day fidelity")
        print(f"    • Fast phase: {2*N_PARAMS} iterations at 30-day fidelity")
        print(f"    • Full phase: {2*N_PARAMS}+ iterations at 180-day fidelity")
        print("  ✓ Adaptive time windows for fair comparison:")
        print("    • 30-day runs: compare entire simulation (days 0-30)")
        print("    • 180-day runs: compare last 30 days (equilibrated)")
        print("  ✓ Fidelity-aware baseline tracking")
        print("  ✓ Thompson sampling (10% exploration)")
        print("  ✓ Anti-stagnation (auto-restart)")
        print("  ✓ 3-way comparison visualization")
        print(f"  Max iterations: {max_iterations}")
        print("="*70)
        
        # Phase 1: Initial sampling with WARM-START
        n_existing = len(self.X_samples)
        if n_existing < self.n_initial_samples:
            print(f"\n{'='*70}\nPHASE 1: WARM-START INITIALIZATION (seed={self.random_seed})\n{'='*70}")
            initial_samples = generate_smart_initial_samples(self.n_initial_samples, 
                                                            include_default=True,
                                                            base_seed=self.random_seed)
            
            for i, params in enumerate(initial_samples[n_existing:]):
                iter_num = i + n_existing
                is_default = (iter_num == 0 and n_existing == 0)  # First sample is default
                
                print(f"\n[Initial {iter_num+1}/{self.n_initial_samples}]" + 
                      (Colors.star(" DEFAULT PARAMETERS") if is_default else ""))
                
                loss, results, detailed = run_lowres_with_params(
                    params, config_base, highres_results, iteration=iter_num
                )
                self.X_samples.append(params)
                self.y_samples.append(loss)
                self.detailed_outputs.append(detailed)
                
                # Track baseline from default params
                if is_default and np.isfinite(loss):
                    _, fidelity_desc, _ = get_adaptive_sim_days(iter_num)
                    self.current_fidelity = fidelity_desc
                    self.baseline_loss = loss
                    self.baseline_loss_by_fidelity[fidelity_desc] = loss
                    self.default_results = results
                    print(Colors.cyan(f"  → Baseline loss at {fidelity_desc}: {loss:.6f}"))
                
                if np.isfinite(loss) and loss < self.best_loss:
                    self.best_loss, self.best_params = loss, params.copy()
                    self.best_iteration, self.iterations_without_improvement = len(self.X_samples) - 1, 0
                    self.best_params_original_iteration = iter_num  # Track discovery iteration
                    if is_default:
                        print(Colors.star(f"BASELINE SET: {Colors.green(f'{loss:.6f}')}"))
                    else:
                        # Compare to baseline at SAME fidelity
                        _, fidelity_desc, _ = get_adaptive_sim_days(iter_num)
                        fidelity_baseline = self.baseline_loss_by_fidelity.get(fidelity_desc, self.baseline_loss)
                        if fidelity_baseline:
                            improvement = (fidelity_baseline - loss) / fidelity_baseline * 100
                            print(Colors.star(f"NEW BEST: {Colors.green(f'{loss:.6f}')} ({improvement:+.1f}% vs baseline @ {fidelity_desc})"))
                        else:
                            print(Colors.star(f"NEW BEST: {Colors.green(f'{loss:.6f}')}"))
                self.save_progress()
        
        # Phase 2: Bayesian optimization
        print(f"\n{'='*70}\nPHASE 2: BAYESIAN OPTIMIZATION\n{'='*70}")
        
        for iteration in range(len(self.X_samples), max_iterations):
            self.iteration, self.iterations_without_improvement = iteration, self.iterations_without_improvement + 1
            
            print(f"\n{'='*70}\n{Colors.cyan(f'ITERATION {iteration + 1}/{max_iterations}')}\n{'='*70}")
            
            # Check if fidelity level changed - if so, re-evaluate baseline AND best params
            _, fidelity_desc, _ = get_adaptive_sim_days(iteration)
            if fidelity_desc != self.current_fidelity:
                old_fidelity = self.current_fidelity
                self.current_fidelity = fidelity_desc
                print(Colors.red(f"\n{'='*70}"))
                print(Colors.red(f"⚠ FIDELITY TRANSITION: {old_fidelity} → {fidelity_desc}"))
                print(Colors.red(f"{'='*70}"))
                
                # Re-evaluate baseline at new fidelity if not already done
                if fidelity_desc not in self.baseline_loss_by_fidelity:
                    print(Colors.cyan(f"→ Re-evaluating BASELINE at {fidelity_desc} fidelity..."))
                    default_array = params_dict_to_array(DEFAULT_PARAMS)
                    baseline_loss, baseline_results, _ = run_lowres_with_params(
                        default_array, config_base, highres_results, iteration=iteration
                    )
                    if np.isfinite(baseline_loss):
                        self.baseline_loss_by_fidelity[fidelity_desc] = baseline_loss
                        self.baseline_loss = baseline_loss
                        # Store the results if this is the final fidelity
                        if fidelity_desc == 'FULL (180d)':
                            self.default_results = baseline_results
                        print(Colors.cyan(f"→ Baseline at {fidelity_desc}: {baseline_loss:.6f}"))
                    else:
                        print(Colors.red(f"→ Baseline evaluation failed at {fidelity_desc}"))
                
                # CRITICAL: Re-evaluate current best parameters at new fidelity!
                if self.best_params is not None:
                    print(Colors.yellow(f"\n→ Re-evaluating BEST PARAMETERS at {fidelity_desc} fidelity..."))
                    print(Colors.yellow(f"   Old best loss ({old_fidelity}): {self.best_loss:.6f}"))
                    
                    best_loss_new_fidelity, _, _ = run_lowres_with_params(
                        self.best_params, config_base, highres_results, iteration=iteration
                    )
                    
                    if np.isfinite(best_loss_new_fidelity):
                        old_best = self.best_loss
                        self.best_loss = best_loss_new_fidelity
                        print(Colors.yellow(f"   New best loss ({fidelity_desc}): {best_loss_new_fidelity:.6f}"))
                        
                        # Calculate change
                        change_pct = (best_loss_new_fidelity - old_best) / old_best * 100
                        if change_pct > 0:
                            print(Colors.red(f"   ⚠ Loss INCREASED by {change_pct:.1f}% at higher fidelity"))
                        else:
                            print(Colors.green(f"   ✓ Loss decreased by {-change_pct:.1f}% at higher fidelity"))
                        
                        # Compare to new baseline
                        if fidelity_desc in self.baseline_loss_by_fidelity:
                            improvement = (self.baseline_loss_by_fidelity[fidelity_desc] - best_loss_new_fidelity) / \
                                        self.baseline_loss_by_fidelity[fidelity_desc] * 100
                            print(Colors.cyan(f"   → Improvement vs {fidelity_desc} baseline: {improvement:+.1f}%"))
                    else:
                        print(Colors.red(f"   ✗ Re-evaluation failed, keeping old best loss"))
                
                print(Colors.red(f"{'='*70}\n"))
            
            # Stagnation check
            if self.iterations_without_improvement >= self.stagnation_threshold:
                print(Colors.red(f"\n⚠ STAGNATION: {self.iterations_without_improvement} iterations w/o improvement"))
                print(Colors.yellow("→ Triggering exploration restart"))
                self.trigger_exploration_restart()
            
            # Fit GP
            X_warped = np.array([warp_parameters(x) for x in self.X_samples])
            y_array = np.array(self.y_samples)
            valid_mask = np.isfinite(y_array)
            n_valid = np.sum(valid_mask)
            
            print(f"Valid samples: {Colors.cyan(str(n_valid))}/{len(y_array)}")
            
            kappa = self.get_adaptive_kappa()
            if kappa > 2.0:
                print(Colors.yellow(f"  ℹ Increased exploration: kappa = {kappa:.1f}"))
            
            # Thompson sampling with some probability
            use_thompson = np.random.rand() < self.use_thompson_sampling_prob
            
            if n_valid < 5:
                print(Colors.yellow("  ⚠ Too few valid samples, random exploration"))
                next_params = unwarp_parameters(np.random.uniform(0, 1, N_PARAMS))
            elif use_thompson:
                print(Colors.cyan("  → Using Thompson sampling for exploration"))
                X_valid, y_valid = X_warped[valid_mask], y_array[valid_mask]
                self.gp.fit(X_valid, y_valid)
                
                # Track parameter importance even with Thompson sampling
                importance = self.gp.get_parameter_importance()
                self.importance_history.append(importance)
                
                # Print top 3 most important parameters
                sorted_indices = np.argsort(importance)[::-1][:3]
                print("  Top 3 important parameters:")
                for rank, idx in enumerate(sorted_indices, 1):
                    print(f"    {rank}. {PARAM_NAMES[idx]}: {importance[idx]:.3f}")
                
                tr_bounds = self.trust_region.get_trust_region_bounds()
                thompson_sample = thompson_sampling(self.gp, tr_bounds, n_samples=1)[0]
                next_params = unwarp_parameters(thompson_sample)
            else:
                X_valid, y_valid = X_warped[valid_mask], y_array[valid_mask]
                print("  Fitting 8-model ensemble GP...")
                self.gp.fit(X_valid, y_valid)
                
                # Track parameter importance
                importance = self.gp.get_parameter_importance()
                self.importance_history.append(importance)
                
                # Print parameter importance (show relative values)
                print("  Parameter importance (relative):")
                sorted_indices = np.argsort(importance)[::-1]  # Sort descending
                for rank, idx in enumerate(sorted_indices, 1):
                    name = PARAM_NAMES[idx]
                    imp_val = importance[idx]
                    if rank <= 3:
                        imp_str = f"{Colors.green('HIGH')}"
                    elif rank <= 5:
                        imp_str = f"{Colors.cyan('med')}"
                    else:
                        imp_str = "low"
                    print(f"    {rank}. {name}: {imp_val:.3f} ({imp_str})")
                
                # Update trust region
                if self.best_params is not None:
                    self.trust_region.best_center = warp_parameters(self.best_params)
                
                tr_bounds = self.trust_region.get_trust_region_bounds()
                print(f"  Trust region: {Colors.cyan(f'{self.trust_region.trust_radius:.2f}')}")
                
                # Optimize acquisition
                print(f"  Optimizing acquisition (kappa={kappa:.1f})...")
                best_y = np.min(y_valid)
                acq_fn = lambda X: hybrid_acquisition_with_penalization(
                    X, self.gp, best_y, self.X_samples, xi=0.01, kappa=kappa, penalization_weight=0.3
                )
                
                next_params_warped = optimize_acquisition_multistart(acq_fn, tr_bounds, n_starts=20, n_random=1000)
                acq_val = acq_fn(next_params_warped.reshape(1, -1))[0]
                print(f"  Selected point (acq={Colors.cyan(f'{acq_val:.4f}')})")
                next_params = unwarp_parameters(next_params_warped)
            
            # Evaluate with adaptive fidelity
            loss, results, detailed = run_lowres_with_params(
                next_params, config_base, highres_results, iteration=iteration
            )
            self.X_samples.append(next_params)
            self.y_samples.append(loss)
            self.detailed_outputs.append(detailed)
            
            # Update best
            new_best = False
            if np.isfinite(loss) and loss < self.best_loss:
                self.best_loss, self.best_params = loss, next_params.copy()
                self.best_iteration, self.iterations_without_improvement = iteration, 0
                self.best_params_original_iteration = iteration  # Track discovery iteration
                new_best = True
                # Compare to baseline at CURRENT fidelity
                fidelity_baseline = self.baseline_loss_by_fidelity.get(self.current_fidelity, self.baseline_loss)
                if fidelity_baseline:
                    improvement = (fidelity_baseline - loss) / fidelity_baseline * 100
                    print(Colors.star(f"NEW BEST: {Colors.green(f'{loss:.6f}')} ({improvement:+.1f}% vs baseline @ {self.current_fidelity})"))
                else:
                    print(Colors.star(f"NEW BEST: {Colors.green(f'{loss:.6f}')}"))
            
            self.trust_region.update(new_best, warp_parameters(self.best_params) if new_best else None)
            self.print_status()
            self.save_progress()
            
            # Generate plots every 10 iterations
            if (iteration + 1) % 10 == 0:
                print("\n  Generating visualization...")
                visualizer = OptimizationVisualizer(self)
                visualizer.create_all_plots()
        
        # Final visualization
        print("\n" + "="*70)
        print("GENERATING FINAL VISUALIZATIONS")
        print("="*70)
        visualizer = OptimizationVisualizer(self)
        visualizer.create_all_plots()
        
        return self.get_best_params()
    
    def get_adaptive_kappa(self):
        """Adaptive kappa: higher when stuck"""
        if self.iterations_without_improvement < 6:
            return 2.0
        elif self.iterations_without_improvement < 10:
            return 3.0
        return 4.0
    
    def trigger_exploration_restart(self):
        """Reset trust region and add random sample"""
        self.trust_region.reset_for_exploration()
        print(Colors.yellow("  → Random sample will be added next"))
        self.iterations_without_improvement = 0
        print(Colors.green("  ✓ Restart complete"))
    
    def print_status(self):
        """Print status with fidelity-aware baseline comparison"""
        n_valid = np.sum(np.isfinite(self.y_samples))
        n_failed = len(self.y_samples) - n_valid
        print(f"\n{Colors.bold('Status:')}")
        print(f"  Valid: {Colors.cyan(str(n_valid))}/{len(self.y_samples)}")
        print(f"  Failed: {Colors.yellow(str(n_failed))}")
        
        # Show current fidelity
        if self.current_fidelity:
            print(f"  Current fidelity: {Colors.cyan(self.current_fidelity)}")
        
        # Show baselines for each fidelity
        if self.baseline_loss_by_fidelity:
            print(f"  Baselines by fidelity:")
            for fidelity, baseline in sorted(self.baseline_loss_by_fidelity.items()):
                print(f"    {fidelity}: {Colors.cyan(f'{baseline:.6f}')}")
        
        # Compare best to baseline at current fidelity
        if self.best_params_original_iteration is not None:
            print(f"  {Colors.bold('Best loss:')} {Colors.green(f'{self.best_loss:.6f}')} " +
                  Colors.cyan(f'(discovered at iteration {self.best_params_original_iteration + 1})'))
        else:
            print(f"  {Colors.bold('Best loss:')} {Colors.green(f'{self.best_loss:.6f}')} " +
                  Colors.cyan(f'(iteration {self.best_iteration + 1})'))
        
        if self.current_fidelity and self.current_fidelity in self.baseline_loss_by_fidelity:
            fidelity_baseline = self.baseline_loss_by_fidelity[self.current_fidelity]
            improvement = (fidelity_baseline - self.best_loss) / fidelity_baseline * 100
            print(f"    → vs {self.current_fidelity} baseline: {Colors.green(f'{improvement:+.1f}%')}")
        
        stag_str = f"{self.iterations_without_improvement}/{self.stagnation_threshold}"
        stag_str = Colors.yellow(stag_str) if self.iterations_without_improvement >= 10 else Colors.cyan(stag_str)
        print(f"  Iterations w/o improvement: {stag_str}")
    
    def get_best_params(self):
        if self.best_params is None:
            raise ValueError("No valid parameters found!")
        return {PARAM_NAMES[i]: float(self.best_params[i]) for i in range(N_PARAMS)}
    
    def save_progress(self, filename='enhanced_gp_progress.pkl'):
        data = {
            'X_samples': self.X_samples, 'y_samples': self.y_samples, 'detailed_outputs': self.detailed_outputs,
            'best_loss': self.best_loss, 'best_params': self.best_params, 'best_iteration': self.best_iteration,
            'best_params_original_iteration': self.best_params_original_iteration,
            'iteration': self.iteration, 'iterations_without_improvement': self.iterations_without_improvement,
            'trust_region_state': {'radius': self.trust_region.trust_radius, 'center': self.trust_region.best_center,
                                  'success_count': self.trust_region.success_count, 'fail_count': self.trust_region.fail_count},
            'importance_history': self.importance_history,
            'baseline_loss': self.baseline_loss,
            'baseline_loss_by_fidelity': self.baseline_loss_by_fidelity,
            'current_fidelity': self.current_fidelity,
            'default_results': self.default_results,
            'random_seed': self.random_seed
        }
        with open(filename, 'wb') as f:
            pickle.dump(data, f)
        print(f"  ✓ Progress saved")
    
    @classmethod
    def load_progress(cls, filename='enhanced_gp_progress.pkl'):
        with open(filename, 'rb') as f:
            data = pickle.load(f)
        
        optimizer = cls(random_seed=data.get('random_seed', 42))
        optimizer.X_samples, optimizer.y_samples = data['X_samples'], data['y_samples']
        optimizer.detailed_outputs = data['detailed_outputs']
        optimizer.best_loss, optimizer.best_params = data['best_loss'], data['best_params']
        optimizer.best_iteration, optimizer.iteration = data['best_iteration'], data['iteration']
        optimizer.best_params_original_iteration = data.get('best_params_original_iteration', optimizer.best_iteration)
        optimizer.iterations_without_improvement = data.get('iterations_without_improvement', 0)
        optimizer.importance_history = data.get('importance_history', [])
        optimizer.baseline_loss = data.get('baseline_loss', None)
        optimizer.baseline_loss_by_fidelity = data.get('baseline_loss_by_fidelity', {})
        optimizer.current_fidelity = data.get('current_fidelity', None)
        optimizer.default_results = data.get('default_results', None)
        
        if 'trust_region_state' in data:
            tr = data['trust_region_state']
            optimizer.trust_region.trust_radius, optimizer.trust_region.best_center = tr['radius'], tr['center']
            optimizer.trust_region.success_count, optimizer.trust_region.fail_count = tr['success_count'], tr['fail_count']
        
        print(f"✓ Loaded checkpoint (seed={optimizer.random_seed}):")
        print(f"  Iterations: {len(optimizer.X_samples)}")
        n_valid = np.sum(np.isfinite(optimizer.y_samples))
        print(f"  Valid: {Colors.cyan(str(n_valid))}/{len(optimizer.y_samples)}")
        
        if optimizer.baseline_loss_by_fidelity:
            print(f"  Baselines by fidelity:")
            for fidelity, baseline in sorted(optimizer.baseline_loss_by_fidelity.items()):
                print(f"    {fidelity}: {Colors.cyan(f'{baseline:.6f}')}")
        
        print(f"  {Colors.bold('Best loss:')} {Colors.green(f'{optimizer.best_loss:.6f}')} " +
              Colors.cyan(f'(discovered at iteration {optimizer.best_params_original_iteration + 1})'))
        
        if optimizer.current_fidelity and optimizer.current_fidelity in optimizer.baseline_loss_by_fidelity:
            fidelity_baseline = optimizer.baseline_loss_by_fidelity[optimizer.current_fidelity]
            improvement = (fidelity_baseline - optimizer.best_loss) / fidelity_baseline * 100
            print(f"    → vs {optimizer.current_fidelity}: {Colors.green(f'{improvement:+.1f}%')}")
        
        return optimizer

# Main function with 3-way comparison
def main(checkpoint_file='enhanced_gp_progress.pkl', max_iterations=100, random_seed=42):
    """
    Main optimization routine with 3-way comparison
    
    Args:
        checkpoint_file: Path to checkpoint file for resuming
        max_iterations: Maximum number of optimization iterations
        random_seed: Random seed for reproducibility (affects initial sampling and exploration)
    """
    if not os.path.exists('highres_results.pkl'):
        print("\n✗ Error: highres_results.pkl not found!")
        return
    
    with open('highres_results.pkl', 'rb') as f:
        highres_results = pickle.load(f)
    print(f"\n✓ Loaded high-res: {highres_results['config']['nx']}x{highres_results['config']['ny']}")
    
    from main_comparison import config_lowres
    config_base = config_lowres.copy()
    
    if os.path.exists(checkpoint_file):
        print(f"\n✓ Checkpoint found")
        optimizer = EnhancedGPOptimizer.load_progress(checkpoint_file)
    else:
        print(f"\n✓ Starting new optimization (seed={random_seed})")
        print(f"  Initial samples: {4 * N_PARAMS} (4x parameters)")
        print(f"  Fast phase: {2 * N_PARAMS} iterations at 30-day fidelity")
        print(f"  Full phase: {2 * N_PARAMS}+ iterations at 180-day fidelity")
        optimizer = EnhancedGPOptimizer(random_seed=random_seed)  # Will use 4*N_PARAMS initial samples
    
    best_params = optimizer.optimize(config_base, highres_results, max_iterations)
    
    print("\n" + "="*70)
    print("OPTIMIZATION COMPLETE")
    print("="*70)
    
    n_valid = np.sum(np.isfinite(optimizer.y_samples))
    n_failed = len(optimizer.y_samples) - n_valid
    print(f"\nTotal iterations: {len(optimizer.y_samples)}")
    print(f"  Valid: {Colors.cyan(str(n_valid))}")
    print(f"  Failed: {Colors.yellow(str(n_failed))}")
    
    # Show all baselines
    if optimizer.baseline_loss_by_fidelity:
        print(f"\nBaselines by fidelity:")
        for fidelity, baseline in sorted(optimizer.baseline_loss_by_fidelity.items()):
            print(f"  {fidelity}: {Colors.cyan(f'{baseline:.6f}')}")
    
    # Final comparison at FULL fidelity
    final_baseline = optimizer.baseline_loss_by_fidelity.get('FULL (180d)', optimizer.baseline_loss)
    if final_baseline:
        improvement = (final_baseline - optimizer.best_loss) / final_baseline * 100
        print(f"\n{Colors.bold('Final Comparison at FULL (180d) Fidelity:')}")
        print(f"  Baseline (default): {Colors.cyan(f'{final_baseline:.6f}')}")
        print(f"  Best loss: {Colors.green(f'{optimizer.best_loss:.6f}')} " +
              Colors.green(f'[{improvement:+.1f}% improvement]'))
        
        # Show discovery info
        if optimizer.best_params_original_iteration is not None:
            print(f"  Best parameters discovered at: iteration {optimizer.best_params_original_iteration + 1}")
            n_fast_phase_end = (4 * N_PARAMS) + (2 * N_PARAMS)
            if optimizer.best_params_original_iteration < n_fast_phase_end:
                print(Colors.yellow(f"    (during 30-day fast exploration phase)"))
                print(Colors.cyan(f"    Loss was re-evaluated at full 180-day fidelity"))
            else:
                print(Colors.cyan(f"    (during 180-day full precision phase)"))
    else:
        print(f"\n{Colors.bold('Best loss:')} {Colors.green(f'{optimizer.best_loss:.6f}')}")
        print(Colors.yellow("  Note: No full-fidelity baseline available"))
    
    print(f"\n{Colors.bold('Best parameters:')}")
    for name, val in best_params.items():
        default_val = DEFAULT_PARAMS[name]
        change = (val - default_val) / default_val * 100 if default_val != 0 else 0
        print(f"  {name}: {Colors.cyan(f'{val:.6e}')} (default: {default_val:.6e}, {change:+.1f}%)")
    
    # Save results
    with open('enhanced_gp_optimal_params.pkl', 'wb') as f:
        pickle.dump(best_params, f)
    with open('enhanced_gp_optimal_config.txt', 'w') as f:
        f.write("'subgrid_params': {\n")
        for name, val in best_params.items():
            f.write(f"    '{name}': {val:.6e},\n")
        f.write("}\n")
    
    print("\n✓ Saved: enhanced_gp_optimal_params.pkl")
    print("✓ Saved: enhanced_gp_optimal_config.txt")
    
    print("\n" + "="*70)
    print("NOTE: DYNAMIC 2-LEVEL MULTI-FIDELITY WITH ADAPTIVE BASELINES")
    print("="*70)
    print(f"The optimizer uses a dynamic 2-level fidelity strategy (based on {N_PARAMS} params):")
    print(f"  Initial phase (iterations 0-{4*N_PARAMS-1}):  30-day runs at low fidelity")
    print(f"    - {4*N_PARAMS} samples for good initial coverage")
    print(f"    - Compares entire simulation (days 0-30)")
    print(f"    - Baseline tracked at 30-day fidelity")
    print(f"  Fast phase (iterations {4*N_PARAMS}-{4*N_PARAMS + 2*N_PARAMS - 1}):   30-day BO runs")
    print(f"    - {2*N_PARAMS} iterations of fast exploration")
    print(f"    - ~6x speedup vs full fidelity")
    print(f"  Full phase (iterations {4*N_PARAMS + 2*N_PARAMS}+):   180-day BO runs")
    print(f"    - {2*N_PARAMS}+ iterations at full precision")
    print("    - Compares last 30 days (equilibrated state)")
    print("    - Baseline tracked at 180-day fidelity")
    print("\nThis ensures:")
    print("  ✓ Good initial parameter space coverage")
    print("  ✓ Fast exploration in early BO iterations")
    print("  ✓ Fair apples-to-apples comparisons at each fidelity")
    print("  ✓ Final results use full 180-day simulations")
    print("\nSeed robustness:")
    print(f"  ✓ Random seed used: {optimizer.random_seed}")
    print("  ✓ Multiple complementary seeds used internally")
    print("  ✓ Small perturbations added to reduce grid artifacts")
    print("  ℹ Different seeds may find best at different iterations")
    print("    but final performance should be similar (~5-10% variation)")
    print("="*70)
    
    # Run final simulation with optimized parameters for 3-way comparison
    print("\n" + "="*70)
    print("RUNNING FINAL COMPARISON SIMULATIONS")
    print("="*70)
    
    # Ensure we have full-fidelity baseline (180 days)
    if 'FULL (180d)' not in optimizer.baseline_loss_by_fidelity or optimizer.default_results is None:
        print("\nRunning default parameters at FULL fidelity (180 days)...")
        config_default = config_base.copy()
        config_default['subgrid_params'] = DEFAULT_PARAMS
        from main_comparison import run_simulation
        optimizer.default_results = run_simulation(config_default, sim_days=180, save_interval_hours=12)
        
        # Compute baseline loss at full fidelity
        baseline_loss_full, _ = compute_loss(optimizer.default_results, highres_results, 
                                             n_days_avg=30, return_fields=False, adaptive_window=False)
        optimizer.baseline_loss_by_fidelity['FULL (180d)'] = baseline_loss_full
        optimizer.baseline_loss = baseline_loss_full
        print(f"  ✓ Default simulation complete - Loss: {baseline_loss_full:.6f}")
    else:
        print("\n✓ Using cached default results at FULL fidelity")
    
    # Run optimized parameters simulation (full 180 days for final comparison)
    print("\nRunning optimized parameters simulation (full 180 days)...")
    config_optimized = config_base.copy()
    config_optimized['subgrid_params'] = best_params
    from main_comparison import run_simulation
    optimized_results = run_simulation(config_optimized, sim_days=180, save_interval_hours=12)
    print(f"  ✓ Optimized simulation complete")
    
    # Create 3-way comparison
    create_three_way_comparison(highres_results, optimizer.default_results, optimized_results)
    
    print("\n✓ Saved: optimization_analysis.png")
    print("✓ Saved: parameter_sensitivity.png")
    print("✓ Saved: computational_efficiency.png")
    print("✓ Saved: three_way_comparison.png")
    
    print("\n" + "="*70)
    print("VISUALIZATION GUIDE")
    print("="*70)
    print("1. optimization_analysis.png")
    print("   → Loss evolution, parameter trajectories, trust region")
    print("   → Shows HOW the optimization progressed")
    print("")
    print("2. parameter_sensitivity.png")
    print("   → Correlation analysis, variance explained, ranges explored")
    print("   → Shows WHICH parameters matter most")
    print("   → Red = increasing parameter worsens loss")
    print("   → Green = increasing parameter improves loss")
    print("")
    print("3. computational_efficiency.png")
    print("   → Cost vs improvement, phase breakdown, sample efficiency")
    print("   → Shows HOW EFFICIENTLY we found improvements")
    print("")
    print("4. three_way_comparison.png")
    print("   → Spatial fields: high-res vs default vs optimized")
    print("   → Shows FINAL RESULTS quality")
    print("="*70)
    
    return optimizer, best_params

if __name__ == "__main__":
    # You can change the random_seed parameter to test different initializations
    # The optimizer uses multiple complementary seeds internally for robustness
    # n_initial_samples is now dynamic: 4 * N_PARAMS (will be 24 for 6 parameters)
    optimizer, best_params = main(max_iterations=50, random_seed=23)


✓ Loaded high-res: 512x256

✓ Checkpoint found
✓ Loaded checkpoint (seed=23):
  Iterations: 30
  Valid: 30/30
  Baselines by fidelity:
    FAST (30d): 0.206844
  Best loss: 0.102483 (discovered at iteration 28)
    → vs FAST (30d): +50.5%

ENHANCED GP: WARM-START + DYNAMIC 2-LEVEL MULTI-FIDELITY + VISUALIZATION
Features:
  ✓ Warm-start from reference parameters
  ✓ 8-model weighted ensemble
  ✓ Local penalization (space coverage)
  ✓ Dynamic 2-level multi-fidelity strategy (based on 6 params):
    • Initial phase: 24 iterations at 30-day fidelity
    • Fast phase: 12 iterations at 30-day fidelity
    • Full phase: 12+ iterations at 180-day fidelity
  ✓ Adaptive time windows for fair comparison:
    • 30-day runs: compare entire simulation (days 0-30)
    • 180-day runs: compare last 30 days (equilibrated)
  ✓ Fidelity-aware baseline tracking
  ✓ Thompson sampling (10% exploration)
  ✓ Anti-stagnation (auto-restart)
  ✓ 3-way comparison visualization
  Max iterations: 50

PHASE 2: BAYE

100%|██████████| 1440/1440 [00:04<00:00, 330.50it/s]



LowRes_64x32 Simulation Complete!
  → Using entire simulation (days 0-30) for loss
  Loss: 0.111881
  → Trust region shrunk to 0.25

Status:
  Valid: 31/31
  Failed: 0
  Current fidelity: FAST (30d)
  Baselines by fidelity:
    FAST (30d): 0.206844
  Best loss: 0.102483 (discovered at iteration 28)
    → vs FAST (30d) baseline: +50.5%
  Iterations w/o improvement: 3/15
  ✓ Progress saved

ITERATION 32/50
Valid samples: 31/31
  Fitting 8-model ensemble GP...
  Parameter importance (relative):
    1. energy_correction: 10.144 (HIGH)
    2. enstrophy_correction: 4.524 (HIGH)
    3. viscosity_scale: 1.601 (HIGH)
    4. drag_scale: 0.399 (med)
    5. smagorinsky_coeff: 0.019 (med)
    6. eddy_diffusivity: 0.019 (low)
  Trust region: 0.25
  Optimizing acquisition (kappa=2.0)...
  Selected point (acq=0.0000)

Testing parameters - Fidelity: FAST (30d)
  viscosity_scale: 3.203796e+00
  drag_scale: 7.684946e-01
  eddy_diffusivity: 3.992782e+04
  smagorinsky_coeff: 1.979382e-01
  energy_correcti

100%|██████████| 1440/1440 [00:04<00:00, 330.18it/s]



LowRes_64x32 Simulation Complete!
  → Using entire simulation (days 0-30) for loss
  Loss: 0.108828

Status:
  Valid: 32/32
  Failed: 0
  Current fidelity: FAST (30d)
  Baselines by fidelity:
    FAST (30d): 0.206844
  Best loss: 0.102483 (discovered at iteration 28)
    → vs FAST (30d) baseline: +50.5%
  Iterations w/o improvement: 4/15
  ✓ Progress saved

ITERATION 33/50
Valid samples: 32/32
  Fitting 8-model ensemble GP...
  Parameter importance (relative):
    1. energy_correction: 10.440 (HIGH)
    2. enstrophy_correction: 4.526 (HIGH)
    3. viscosity_scale: 1.619 (HIGH)
    4. drag_scale: 0.372 (med)
    5. eddy_diffusivity: 0.027 (med)
    6. smagorinsky_coeff: 0.027 (low)
  Trust region: 0.25
  Optimizing acquisition (kappa=2.0)...
  Selected point (acq=0.0000)

Testing parameters - Fidelity: FAST (30d)
  viscosity_scale: 3.206823e+00
  drag_scale: 8.428643e-01
  eddy_diffusivity: 2.754158e+04
  smagorinsky_coeff: 2.122382e-01
  energy_correction: 4.903970e-04
  enstrophy_cor

...


LowRes_64x32 Simulation Complete!
  → Using last 30 days for loss (equilibrated state)
  Loss: 0.137583

Status:
  Valid: 49/49
  Failed: 0
  Current fidelity: FULL (180d)
  Baselines by fidelity:
    FAST (30d): 0.206844
    FULL (180d): 0.628715
  Best loss: 0.127344 (discovered at iteration 41)
    → vs FULL (180d) baseline: +79.7%
  Iterations w/o improvement: 8/15
  ✓ Progress saved

ITERATION 50/50
Valid samples: 49/49
  ℹ Increased exploration: kappa = 3.0
  → Using Thompson sampling for exploration
  Top 3 important parameters:
    1. energy_correction: 24.333
    2. viscosity_scale: 7.930
    3. enstrophy_correction: 2.015

Testing parameters - Fidelity: FULL (180d)
  viscosity_scale: 2.887966e+00
  drag_scale: 9.642765e-01
  eddy_diffusivity: 4.134586e+04
  smagorinsky_coeff: 1.943351e-01
  energy_correction: -5.532048e-05
  enstrophy_correction: 1.305232e-09

Running LowRes_64x32 Simulation
Grid: 64 x 32
Resolution: 31.2 km per grid point

Subgrid Parameters:
  viscosity_sc

100%|██████████| 8640/8640 [00:26<00:00, 328.27it/s]



LowRes_64x32 Simulation Complete!
  → Using last 30 days for loss (equilibrated state)
  Loss: 0.137583
  → Trust region shrunk to 0.05

Status:
  Valid: 50/50
  Failed: 0
  Current fidelity: FULL (180d)
  Baselines by fidelity:
    FAST (30d): 0.206844
    FULL (180d): 0.628715
  Best loss: 0.127344 (discovered at iteration 41)
    → vs FULL (180d) baseline: +79.7%
  Iterations w/o improvement: 9/15
  ✓ Progress saved

  Generating visualization...

GENERATING VISUALIZATION SUITE

✓ Saved comprehensive analysis: optimization_analysis.png
✓ Saved sensitivity analysis: parameter_sensitivity.png
✓ Saved efficiency analysis: computational_efficiency.png
✓ All visualizations complete!
  - optimization_analysis.png: Loss curves, parameters, trust region
  - parameter_sensitivity.png: Which parameters matter most
  - computational_efficiency.png: Cost vs improvement analysis

GENERATING FINAL VISUALIZATIONS

GENERATING VISUALIZATION SUITE

✓ Saved comprehensive analysis: optimization_analys

100%|██████████| 8640/8640 [00:25<00:00, 337.18it/s]



LowRes_64x32 Simulation Complete!
  ✓ Optimized simulation complete

GENERATING ENHANCED 3-WAY COMPARISON
✓ Saved enhanced 3-way comparison: three_way_comparison.png

COMPARISON SUMMARY (All using last 30 days)
High-Res (Reference):
  Resolution: 512x256
  Time window: last 30 days (equilibrated state)

Low-Res DEFAULT:
  Resolution: 64x32
  PV Loss: 0.726491
  Streamfn Loss: 0.482050
  Total Loss: 0.628715

Low-Res OPTIMIZED:
  Resolution: 64x32
  PV Loss: 0.174910 (+75.9%)
  Streamfn Loss: 0.055995 (+88.4%)
  Total Loss: 0.127344

Spatial Improvement Statistics:
  PV Error Reduction:
    Mean: 0.000001
    Median: 0.000000
    % improved points: 84.2%
  Streamfunction Error Reduction:
    Mean: 14103.821286
    Median: 11611.008305
    % improved points: 93.2%

★ TOTAL IMPROVEMENT: 79.7%

✓ Saved: optimization_analysis.png
✓ Saved: parameter_sensitivity.png
✓ Saved: computational_efficiency.png
✓ Saved: three_way_comparison.png

VISUALIZATION GUIDE
1. optimization_analysis.png
   → 